<a href="https://colab.research.google.com/github/SamsJara/TFG_detector_hurtos/blob/main/TesisV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **CONECTAR A GOOGLE DRIVE**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Ruta real en tu Google Drive
origen = "/content/drive/MyDrive/ColabNotebooks/TesisV2"

# Ruta simbólica dentro de Colab
destino = "/content/tesisV2"

# Crear enlace simbólico (acceso directo)
if not os.path.exists(destino):
    os.symlink(origen, destino)

# Cambiar el directorio actual a esa carpeta
os.chdir(destino)


### **ARBOL**

In [ ]:
!apt install tree
!tree /content/tesis/



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tree is already the newest version (2.0.2-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
/content/tesis/runsPersona/
└── detect
    ├── predict
    │   └── hurto1.avi
    └── yolov8_personas_hurtos
        ├── args.yaml
        ├── BoxF1_curve.png
        ├── BoxP_curve.png
        ├── BoxPR_curve.png
        ├── BoxR_curve.png
        ├── confusion_matrix_normalized.png
        ├── confusion_matrix.png
        ├── labels_correlogram.jpg
        ├── labels.jpg
        ├── results.csv
        ├── results.png
        ├── train_batch0.jpg
        ├── train_batch1.jpg
        ├── train_batch2.jpg
        ├── train_batch33920.jpg
        ├── train_batch33921.jpg
        ├── train_batch33922.jpg
        ├── val_batch0_labels.jpg
        ├── val_batch0_pred.jpg
        ├── val_batch1_labels.jpg
        ├── val_batch1_pred.jpg
        ├── val_batch2_labels.jpg
        ├── val_b

In [ ]:
!rm -rf /content/sample_data/


In [ ]:
import os
import yaml

# Ruta base donde se va a crear el dataset_final
base_path = 'datasetv1'  # O podés usar 'content/tesis/dataset_final' si estás en Colab

# Crear carpetas necesarias
splits = ['train', 'valid', 'test']
for split in splits:
    os.makedirs(os.path.join(base_path, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(base_path, split, 'labels'), exist_ok=True)

# Crear archivo data.yaml
data_yaml = {
    'train': 'train/images',
    'val': 'valid/images',
    'test': 'test/images',
    'nc': 2,
    'names': ['normal', 'shoplifting']
}

with open(os.path.join(base_path, 'data.yaml'), 'w') as f:
    yaml.dump(data_yaml, f)

print(f"✅ Estructura creada en: {base_path}")


## ***DESCOMPRIMIR ZIP***

In [ ]:
!unzip -q /content/dataset3.zip -d  /content/tesis/dataset3

warning [/content/dataset3.zip]:  514762 extra bytes at beginning or within zipfile
  (attempting to process anyway)
file #1:  bad zipfile offset (local header sig):  514762
  (attempting to re-compensate)
error: not enough memory for bomb detection
file #2:  bad zipfile offset (local header sig):  895700260
  (attempting to re-compensate)


## ***YOLO, OPENCV, SYMPY***

In [ ]:
!pip install ultralytics opencv-python
!pip install -U sympy==1.12
import sympy
print(sympy.__version__)  # debería mostrar 1.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 56.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.2
    Uninstalling numpy-2.3.2:
      Successfully uninstalled numpy-2.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.
datasets 4.0.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.7.0 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 32.7 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.14.0
    Uninstalling sympy-1.14.0:
      Successfully uninstalled sympy-1.14.0


1.14.0


## ***PRUEBA: TRACKING DE PERSONAS CON YOLO, COCO Y BYTETRACK PARA ID***

In [ ]:
import os
import cv2
from ultralytics import YOLO
from collections import defaultdict
from datetime import datetime

SOURCE_VIDEO = "/content/tesisV2/videos/supermas1-1.mp4"   # <-- cambia esto
OUTPUT_DIR   = "/content/tesisV2/clips_personas"      # <-- cambia si querés
MODEL_PATH   = "yolov8s.pt"                         # o 'yolov8n.pt' para más FPS
CONF         = 0.25
CLIP_SECS    = 3           # duración de cada clip
IDLE_SECS    = 0.7         # si un ID no aparece por este tiempo, se cierra su writer
TRACKER_CFG  = "bytetrack.yaml"  # liviano y sin re-id

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Abrimos para conocer FPS y frame size
cap0 = cv2.VideoCapture(SOURCE_VIDEO)
if not cap0.isOpened():
    raise RuntimeError(f"No pude abrir el video: {SOURCE_VIDEO}")
fps = cap0.get(cv2.CAP_PROP_FPS) or 30.0
width  = int(cap0.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap0.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap0.release()

clip_len_frames = int(CLIP_SECS * fps)
idle_frames     = int(IDLE_SECS * fps)

# Estado por ID: writer actual, frames escritos en el segmento, índice de segmento, última vez visto
state = {}
# Para contar frames globales (necesario para medir idle)
frame_idx = 0

# Codec y helper para crear writer
fourcc = cv2.VideoWriter_fourcc(*"mp4v")

def ensure_writer_for_id(person_id):
    info = state.get(person_id)
    if info is None:
        person_dir = os.path.join(OUTPUT_DIR, f"person_{person_id:04d}")
        os.makedirs(person_dir, exist_ok=True)
        seg_idx = 0
        out_path = os.path.join(
            person_dir,
            f"id{person_id:04d}_seg{seg_idx:03d}.mp4"
        )
        writer = cv2.VideoWriter(out_path, fourcc, fps, (width, height))
        state[person_id] = {
            "writer": writer,
            "frames_in_seg": 0,
            "seg_idx": seg_idx,
            "last_seen": frame_idx
        }
    return state[person_id]

def rotate_segment(person_id):
    info = state[person_id]
    # Cerrar actual
    if info["writer"] is not None:
        info["writer"].release()
    # Abrir nuevo
    info["seg_idx"] += 1
    person_dir = os.path.join(OUTPUT_DIR, f"person_{person_id:04d}")
    out_path = os.path.join(
        person_dir,
        f"id{person_id:04d}_seg{info['seg_idx']:03d}.mp4"
    )
    info["writer"] = cv2.VideoWriter(out_path, fourcc, fps, (width, height))
    info["frames_in_seg"] = 0

def close_and_remove(person_id):
    info = state.get(person_id)
    if not info:
        return
    if info["writer"] is not None:
        info["writer"].release()
    del state[person_id]

# Cargar el modelo
model = YOLO(MODEL_PATH)

# Stream de tracking frame a frame
for result in model.track(
    source=SOURCE_VIDEO,
    stream=True,
    classes=[0],           # solo 'person'
    conf=CONF,
    tracker=TRACKER_CFG,
    persist=True
):
    frame = result.orig_img  # BGR
    ids = []
    if result.boxes is not None and result.boxes.id is not None:
        ids = result.boxes.id.int().tolist()

    # Escribir frame por cada ID detectado
    for pid in ids:
        info = ensure_writer_for_id(pid)
        info["writer"].write(frame)
        info["frames_in_seg"] += 1
        info["last_seen"] = frame_idx

        # rotar segmento cuando llegue a CLIP_SECS
        if info["frames_in_seg"] >= clip_len_frames:
            rotate_segment(pid)

    # Cerrar escritores de IDs que no aparecieron recientemente
    to_close = []
    for pid, info in list(state.items()):
        if frame_idx - info["last_seen"] > idle_frames:
            to_close.append(pid)
    for pid in to_close:
        close_and_remove(pid)

    frame_idx += 1

# Flush final por si quedó algo abierto
for pid in list(state.keys()):
    close_and_remove(pid)

print(f"Listo. Clips guardados en: {OUTPUT_DIR}")


requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 0.8s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


video 1/1 (frame 1/318) /content/tesisV2/videos/supermas1-1.mp4: 384x640 6 persons, 528.1ms
video 1/1 (frame 2/318) /content/tesisV2/videos/supermas1-1.mp4: 384x640 6 persons, 364.4ms
video 1/1 (frame 3/318) /content/tesisV2/videos/supermas1-1.mp4: 384x640 6 persons, 331.1ms
video 1/1 (frame 4/318) /content/tesisV2/videos/supermas1-1.mp4: 384x640 6 persons, 353.2ms
video 1/1 (frame 5/318) /content/tesisV2/videos/supermas1-1.mp4: 384x640 5 persons, 324.9ms
video 1/1 (frame 6/318) /content/tesisV2/videos/supermas1-1.mp4: 384x640 5 persons, 342.5ms
video 1/1 (frame 7/318) /content/tesisV2/videos/supermas1-1.mp4: 384x640 5 persons, 322.3ms
video 1/1 (frame 8/318) /content/tesisV2/videos/supermas1-1.mp4: 384x640 5 persons, 345.7ms
video 1/1 (frame 9/318) /content/tesisV2/video

## ***PRUEBA: CORRE YOLOv8 + TRACKER***

In [ ]:
!yolo track model=yolov8s.pt source="/content/tesisV2/videos/supermas1_2.mp4" classes=0 conf=0.25 save=True project="/content/tesisV2/outputs" name="boxed"


Ultralytics 8.3.176 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
YOLOv8s summary (fused): 72 layers, 11,156,544 parameters, 0 gradients, 28.6 GFLOPs

video 1/1 (frame 1/280) /content/tesisV2/videos/supermas1_2.mp4: 384x640 6 persons, 368.3ms
video 1/1 (frame 2/280) /content/tesisV2/videos/supermas1_2.mp4: 384x640 6 persons, 347.5ms
video 1/1 (frame 3/280) /content/tesisV2/videos/supermas1_2.mp4: 384x640 4 persons, 346.8ms
video 1/1 (frame 4/280) /content/tesisV2/videos/supermas1_2.mp4: 384x640 4 persons, 443.9ms
video 1/1 (frame 5/280) /content/tesisV2/videos/supermas1_2.mp4: 384x640 5 persons, 594.2ms
video 1/1 (frame 6/280) /content/tesisV2/videos/supermas1_2.mp4: 384x640 5 persons, 556.7ms
video 1/1 (frame 7/280) /content/tesisV2/videos/supermas1_2.mp4: 384x640 4 persons, 530.6ms
video 1/1 (frame 8/280) /content/tesisV2/videos/supermas1_2.mp4: 384x640 5 persons, 543.5ms
video 1/1 (frame 9/280) /content/tesisV2/videos/supermas1_2.mp4: 384x640 5 persons, 549.6ms
video 1

In [ ]:
!pip -q install transformers==4.43.3 timm==1.0.7 pytorchvideo==0.1.5 accelerate evaluate decord
import os, pandas as pd, torch, decord
from datasets import Dataset
from transformers import AutoImageProcessor, VideoMAEForVideoClassification, TrainingArguments, Trainer
decord.bridge.set_bridge('torch')

BASE = "/content/cortos/video_clips_224"  # el mismo OUT_DIR
label2id={"normal":0,"shoplifting":1}; id2label={v:k for k,v in label2id.items()}

def load_csv(name):
    df=pd.read_csv(os.path.join(BASE,f"{name}.csv"),header=None,names=["path","label"])
    return Dataset.from_pandas(df.assign(label=df["label"].map(label2id)))

def load_video_torch(path,num_frames=16):
    vr=decord.VideoReader(path)
    import torch
    idx=torch.linspace(0,len(vr)-1,steps=num_frames).long()
    return vr.get_batch(idx).permute(0,3,1,2)

train_ds, val_ds = load_csv("train"), load_csv("val")
processor=AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
model=VideoMAEForVideoClassification.from_pretrained(
    "MCG-NJU/videomae-base", num_labels=2, label2id=label2id, id2label=id2label
)

def preprocess(ex):
    frames=load_video_torch(ex["path"],num_frames=16)
    ex["pixel_values"]=processor(list(frames),return_tensors="pt")["pixel_values"][0]
    return ex

train_ds=train_ds.map(preprocess); val_ds=val_ds.map(preprocess)
def collate(b):
    return {"pixel_values":torch.stack([x["pixel_values"] for x in b]),
            "labels":torch.tensor([x["label"] for x in b])}

args=TrainingArguments(output_dir="/content/videomae_shoplift",
    per_device_train_batch_size=4, per_device_eval_batch_size=4,
    gradient_accumulation_steps=2, fp16=True, learning_rate=5e-5,
    num_train_epochs=5, evaluation_strategy="epoch", save_strategy="epoch",
    logging_steps=20, report_to="none")

trainer=Trainer(model=model,args=args,train_dataset=train_ds,eval_dataset=val_ds,data_collator=collate)
trainer.train()


## ***CREACION DE CLIPS NORMALES (SOLO CON PERSONAS)***

In [ ]:
# ================= SOLO NEGATIVOS (masivos) + CACHE + ESPEJO A DRIVE =================
import os, glob, random, subprocess, cv2, collections, numpy as np, json, time, torch
from tqdm import tqdm
from ultralytics import YOLO

# ---------- PARÁMETROS RÁPIDOS ----------
SRC_DIR = "/content/tesisV2/Datasets"                      # normal en: Training_/Testing_*
OUT_DIR = "/content/tesisV2/cortos/video_clips_192"        # donde ya están tus POS
DRIVE_MIRROR = "/content/drive/MyDrive/tesisV2/cortos/video_clips_192"  # espejo
N_NEG_VIDEOS = 150     # ← subí esto para más negativos (antes 50/100/120). Poné 150–300 si querés muchos.
HOP_NEG = 8.0          # ← bajá a 6.0 si querés aún más clips por video
SIZE, FPS = 192, 12
FFMPEG_PRESET = "veryfast"

PERSON_CONF = 0.25
NEG_PERSON_THR = 0.40
MAX_FRAMES = 60
VID_STRIDE = 3
IMGSZ = 384
DEVICE = 0 if (torch.cuda.is_available() and torch.cuda.device_count()>0) else "cpu"
SYNC_EVERY = 10        # copia a Drive cada X videos segmentados

print("Device:", "GPU" if DEVICE==0 else "CPU")

# ---------- RUTAS ----------
POS_DIR = os.path.join(OUT_DIR, "shoplifting")
NEG_DIR = os.path.join(OUT_DIR, "normal")
CACHE_DIR = os.path.join(OUT_DIR, "_cache_pr")
os.makedirs(NEG_DIR, exist_ok=True)
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(DRIVE_MIRROR, exist_ok=True)

# ---------- FUNCIONES ----------
def list_videos(folder):
    vids=[]; exts=("*.mp4","*.avi","*.mov","*.mkv","*.MP4","*.AVI","*.MOV","*.MKV")
    for e in exts: vids += glob.glob(os.path.join(folder, "**", e), recursive=True)
    return vids

def segment_to_clips_resume(src_path, out_dir, clip_s=3, hop_s=8.0, size=192, fps=12, preset="veryfast"):
    cap=cv2.VideoCapture(src_path)
    if not cap.isOpened(): return []
    vfps=cap.get(cv2.CAP_PROP_FPS) or 30
    frames=int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    cap.release()
    if frames<=0: return []
    dur=frames/vfps
    starts=[]; t=0.0
    while t+clip_s<=dur: starts.append(round(t,3)); t+=hop_s
    base=os.path.splitext(os.path.basename(src_path))[0]
    outs=[]
    for ss in starts:
        outp=os.path.join(out_dir, f"{base}_ss{int(ss*1000):07d}.mp4")
        if os.path.exists(outp):
            outs.append(outp); continue
        cmd=["ffmpeg","-y","-hide_banner","-loglevel","error",
             "-ss", f"{ss:.3f}", "-t", str(clip_s), "-i", src_path,
             "-vf", f"scale={size}:{size}:force_original_aspect_ratio=decrease,pad={size}:{size}:(ow-iw)/2:(oh-ih)/2,format=yuv420p,fps={fps}",
             "-an","-r", str(fps), "-preset", preset, outp]
        subprocess.run(cmd)
        if os.path.exists(outp): outs.append(outp)
    return outs

def _pr_path(p):
    return os.path.join(CACHE_DIR, os.path.basename(p)+".pr.json")

model = YOLO("yolov8n.pt")

def person_ratio_cached(path, conf=PERSON_CONF, max_frames=MAX_FRAMES, vid_stride=VID_STRIDE, imgsz=IMGSZ):
    c=_pr_path(path)
    if os.path.exists(c):
        try: return json.load(open(c))["pr"]
        except: pass
    det=tot=0
    gen = model.predict(source=path, classes=[0], conf=conf, stream=True, verbose=False,
                        vid_stride=vid_stride, imgsz=imgsz, device=DEVICE)
    for r in gen:
        tot += 1
        if r.boxes is not None and len(r.boxes)>0:
            det += 1
        if tot >= max_frames:
            break
    pr = (det/tot) if tot else 0.0
    try: json.dump({"pr":pr}, open(c,"w"))
    except: pass
    return pr

def rsync_to_drive():
    # copia incremental, no pisa lo existente
    os.system(f'rsync -a --ignore-existing "{OUT_DIR}/" "{DRIVE_MIRROR}/"')

# ---------- POS EXISTENTES ----------
pos_clips = sorted(glob.glob(os.path.join(POS_DIR, "*.mp4")))
print("POS existentes:", len(pos_clips))

# ---------- ELEGIR NORMALES ----------
train_norm = list_videos(os.path.join(SRC_DIR, "Training_Normal_Videos_Anomaly"))
test_norm  = list_videos(os.path.join(SRC_DIR, "Testing_Normal_Videos_Anomaly"))
normal_pool = train_norm + test_norm
random.shuffle(normal_pool)
normal_pool = normal_pool[:N_NEG_VIDEOS]
print("Normales a procesar (videos):", len(normal_pool))

# ---------- SEGMENTAR + SYNC POR LOTES ----------
neg_raw_before = len(glob.glob(os.path.join(NEG_DIR, "*.mp4")))
processed = 0
for v in tqdm(normal_pool, desc="Segmentando NEG"):
    _ = segment_to_clips_resume(v, NEG_DIR, 3, HOP_NEG, SIZE, FPS, FFMPEG_PRESET)
    processed += 1
    if processed % SYNC_EVERY == 0:
        rsync_to_drive()
neg_raw_after = len(glob.glob(os.path.join(NEG_DIR, "*.mp4")))
print("NEG brutos nuevos:", neg_raw_after - neg_raw_before, " | TOTAL NEG brutos:", neg_raw_after)

# ---------- FILTRAR PERSONA (usa cache / reanuda) ----------
neg_all = sorted(glob.glob(os.path.join(NEG_DIR, "*.mp4")))
neg_kept = []
for p in tqdm(neg_all, desc="Filtrando NEG (person)"):
    try:
        if person_ratio_cached(p) >= NEG_PERSON_THR:
            neg_kept.append(p)
        else:
            os.remove(p)
    except Exception:
        pass
print("NEG con persona:", len(neg_kept))

# ---------- BALANCE FINAL vs POS (1.5x) ----------
target_neg = int(len(pos_clips) * 1.5)
random.shuffle(neg_kept)
if len(neg_kept) > target_neg:
    for p in neg_kept[target_neg:]:
        try: os.remove(p)
        except: pass
    neg_kept = neg_kept[:target_neg]
print(f"Balance final -> POS: {len(pos_clips)}  NEG: {len(neg_kept)}")

# ---------- CSVs (split por video, sin fuga) ----------
def base_from_clip(p): return os.path.basename(p).split("_ss")[0]
def group_by_video(clips):
    d=collections.defaultdict(list)
    for p in clips: d[base_from_clip(p)].append(p)
    return list(d.values())
def split_groups(groups, r=(0.8,0.1,0.1)):
    n=len(groups); a=int(r[0]*n); b=int((r[0]+r[1])*n)
    return groups[:a], groups[a:b], groups[b:]
def flatten(gs):
    out=[]; [out.extend(g) for g in gs]; return out

pos_groups = group_by_video(pos_clips)
neg_groups = group_by_video(neg_kept)
random.shuffle(pos_groups); random.shuffle(neg_groups)
p_tr,p_va,p_te = split_groups(pos_groups)
n_tr,n_va,n_te = split_groups(neg_groups)

splits = {
  "train": [(p,"shoplifting") for p in flatten(p_tr)] + [(p,"normal") for p in flatten(n_tr)],
  "val":   [(p,"shoplifting") for p in flatten(p_va)] + [(p,"normal") for p in flatten(n_va)],
  "test":  [(p,"shoplifting") for p in flatten(p_te)] + [(p,"normal") for p in flatten(n_te)],
}
for name,items in splits.items():
    random.shuffle(items)
    with open(os.path.join(OUT_DIR,f"{name}.csv"),"w") as f:
        for p,c in items: f.write(f"{p},{c}\n")
    print(name, "->", len(items))

# sync final
rsync_to_drive()
print("CSV y clips listos en:", OUT_DIR)
print("Espejo en Drive:", DRIVE_MIRROR)


Device: CPU
POS existentes: 9834
Normales a procesar (videos): 150


Segmentando NEG: 100%|██████████| 150/150 [4:06:24<00:00, 98.57s/it]


NEG brutos nuevos: 2989  | TOTAL NEG brutos: 3681


Filtrando NEG (person): 100%|██████████| 3681/3681 [1:01:24<00:00,  1.00s/it]


NEG con persona: 2391
Balance final -> POS: 9834  NEG: 2391
train -> 10668
val -> 787
test -> 770
CSV y clips listos en: /content/tesisV2/cortos/video_clips_192
Espejo en Drive: /content/drive/MyDrive/tesisV2/cortos/video_clips_192


## ***PRUEBA: JSON CON LA PROBABILIDAD DE HURTO. MP4 CON BARRAS***

In [ ]:
# === INFERENCIA en video largo (usa checkpoint linear probe) ===
import os, json, cv2, math, torch, numpy as np
from torchvision.models.video import r3d_18, R3D_18_Weights

# RUTAS
MODEL_PATH = "/content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_linearprobe.pt"  # <- el que guardó el training
VIDEO_IN   = "/content/tesisV2/videos/supermas1-1.mp4"                                 # <-- cambia a tu video
OUT_JSON   = "/content/tesisV2/preds_supermas1-1.json"
OUT_VIS    = "/content/tesisV2/preds_supermas1-1_vis.mp4"

# Debe matchear lo que usaste en training rápido
NUM_FRAMES = 8
IMG_SIZE   = 96

# Ventana y hop (en segundos)
WIN_SEC = 3.0
HOP_SEC = 1.0
THRESH  = 0.55  # umbral para marcar "alarma"

device = "cuda" if torch.cuda.is_available() else "cpu"

# Modelo igual que en training (congelado, misma capa final)
weights = R3D_18_Weights.KINETICS400_V1
model = r3d_18(weights=weights)
model.fc = torch.nn.Linear(model.fc.in_features, 2)
ckpt = torch.load(MODEL_PATH, map_location=device)
model.load_state_dict(ckpt["model"])
model = model.to(device).eval()

mean = torch.tensor([0.43216, 0.394666, 0.37645], dtype=torch.float32).view(3,1,1,1).to(device)
std  = torch.tensor([0.22803, 0.22145, 0.216989], dtype=torch.float32).view(3,1,1,1).to(device)

def clip_to_tensor(frames_rgb):
    x = np.stack(frames_rgb, axis=0).astype(np.float32)/255.0  # T,H,W,C
    x = torch.from_numpy(x).permute(3,0,1,2).to(device)        # C,T,H,W
    x = (x - mean) / std
    return x.unsqueeze(0)  # B=1

cap = cv2.VideoCapture(VIDEO_IN)
fps = cap.get(cv2.CAP_PROP_FPS) or 30
n   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
w   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)); h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"Video: {n} frames, {fps:.1f} fps")

win = int(WIN_SEC*fps); hop = int(HOP_SEC*fps)
scores = []  # (t0,t1,p_shop)

for start in range(0, max(1, n - win + 1), hop):
    idxs = np.linspace(start, start+win-1, num=NUM_FRAMES, dtype=int)
    frames=[]
    for k in idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(k))
        ok, fr = cap.read()
        if not ok or fr is None:
            fr = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)
        else:
            fr = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
            fr = cv2.resize(fr, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)
        frames.append(fr)
    x = clip_to_tensor(frames)
    with torch.no_grad():
        logits = model(x)[0]
        p_shop = torch.softmax(logits, dim=0)[1].item()
    t0 = start/fps; t1 = (start+win)/fps
    scores.append((t0, t1, p_shop))

cap.release()

# Suavizado (media móvil simple para reducir ruido)
win_smooth = 3
probs = np.array([p for _,_,p in scores], dtype=np.float32)
if len(probs) >= win_smooth:
    kernel = np.ones(win_smooth)/win_smooth
    probs_s = np.convolve(probs, kernel, mode='same')
else:
    probs_s = probs

# Guardar JSON
with open(OUT_JSON, "w") as f:
    json.dump([{"t0":float(a),"t1":float(b),"p":float(p),"p_smooth":float(ps)}
               for (a,b,_), ps in zip(scores, probs_s)], f, indent=2)
print("Guardado:", OUT_JSON)

# Render con barra de probabilidad
cap = cv2.VideoCapture(VIDEO_IN)
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(OUT_VIS, fourcc, fps, (w,h))
bar_h = 18

# Expande probs_smooth por frame para pintar
score_by_frame = np.zeros(n, dtype=np.float32)
for i, (t0,t1,_) in enumerate(scores):
    a = int(t0*fps); b = min(n-1, int(t1*fps))
    score_by_frame[a:b+1] = max(0.0, min(1.0, probs_s[i]))

i=0
while True:
    ok, fr = cap.read()
    if not ok: break
    p = float(score_by_frame[i]) if i < len(score_by_frame) else 0.0
    bw = int(p * w)
    color = (0,0,255) if p >= THRESH else (0,255,0)
    cv2.rectangle(fr, (0,0), (bw, bar_h), color, -1)
    cv2.putText(fr, f"shoplifting prob: {p:.2f}", (10, bar_h+22),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2, cv2.LINE_AA)
    out.write(fr); i+=1

cap.release(); out.release()
print("Video con barra:", OUT_VIS)



KeyboardInterrupt



In [ ]:
# ===== Elegir umbral óptimo en VAL =====
import os, torch, numpy as np
from torchvision.models.video import r3d_18, R3D_18_Weights

OUT_DIR  = "/content/tesisV2/cortos/video_clips_192"
VAL_CSV  = os.path.join(OUT_DIR, "val.csv")
MODEL_P  = "/content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_linearprobe.pt"

CLASSES = {"normal":0, "shoplifting":1}
NUM_FRAMES, IMG_SIZE = 8, 96

def read_csv(p):
    items=[];
    for line in open(p):
        path,lab = line.strip().split(",")
        items.append((path, CLASSES[lab]))
    return items
val_items = read_csv(VAL_CSV)

# Modelo
device = "cuda" if torch.cuda.is_available() else "cpu"
weights = r3d_18(weights=r3d_18.R3D_18_Weights.KINETICS400_V1)
weights.fc = None  # dummy para que no binde? (ignorado)
model = r3d_18(weights=r3d_18.R3D_18_Weights.KINETICS400_V1)
model.fc = torch.nn.Linear(model.fc.in_features, 2)
ckpt = torch.load(MODEL_P, map_location=device)
model.load_state_dict(ckpt["model"]); model = model.to(device).eval()

mean = torch.tensor([0.43216, 0.394666, 0.37645], dtype=torch.float32).view(3,1,1,1).to(device)
std  = torch.tensor([0.22803, 0.22145, 0.216989], dtype=torch.float32).view(3,1,1,1).to(device)

import cv2
def clip_tensor(path):
    cap = cv2.VideoCapture(path); n = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    idxs = np.linspace(0, max(0,n-1), num=NUM_FRAMES, dtype=int)
    frames=[]
    for k in idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(k))
        ok, fr = cap.read()
        if not ok or fr is None: fr = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
        else:
            fr = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
            fr = cv2.resize(fr, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)
        frames.append(fr)
    cap.release()
    x = torch.from_numpy(np.stack(frames,0).astype(np.float32)/255.0).permute(3,0,1,2).to(device)
    return ((x-mean)/std).unsqueeze(0)

# Barrido de thresholds
from sklearn.metrics import precision_recall_fscore_support
probs=[]; ys=[]
with torch.no_grad():
    for p,y in val_items:
        x = clip_tensor(p)
        s = torch.softmax(model(x)[0], dim=0)[1].item()  # prob shoplifting
        probs.append(s); ys.append(y)
probs = np.array(probs); ys = np.array(ys)

best = (0.0, -1.0, 0,0,0)  # thr, metric, p,r,f1
for thr in np.linspace(0.3, 0.9, 25):
    preds = (probs >= thr).astype(int)
    p,r,f1,_ = precision_recall_fscore_support(ys, preds, average='binary')  # binario = clase 1 (shoplifting)
    # Ej: max F1 de shoplifting (cambia a 'p' si quieres max precision)
    score = f1
    if score > best[1]:
        best = (thr, score, p, r, f1)
print(f"Umbral óptimo(F1 shop): {best[0]:.2f}  | P:{best[2]:.3f} R:{best[3]:.3f} F1:{best[4]:.3f}")


AttributeError: 'function' object has no attribute 'R3D_18_Weights'

## ***ENTRENAMIENTO RAPIDO CON CPU***

In [ ]:
# === LINEAR PROBE en CPU (rápido) ===
import os, random, time, gc, hashlib, numpy as np, torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import cv2

# Rutas
OUT_DIR  = "/content/tesisV2/cortos/video_clips_192"        # contiene train.csv / val.csv / test.csv
SAVE_DIR = "/content/drive/MyDrive/tesisV2/models"
CACHE_DIR= "/content/tesisV2/frame_cache_npy"
os.makedirs(SAVE_DIR, exist_ok=True); os.makedirs(CACHE_DIR, exist_ok=True)

# Hiperparámetros (ligeros para CPU)
CLASSES = {"normal":0, "shoplifting":1}
NUM_FRAMES = 8
IMG_SIZE   = 96
BATCH      = 8             # si se queda corto de RAM, baja a 4
EPOCHS     = 5             # rápido; luego puedes subir
LR_HEAD    = 1e-3          # LR más alto para la capa final
NUM_WORKERS= 0             # CPU en Colab → 0

# CSVs
TRAIN_CSV = os.path.join(OUT_DIR, "train.csv")
VAL_CSV   = os.path.join(OUT_DIR, "val.csv")
TEST_CSV  = os.path.join(OUT_DIR, "test.csv")

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

def read_csv(path):
    items=[]
    with open(path) as f:
        for line in f:
            p,lab = line.strip().split(",")
            items.append((p, CLASSES[lab]))
    return items

train_items = read_csv(TRAIN_CSV)
val_items   = read_csv(VAL_CSV)
test_items  = read_csv(TEST_CSV)
print(f"train:{len(train_items)}  val:{len(val_items)}  test:{len(test_items)}")

# (Opcional) Submuestreo para ir aún más rápido en la primera corrida
SUBSAMPLE_TRAIN = None  # p.ej. 6000; déjalo en None para usar todo
if SUBSAMPLE_TRAIN:
    random.shuffle(train_items); train_items = train_items[:SUBSAMPLE_TRAIN]

# -------- utilidades dataset con cache de frames --------
def sample_frame_indices(n_total, n_sample):
    if n_total <= n_sample:
        return np.linspace(0, max(0, n_total-1), num=n_sample, dtype=int)
    return np.linspace(0, n_total-1, num=n_sample, dtype=int)

def cache_path(video_path, nframes, size):
    h = hashlib.md5(f"{video_path}|{nframes}|{size}".encode()).hexdigest()
    base = os.path.basename(video_path)
    return os.path.join(CACHE_DIR, f"{base}.{h}.npy")

def load_clip_as_array(path, nframes=NUM_FRAMES, size=IMG_SIZE):
    cpath = cache_path(path, nframes, size)
    if os.path.exists(cpath):
        try:
            arr = np.load(cpath)
            if arr.shape == (nframes, size, size, 3):
                return arr
        except Exception:
            pass
    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        arr = np.zeros((nframes, size, size, 3), dtype=np.uint8)
        np.save(cpath, arr); return arr
    n = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    idxs = sample_frame_indices(n, nframes)
    frames=[]
    for k in idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(k))
        ok, fr = cap.read()
        if not ok or fr is None:
            fr = np.zeros((size, size, 3), dtype=np.uint8)
        else:
            fr = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
            fr = cv2.resize(fr, (size, size), interpolation=cv2.INTER_AREA)
        frames.append(fr)
    cap.release()
    arr = np.stack(frames, axis=0).astype(np.uint8)  # (T,H,W,C)
    try: np.save(cpath, arr)
    except Exception: pass
    return arr

class ClipDataset(Dataset):
    def __init__(self, items, train=True):
        self.items = items
        self.train = train
        self.mean = torch.tensor([0.43216, 0.394666, 0.37645], dtype=torch.float32).view(3,1,1,1)
        self.std  = torch.tensor([0.22803, 0.22145, 0.216989], dtype=torch.float32).view(3,1,1,1)
    def __len__(self): return len(self.items)
    def __getitem__(self, i):
        path, y = self.items[i]
        arr = load_clip_as_array(path, NUM_FRAMES, IMG_SIZE)       # (T,H,W,C) uint8
        x = torch.from_numpy(arr.astype(np.float32)/255.0).permute(3,0,1,2)  # C,T,H,W
        if self.train and random.random() < 0.1:
            x = torch.flip(x, dims=[3])  # flip horizontal
        x = (x - self.mean) / self.std
        return x, torch.tensor(y, dtype=torch.long)

train_loader = DataLoader(ClipDataset(train_items, True),  batch_size=BATCH, shuffle=True,  num_workers=NUM_WORKERS)
val_loader   = DataLoader(ClipDataset(val_items,   False), batch_size=BATCH, shuffle=False, num_workers=NUM_WORKERS)
test_loader  = DataLoader(ClipDataset(test_items,  False), batch_size=BATCH, shuffle=False, num_workers=NUM_WORKERS)

# -------- modelo: R3D-18 congelado (solo entrenamos la capa final) --------
from torchvision.models.video import r3d_18, R3D_18_Weights
weights = R3D_18_Weights.KINETICS400_V1
model = r3d_18(weights=weights)

# congelar todo
for p in model.parameters():
    p.requires_grad = False

# reemplazar y entrenar solo la capa final
model.fc = nn.Linear(model.fc.in_features, 2)
for p in model.fc.parameters():
    p.requires_grad = True

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.fc.parameters(), lr=LR_HEAD, weight_decay=0.0)

@torch.no_grad()
def eval_epoch(loader):
    model.train(False)
    total=correct=0; loss_sum=0.0
    for x,y in loader:
        x, y = x.to(device), y.to(device)
        logits = model(x); loss = criterion(logits, y)
        loss_sum += float(loss.item())*x.size(0)
        pred = logits.argmax(1); correct += int((pred==y).sum())
        total += x.size(0)
    return loss_sum/total, correct/total

def train_epoch(loader):
    model.train(True)
    total=correct=0; loss_sum=0.0
    for x,y in loader:
        x, y = x.to(device), y.to(device)
        logits = model(x); loss = criterion(logits, y)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        loss_sum += float(loss.item())*x.size(0)
        pred = logits.argmax(1); correct += int((pred==y).sum())
        total += x.size(0)
    return loss_sum/total, correct/total

best_val = 0.0
best_path = os.path.join(SAVE_DIR, "r3d18_shoplifting_linearprobe.pt")
for e in range(1, EPOCHS+1):
    t0=time.time()
    tr_loss, tr_acc = train_epoch(train_loader)
    val_loss, val_acc = eval_epoch(val_loader)
    dt = time.time()-t0
    print(f"Epoch {e:02d}/{EPOCHS} | train {tr_loss:.4f}/{tr_acc:.3f} | val {val_loss:.4f}/{val_acc:.3f} | {dt/60:.1f} min")
    if val_acc > best_val:
        best_val = val_acc
        torch.save({"model":model.state_dict(), "classes":CLASSES}, best_path)
        print("✔️ guardado mejor:", best_path)
    gc.collect()

# TEST final
ckpt = torch.load(best_path, map_location=device)
model.load_state_dict(ckpt["model"]); model.eval()

from sklearn.metrics import classification_report, confusion_matrix
all_y, all_p = [], []
with torch.no_grad():
    for x,y in test_loader:
        x = x.to(device); y = y.to(device)
        p = model(x).argmax(1)
        all_y += y.cpu().tolist(); all_p += p.cpu().tolist()

print(confusion_matrix(all_y, all_p))
print(classification_report(all_y, all_p, target_names=["normal","shoplifting"]))
print("Best ckpt:", best_path)


Device: cpu
train:10668  val:787  test:770
Epoch 01/5 | train 0.2054/0.924 | val 0.5633/0.804 | 105.4 min
✔️ guardado mejor: /content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_linearprobe.pt
Epoch 02/5 | train 0.1344/0.952 | val 0.6471/0.809 | 85.1 min
✔️ guardado mejor: /content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_linearprobe.pt
Epoch 03/5 | train 0.1187/0.957 | val 0.7793/0.816 | 84.8 min
✔️ guardado mejor: /content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_linearprobe.pt
Epoch 04/5 | train 0.1041/0.964 | val 1.1228/0.748 | 85.2 min
Epoch 05/5 | train 0.1175/0.959 | val 1.0136/0.773 | 85.7 min
[[ 85 106]
 [181 398]]
              precision    recall  f1-score   support

      normal       0.32      0.45      0.37       191
 shoplifting       0.79      0.69      0.73       579

    accuracy                           0.63       770
   macro avg       0.55      0.57      0.55       770
weighted avg       0.67      0.63      0.64       770

Best ckpt: /content/drive/

In [ ]:
# ===== Fine-tuning R3D-18 por fases (con pesos de clase) =====
import os, random, time, gc, hashlib, numpy as np, torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import cv2

# ----------------- RUTAS / CONFIG -----------------
OUT_DIR   = "/content/tesisV2/cortos/video_clips_192"   # tiene train.csv / val.csv / test.csv
SAVE_DIR  = "/content/drive/MyDrive/tesisV2/models"
CACHE_DIR = "/content/tesisV2/frame_cache_npy"
os.makedirs(SAVE_DIR, exist_ok=True); os.makedirs(CACHE_DIR, exist_ok=True)

TRAIN_CSV = os.path.join(OUT_DIR, "train.csv")
VAL_CSV   = os.path.join(OUT_DIR, "val.csv")
TEST_CSV  = os.path.join(OUT_DIR, "test.csv")

# Tamaño de clip
NUM_FRAMES = 16      # (mejor que 8)
IMG_SIZE   = 112

# Batch/épocas por fase (ajustá si CPU/GPU)
BATCH_LINEAR = 8
BATCH_FT     = 6
EPOCHS_LINEAR = 3    # fase 1: solo fc
EPOCHS_L4     = 5    # fase 2: descongelar layer4
EPOCHS_L34    = 0    # fase 3 opcional: layer3+layer4 (poné 3-5 si querés)

# Pesos de clase para bajar FP (más peso a "normal")
W_NORMAL = 1.6
W_SHOP   = 1.0

LR_FC   = 1e-3       # capa final
LR_L4   = 3e-4       # fine-tune layer4
LR_L34  = 2e-4       # fine-tune layer3+4

NUM_WORKERS = 0

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# ----------------- UTILIDADES DE DATOS (cache de frames) -----------------
CLASSES = {"normal":0, "shoplifting":1}

def read_csv(path):
    items=[]
    with open(path) as f:
        for line in f:
            p,lab = line.strip().split(",")
            items.append((p, CLASSES[lab]))
    return items

train_items = read_csv(TRAIN_CSV)
val_items   = read_csv(VAL_CSV)
test_items  = read_csv(TEST_CSV)
print(f"train:{len(train_items)}  val:{len(val_items)}  test:{len(test_items)}")

def sample_frame_indices(n_total, n_sample, jitter=True):
    # muestreo uniforme con pequeño jitter temporal
    idx = np.linspace(0, max(0, n_total-1), num=n_sample)
    if jitter and n_total > n_sample:
        noise = np.random.uniform(-0.5, 0.5, size=n_sample)
        idx = np.clip(idx + noise, 0, max(0, n_total-1))
    return idx.astype(int)

def cache_path(video_path, nframes, size):
    h = hashlib.md5(f"{video_path}|{nframes}|{size}".encode()).hexdigest()
    base = os.path.basename(video_path)
    return os.path.join(CACHE_DIR, f"{base}.{h}.npy")

def load_clip_as_array(path, nframes=NUM_FRAMES, size=IMG_SIZE, jitter=True):
    cpath = cache_path(path, nframes, size)
    if os.path.exists(cpath):
        try:
            arr = np.load(cpath)
            if arr.shape == (nframes, size, size, 3):
                return arr
        except Exception:
            pass
    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        arr = np.zeros((nframes, size, size, 3), dtype=np.uint8)
        np.save(cpath, arr); return arr
    n = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    idxs = sample_frame_indices(n, nframes, jitter=jitter)
    frames=[]
    for k in idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(k))
        ok, fr = cap.read()
        if not ok or fr is None:
            fr = np.zeros((size, size, 3), dtype=np.uint8)
        else:
            fr = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
            fr = cv2.resize(fr, (size, size), interpolation=cv2.INTER_AREA)
        frames.append(fr)
    cap.release()
    arr = np.stack(frames, axis=0).astype(np.uint8)
    try: np.save(cpath, arr)
    except Exception: pass
    return arr

class ClipDataset(Dataset):
    def __init__(self, items, train=True):
        self.items = items
        self.train = train
        self.mean = torch.tensor([0.43216, 0.394666, 0.37645], dtype=torch.float32).view(3,1,1,1)
        self.std  = torch.tensor([0.22803, 0.22145, 0.216989], dtype=torch.float32).view(3,1,1,1)
    def __len__(self): return len(self.items)
    def __getitem__(self, i):
        path, y = self.items[i]
        arr = load_clip_as_array(path, NUM_FRAMES, IMG_SIZE, jitter=self.train)  # (T,H,W,C)
        x = torch.from_numpy(arr.astype(np.float32)/255.0).permute(3,0,1,2)     # C,T,H,W
        # Augment simple
        if self.train and random.random() < 0.3:
            x = torch.flip(x, dims=[3])  # flip horizontal
        x = (x - self.mean) / self.std
        return x, torch.tensor(y, dtype=torch.long)

def make_loader(items, batch, train):
    return DataLoader(ClipDataset(items, train),
                      batch_size=batch, shuffle=train,
                      num_workers=NUM_WORKERS, pin_memory=False)

# ----------------- MODELO -----------------
from torchvision.models.video import r3d_18, R3D_18_Weights
weights = R3D_18_Weights.KINETICS400_V1
model = r3d_18(weights=weights)

# Reemplazamos la cabeza
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

# Pérdida con pesos de clase (más peso a 'normal' para bajar FP)
class_weights = torch.tensor([W_NORMAL, W_SHOP], dtype=torch.float32, device=device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Helpers de entrenamiento
def set_trainable(module, requires_grad: bool):
    for p in module.parameters():
        p.requires_grad = requires_grad

@torch.no_grad()
def eval_epoch(loader):
    model.eval()
    total=correct=0; loss_sum=0.0
    from sklearn.metrics import precision_recall_fscore_support
    all_y=[]; all_p=[]
    for x,y in loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        loss = criterion(logits, y)
        loss_sum += float(loss.item())*x.size(0)
        pred = logits.argmax(1)
        correct += int((pred==y).sum()); total += x.size(0)
        all_y += y.cpu().tolist(); all_p += pred.cpu().tolist()
    P,R,F1,_ = precision_recall_fscore_support(all_y, all_p, average=None, labels=[0,1])  # [normal, shop]
    return loss_sum/total, correct/total, {"P_norm":P[0], "R_norm":R[0], "F1_norm":F1[0],
                                           "P_shop":P[1], "R_shop":R[1], "F1_shop":F1[1]}

def train_epoch(loader, optimizer):
    model.train(True)
    total=correct=0; loss_sum=0.0
    for x,y in loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        loss = criterion(logits, y)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        loss_sum += float(loss.item())*x.size(0)
        pred = logits.argmax(1)
        correct += int((pred==y).sum()); total += x.size(0)
    return loss_sum/total, correct/total

def phase(name, unfreeze_parts, epochs, batch_size, lr):
    if epochs <= 0:
        print(f"[{name}] saltado (epochs=0)");
        return None
    # congelar todo
    set_trainable(model, False)
    # descongelar lo que toque
    for part in unfreeze_parts:
        set_trainable(part, True)
    # optim solo sobre params entrenables
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = optim.AdamW(params, lr=lr, weight_decay=0.01)

    train_loader = make_loader(train_items, batch_size, train=True)
    val_loader   = make_loader(val_items,   batch_size, train=False)

    best_key = None
    best_tuple = None
    best_path = os.path.join(SAVE_DIR, "r3d18_shoplifting_finetune.pt")

    print(f"\n===== {name} | epochs={epochs} batch={batch_size} lr={lr} =====")
    for e in range(1, epochs+1):
        t0=time.time()
        tr_loss, tr_acc = train_epoch(train_loader, optimizer)
        val_loss, val_acc, m = eval_epoch(val_loader)
        dt = time.time()-t0
        # criterio de guardado: F1_shop, y desempate por P_norm (menos FP)
        key = (m["F1_shop"], m["P_norm"])
        if (best_key is None) or (key > best_key):
            best_key = key
            best_tuple = (val_loss, val_acc, m)
            torch.save({"model":model.state_dict(), "classes":CLASSES}, best_path)
            print(f"✔️ guardado mejor: {best_path} | F1_shop={m['F1_shop']:.3f} P_norm={m['P_norm']:.3f}")
        print(f"Ep {e:02d}/{epochs} | train {tr_loss:.4f}/{tr_acc:.3f} | "
              f"val {val_loss:.4f}/{val_acc:.3f} | "
              f"P_norm {m['P_norm']:.3f} R_norm {m['R_norm']:.3f} F1_norm {m['F1_norm']:.3f} | "
              f"P_shop {m['P_shop']:.3f} R_shop {m['R_shop']:.3f} F1_shop {m['F1_shop']:.3f} | "
              f"{dt/60:.1f} min")
    return best_path, best_tuple

# ----------------- FASES -----------------
# Fase 1: solo cabeza
best1 = phase("Fase 1: Linear probe (solo fc)", [model.fc], EPOCHS_LINEAR, BATCH_LINEAR, LR_FC)

# Fase 2: descongelar layer4 (+ fc)
parts = [model.layer4, model.fc]
best2 = phase("Fase 2: Fine-tune layer4", parts, EPOCHS_L4, BATCH_FT, LR_L4)

# Fase 3 (opcional): layer3 + layer4 (+ fc)
if EPOCHS_L34 > 0:
    parts = [model.layer3, model.layer4, model.fc]
    best3 = phase("Fase 3: Fine-tune layer3+4 (opcional)", parts, EPOCHS_L34, BATCH_FT, LR_L34)

# ----------------- TEST FINAL -----------------
from sklearn.metrics import classification_report, confusion_matrix
ckpt_path = os.path.join(SAVE_DIR, "r3d18_shoplifting_finetune.pt")
ckpt = torch.load(ckpt_path, map_location=device)
model.load_state_dict(ckpt["model"]); model.eval()

test_loader = make_loader(test_items, batch_size=8, train=False)
all_y, all_p = [], []
with torch.no_grad():
    for x,y in test_loader:
        x = x.to(device); y = y.to(device)
        p = model(x).argmax(1)
        all_y += y.cpu().tolist(); all_p += p.cpu().tolist()

print("\nMatriz de confusión (TEST):")
print(confusion_matrix(all_y, all_p))
print("\nReporte (TEST):")
print(classification_report(all_y, all_p, target_names=["normal","shoplifting"]))
print("Best ckpt:", ckpt_path)
gc.collect()


Device: cpu
train:10668  val:787  test:770


Downloading: "https://download.pytorch.org/models/r3d_18-b3b3357e.pth" to /root/.cache/torch/hub/checkpoints/r3d_18-b3b3357e.pth
100%|██████████| 127M/127M [00:03<00:00, 41.4MB/s]



===== Fase 1: Linear probe (solo fc) | epochs=3 batch=8 lr=0.001 =====
✔️ guardado mejor: /content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt | F1_shop=0.830 P_norm=0.325
Ep 01/3 | train 0.2160/0.931 | val 0.5774/0.740 | P_norm 0.325 R_norm 0.686 F1_norm 0.441 | P_shop 0.931 R_shop 0.749 F1_shop 0.830 | 380.3 min
Ep 02/3 | train 0.1388/0.956 | val 1.0044/0.642 | P_norm 0.273 R_norm 0.839 F1_norm 0.412 | P_shop 0.955 R_shop 0.607 F1_shop 0.742 | 293.3 min


KeyboardInterrupt: 

In [ ]:
# ===== Fine-tuning R3D-18 por fases (con pesos de clase) [RESUME + SESIÓN CORTA + BEST/LATEST] =====
import os, random, time, gc, hashlib, numpy as np, torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import cv2

# ----------------- RUTAS / CONFIG -----------------
OUT_DIR   = "/content/tesisV2/cortos/video_clips_192"   # tiene train.csv / val.csv / test.csv
SAVE_DIR  = "/content/drive/MyDrive/tesisV2/models"
CACHE_DIR = "/content/tesisV2/frame_cache_npy"
os.makedirs(SAVE_DIR, exist_ok=True); os.makedirs(CACHE_DIR, exist_ok=True)

TRAIN_CSV = os.path.join(OUT_DIR, "train.csv")
VAL_CSV   = os.path.join(OUT_DIR, "val.csv")
TEST_CSV  = os.path.join(OUT_DIR, "test.csv")

# >>> Checkpoints
CKPT_BEST   = os.path.join(SAVE_DIR, "r3d18_shoplifting_finetune.best.pt")
CKPT_LATEST = os.path.join(SAVE_DIR, "r3d18_shoplifting_finetune.latest.pt")

# Tamaño de clip (ajusta si vas en CPU)
NUM_FRAMES = 16      # en CPU: puedes bajar a 8
IMG_SIZE   = 112     # en CPU: puedes bajar a 96

# Batch/épocas por fase (ajusta si CPU/GPU)
BATCH_LINEAR  = 8
BATCH_FT      = 6
EPOCHS_LINEAR = 0    # fase 1: solo fc  (0 = saltar)
EPOCHS_L4     = 5    # fase 2: descongelar layer4
EPOCHS_L34    = 0    # fase 3 opcional: layer3+layer4

# Pesos de clase para bajar FP (más peso a "normal")
W_NORMAL = 1.6
W_SHOP   = 1.0

LR_FC   = 1e-3       # capa final
LR_L4   = 3e-4       # fine-tune layer4
LR_L34  = 2e-4       # fine-tune layer3+4

NUM_WORKERS = 0

# --- SESIÓN CORTA (para que no se caiga en Colab) ---
SHORT_SESSION       = True
MAX_TRAIN_BATCHES   = 300    # procesa solo N batches por época (ajusta)
SAVE_EVERY_BATCHES  = 150    # guarda intermedio cada N batches

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# ----------------- UTILIDADES DE DATOS (cache de frames) -----------------
CLASSES = {"normal":0, "shoplifting":1}

def read_csv(path):
    items=[]
    with open(path) as f:
        for line in f:
            p,lab = line.strip().split(",")
            items.append((p, CLASSES[lab]))
    return items

train_items = read_csv(TRAIN_CSV)
val_items   = read_csv(VAL_CSV)
test_items  = read_csv(TEST_CSV)
print(f"train:{len(train_items)}  val:{len(val_items)}  test:{len(test_items)}")

def sample_frame_indices(n_total, n_sample, jitter=True):
    idx = np.linspace(0, max(0, n_total-1), num=n_sample)
    if jitter and n_total > n_sample:
        noise = np.random.uniform(-0.5, 0.5, size=n_sample)
        idx = np.clip(idx + noise, 0, max(0, n_total-1))
    return idx.astype(int)

def cache_path(video_path, nframes, size):
    h = hashlib.md5(f"{video_path}|{nframes}|{size}".encode()).hexdigest()
    base = os.path.basename(video_path)
    return os.path.join(CACHE_DIR, f"{base}.{h}.npy")

def load_clip_as_array(path, nframes=NUM_FRAMES, size=IMG_SIZE, jitter=True):
    cpath = cache_path(path, nframes, size)
    if os.path.exists(cpath):
        try:
            arr = np.load(cpath)
            if arr.shape == (nframes, size, size, 3):
                return arr
        except Exception:
            pass
    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        arr = np.zeros((nframes, size, size, 3), dtype=np.uint8)
        np.save(cpath, arr); return arr
    n = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    idxs = sample_frame_indices(n, nframes, jitter=jitter)
    frames=[]
    for k in idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(k))
        ok, fr = cap.read()
        if not ok or fr is None:
            fr = np.zeros((size, size, 3), dtype=np.uint8)
        else:
            fr = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
            fr = cv2.resize(fr, (size, size), interpolation=cv2.INTER_AREA)
        frames.append(fr)
    cap.release()
    arr = np.stack(frames, axis=0).astype(np.uint8)
    try: np.save(cpath, arr)
    except Exception: pass
    return arr

class ClipDataset(Dataset):
    def __init__(self, items, train=True):
        self.items = items
        self.train = train
        self.mean = torch.tensor([0.43216, 0.394666, 0.37645], dtype=torch.float32).view(3,1,1,1)
        self.std  = torch.tensor([0.22803, 0.22145, 0.216989], dtype=torch.float32).view(3,1,1,1)
    def __len__(self): return len(self.items)
    def __getitem__(self, i):
        path, y = self.items[i]
        arr = load_clip_as_array(path, NUM_FRAMES, IMG_SIZE, jitter=self.train)  # (T,H,W,C)
        x = torch.from_numpy(arr.astype(np.float32)/255.0).permute(3,0,1,2)     # C,T,H,W
        if self.train and random.random() < 0.3:
            x = torch.flip(x, dims=[3])  # flip horizontal
        x = (x - self.mean) / self.std
        return x, torch.tensor(y, dtype=torch.long)

def make_loader(items, batch, train):
    return DataLoader(ClipDataset(items, train),
                      batch_size=batch, shuffle=train,
                      num_workers=NUM_WORKERS, pin_memory=False)

# ----------------- MODELO -----------------
from torchvision.models.video import r3d_18, R3D_18_Weights
weights = R3D_18_Weights.KINETICS400_V1
model = r3d_18(weights=weights)
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

# >>> RESUME GLOBAL: preferir LATEST, si no existe usar BEST
resume_from = CKPT_LATEST if os.path.exists(CKPT_LATEST) else (CKPT_BEST if os.path.exists(CKPT_BEST) else None)
if resume_from:
    try:
        ckpt = torch.load(resume_from, map_location=device)
        model.load_state_dict(ckpt["model"])
        print(f"✅ Reanudando desde: {resume_from}")
    except Exception as e:
        print(f"⚠️ No pude cargar {resume_from}: {e}. Sigo desde pesos base.")

# Pérdida con pesos de clase (más peso a 'normal')
class_weights = torch.tensor([W_NORMAL, W_SHOP], dtype=torch.float32, device=device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Helpers de entrenamiento
def set_trainable(module, requires_grad: bool):
    for p in module.parameters():
        p.requires_grad = requires_grad

@torch.no_grad()
def eval_epoch(loader):
    model.eval()
    total=correct=0; loss_sum=0.0
    from sklearn.metrics import precision_recall_fscore_support
    all_y=[]; all_p=[]
    for x,y in loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        loss = criterion(logits, y)
        loss_sum += float(loss.item())*x.size(0)
        pred = logits.argmax(1)
        correct += int((pred==y).sum()); total += x.size(0)
        all_y += y.cpu().tolist(); all_p += pred.cpu().tolist()
    P,R,F1,_ = precision_recall_fscore_support(all_y, all_p, average=None, labels=[0,1])  # [normal, shop]
    return loss_sum/total, correct/total, {"P_norm":P[0], "R_norm":R[0], "F1_norm":F1[0],
                                           "P_shop":P[1], "R_shop":R[1], "F1_shop":F1[1]}

def _save_latest(meta):
    try:
        torch.save({"model": model.state_dict(), "classes": CLASSES, "meta": meta}, CKPT_LATEST)
        # print(f"💾 Guardado latest -> {CKPT_LATEST}")
    except Exception as e:
        print("⚠️ No pude guardar LATEST:", e)

def _save_best(meta):
    try:
        torch.save({"model": model.state_dict(), "classes": CLASSES, "meta": meta}, CKPT_BEST)
        print(f"✔️ guardado mejor: {CKPT_BEST} | F1_shop={meta['F1_shop']:.3f} P_norm={meta['P_norm']:.3f}")
    except Exception as e:
        print("⚠️ No pude guardar BEST:", e)

def train_epoch(loader, optimizer, save_every=None, max_batches=None, phase_name=""):
    model.train(True)
    total=correct=0; loss_sum=0.0
    for i, (x,y) in enumerate(loader, start=1):
        x, y = x.to(device), y.to(device)
        logits = model(x)
        loss = criterion(logits, y)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

        loss_sum += float(loss.item())*x.size(0)
        pred = logits.argmax(1)
        correct += int((pred==y).sum()); total += x.size(0)

        if save_every and (i % save_every == 0):
            meta = {"phase": phase_name, "epoch": None, "batch": i}
            _save_latest(meta)

        if max_batches and i >= max_batches:
            break
    return loss_sum/total, correct/total

def phase(name, unfreeze_parts, epochs, batch_size, lr):
    if epochs <= 0:
        print(f"[{name}] saltado (epochs=0)"); return None

    # Congelar todo
    set_trainable(model, False)
    # Descongelar lo que toque
    for part in unfreeze_parts:
        set_trainable(part, True)

    # Re-cargar desde LAST/BEST al entrar en la fase (por si hubo corte)
    resume_from = CKPT_LATEST if os.path.exists(CKPT_LATEST) else (CKPT_BEST if os.path.exists(CKPT_BEST) else None)
    if resume_from:
        try:
            ckpt = torch.load(resume_from, map_location=device)
            model.load_state_dict(ckpt["model"])
            print(f"↪️  [{name}] Arranco desde checkpoint: {resume_from}")
        except Exception as e:
            print(f"⚠️ [{name}] No pude recargar {resume_from}: {e}")

    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = optim.AdamW(params, lr=lr, weight_decay=0.01)

    train_loader = make_loader(train_items, batch_size, train=True)
    val_loader   = make_loader(val_items,   batch_size, train=False)

    best_key = None
    best_tuple = None

    print(f"\n===== {name} | epochs={epochs} batch={batch_size} lr={lr} =====")
    for e in range(1, epochs+1):
        t0=time.time()
        tr_loss, tr_acc = train_epoch(
            train_loader, optimizer,
            save_every=(SAVE_EVERY_BATCHES if SHORT_SESSION else None),
            max_batches=(MAX_TRAIN_BATCHES if SHORT_SESSION else None),
            phase_name=name
        )
        val_loss, val_acc, m = eval_epoch(val_loader)
        dt = time.time()-t0

        # Guardado LATEST por-época con meta completa
        meta = {
            "phase": name, "epoch": e,
            "val_loss": val_loss, "val_acc": val_acc,
            "P_norm": m["P_norm"], "R_norm": m["R_norm"], "F1_norm": m["F1_norm"],
            "P_shop": m["P_shop"], "R_shop": m["R_shop"], "F1_shop": m["F1_shop"],
            "lr": lr, "batch_size": batch_size, "dt_min": dt/60.0
        }
        _save_latest(meta)

        # Criterio de BEST: F1_shop, desempate por P_norm (menos FP)
        key = (m["F1_shop"], m["P_norm"])
        if (best_key is None) or (key > best_key):
            best_key = key
            best_tuple = (val_loss, val_acc, m)
            _save_best(meta)

        print(f"Ep {e:02d}/{epochs} | train {tr_loss:.4f}/{tr_acc:.3f} | "
              f"val {val_loss:.4f}/{val_acc:.3f} | "
              f"P_norm {m['P_norm']:.3f} R_norm {m['R_norm']:.3f} F1_norm {m['F1_norm']:.3f} | "
              f"P_shop {m['P_shop']:.3f} R_shop {m['R_shop']:.3f} F1_shop {m['F1_shop']:.3f} | "
              f"{dt/60:.1f} min")
    return CKPT_BEST, best_tuple

# ----------------- FASES -----------------
best1 = phase("Fase 1: Linear probe (solo fc)", [model.fc], EPOCHS_LINEAR, BATCH_LINEAR, LR_FC)
parts = [model.layer4, model.fc]
best2 = phase("Fase 2: Fine-tune layer4", parts, EPOCHS_L4, BATCH_FT, LR_L4)
if EPOCHS_L34 > 0:
    parts = [model.layer3, model.layer4, model.fc]
    best3 = phase("Fase 3: Fine-tune layer3+4 (opcional)", parts, EPOCHS_L34, BATCH_FT, LR_L34)

# ----------------- TEST FINAL -----------------
from sklearn.metrics import classification_report, confusion_matrix
ckpt_path = CKPT_BEST if os.path.exists(CKPT_BEST) else CKPT_LATEST
ckpt = torch.load(ckpt_path, map_location=device)
model.load_state_dict(ckpt["model"]); model.eval()

test_loader = make_loader(test_items, batch_size=8, train=False)
all_y, all_p = [], []
with torch.no_grad():
    for x,y in test_loader:
        x = x.to(device); y = y.to(device)
        p = model(x).argmax(1)
        all_y += y.cpu().tolist(); all_p += p.cpu().tolist()

print("\nMatriz de confusión (TEST):")
print(confusion_matrix(all_y, all_p))
print("\nReporte (TEST):")
print(classification_report(all_y, all_p, target_names=["normal","shoplifting"]))
print("Best ckpt (si existe):", CKPT_BEST)
print("Latest ckpt:", CKPT_LATEST)
gc.collect()


Device: cpu
train:10668  val:787  test:770
✅ Reanudando desde: /content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt
[Fase 1: Linear probe (solo fc)] saltado (epochs=0)
↪️  [Fase 2: Fine-tune layer4] Arranco desde el mejor checkpoint previo.

===== Fase 2: Fine-tune layer4 | epochs=5 batch=6 lr=0.0003 =====


KeyboardInterrupt: 

In [ ]:
# === EVAL ÚNICA DE UN CHECKPOINT (.pt) SIN ENTRENAR ===
import os, cv2, time, numpy as np, torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.models.video import r3d_18
from sklearn.metrics import classification_report, confusion_matrix

# Rutas
OUT_DIR  = "/content/tesisV2/cortos/video_clips_192"
TEST_CSV = os.path.join(OUT_DIR, "test.csv")

# >>> Cambiá acá el ckpt que quieras evaluar:
CKPT_PATH = "/content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt"  # o ...linearprobe.pt

# Hiperparámetros de PREPROCESADO (ajustá a cómo entrenaste ese ckpt)
NUM_FRAMES = 16   # si el ckpt es el finetune
IMG_SIZE   = 112  # si el ckpt es el finetune
# Si evaluás el linearprobe original, usá:
# NUM_FRAMES, IMG_SIZE = 8, 96

BATCH = 8
CLASSES = {"normal":0, "shoplifting":1}

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_num_threads(2)
try:
    cv2.setNumThreads(0)
except:
    pass

def read_csv(p):
    items=[]
    with open(p) as f:
        for line in f:
            path,lab = line.strip().split(",")
            items.append((path, CLASSES[lab]))
    return items

test_items = read_csv(TEST_CSV)
print("test clips:", len(test_items))

# Normalización usada en entrenamiento (Kinetics)
mean = torch.tensor([0.43216,0.394666,0.37645], dtype=torch.float32).view(3,1,1,1)
std  = torch.tensor([0.22803,0.22145,0.216989], dtype=torch.float32).view(3,1,1,1)

def sample_frame_indices(n_total, n_sample):
    if n_total <= 0: return np.zeros(n_sample, dtype=int)
    return np.linspace(0, max(0, n_total-1), num=n_sample, dtype=int)

def clip_tensor(path, nframes=NUM_FRAMES, size=IMG_SIZE):
    cap = cv2.VideoCapture(path)
    n = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    idxs = sample_frame_indices(n, nframes)
    frames=[]
    for k in idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(k))
        ok, fr = cap.read()
        if not ok or fr is None:
            fr = np.zeros((size, size, 3), np.uint8)
        else:
            fr = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
            fr = cv2.resize(fr, (size, size), interpolation=cv2.INTER_AREA)
        frames.append(fr)
    cap.release()
    x = torch.from_numpy(np.stack(frames,0).astype(np.float32)/255.0).permute(3,0,1,2)  # C,T,H,W
    x = (x - mean) / std
    return x  # CPU

class TestDS(Dataset):
    def __init__(self, items): self.items = items
    def __len__(self): return len(self.items)
    def __getitem__(self, i):
        p,y = self.items[i]
        return clip_tensor(p), torch.tensor(y, dtype=torch.long)

loader = DataLoader(TestDS(test_items), batch_size=BATCH, shuffle=False, num_workers=0)

# Modelo + checkpoint
model = r3d_18(weights=None)
model.fc = nn.Linear(model.fc.in_features, 2)
ckpt = torch.load(CKPT_PATH, map_location=device)
model.load_state_dict(ckpt["model"])
model = model.to(device).eval()

# Eval
all_y, all_p = [], []
t0 = time.time()
with torch.no_grad():
    for X, y in loader:
        X = X.to(device)
        logits = model(X)
        pred = logits.argmax(1).cpu().numpy().tolist()
        all_p += pred
        all_y += y.numpy().tolist()
dt = time.time()-t0

print("\nMatriz de confusión (TEST):")
print(confusion_matrix(all_y, all_p))
print("\nReporte (TEST):")
print(classification_report(all_y, all_p, target_names=["normal","shoplifting"]))
print(f"Checkpoint evaluado: {CKPT_PATH}")
print(f"Tiempo eval: {dt/60:.1f} min en {device}")


test clips: 770

Matriz de confusión (TEST):
[[ 88 103]
 [205 374]]

Reporte (TEST):
              precision    recall  f1-score   support

      normal       0.30      0.46      0.36       191
 shoplifting       0.78      0.65      0.71       579

    accuracy                           0.60       770
   macro avg       0.54      0.55      0.54       770
weighted avg       0.66      0.60      0.62       770

Checkpoint evaluado: /content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt
Tiempo eval: 26.9 min en cpu


In [ ]:
# ===== Fine-tuning R3D-18 por fases + Sampler balanceado + Umbral por FPR =====
import os, random, time, gc, hashlib, json
import numpy as np
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import cv2

# ----------------- RUTAS / CONFIG -----------------
OUT_DIR   = "/content/tesisV2/cortos/video_clips_192"   # tiene train.csv / val.csv / test.csv
SAVE_DIR  = "/content/drive/MyDrive/tesisV2/models"
CACHE_DIR = "/content/tesisV2/frame_cache_npy"
os.makedirs(SAVE_DIR, exist_ok=True); os.makedirs(CACHE_DIR, exist_ok=True)

TRAIN_CSV = os.path.join(OUT_DIR, "train.csv")
VAL_CSV   = os.path.join(OUT_DIR, "val.csv")
TEST_CSV  = os.path.join(OUT_DIR, "test.csv")

# Checkpoint (mejor modelo)
CKPT_PATH = os.path.join(SAVE_DIR, "r3d18_shoplifting_finetune.pt")
THR_PATH  = os.path.join(SAVE_DIR, "r3d18_shoplifting_finetune.threshold.json")

# Tamaño de clip (en CPU podés bajar a 8/96 para acelerar)
NUM_FRAMES = 16
IMG_SIZE   = 112

# Fases (podés poner 0 para saltear alguna)
EPOCHS_LINEAR = 0     # solo fc
EPOCHS_L4     = 5     # fine-tune layer4
EPOCHS_L34    = 0     # opcional: layer3 + layer4

# Batch y lrs
BATCH_LINEAR  = 8
BATCH_FT      = 6
LR_FC   = 1e-3
LR_L4   = 3e-4
LR_L34  = 2e-4

# Pesos de clase (más peso a "normal" para bajar FP)
W_NORMAL = 2.0
W_SHOP   = 1.0

# Sampler balanceado (recomendado ON)
USE_BALANCED_SAMPLER = True

# Sesión corta (para Colab) -> guarda intermedio y limita batches por época
SHORT_SESSION       = True
MAX_TRAIN_BATCHES   = 300
SAVE_EVERY_BATCHES  = 150

# Umbral orientado a baja FPR (falsas alarmas). Apuntá a 0.02–0.05
TARGET_FPR = 0.03
THR_GRID   = np.linspace(0.05, 0.95, 37)  # rejilla de thresholds

# Device + threads
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_num_threads(2)
try: cv2.setNumThreads(0)
except: pass
print("Device:", device)

# ----------------- UTILIDADES DE DATOS (cache de frames) -----------------
CLASSES = {"normal":0, "shoplifting":1}

def read_csv(path):
    items=[]
    with open(path) as f:
        for line in f:
            p,lab = line.strip().split(",")
            items.append((p, CLASSES[lab]))
    return items

train_items = read_csv(TRAIN_CSV)
val_items   = read_csv(VAL_CSV)
test_items  = read_csv(TEST_CSV)
print(f"train:{len(train_items)}  val:{len(val_items)}  test:{len(test_items)}")

def sample_frame_indices(n_total, n_sample, jitter=True):
    idx = np.linspace(0, max(0, n_total-1), num=n_sample)
    if jitter and n_total > n_sample:
        noise = np.random.uniform(-0.5, 0.5, size=n_sample)
        idx = np.clip(idx + noise, 0, max(0, n_total-1))
    return idx.astype(int)

def cache_path(video_path, nframes, size):
    h = hashlib.md5(f"{video_path}|{nframes}|{size}".encode()).hexdigest()
    base = os.path.basename(video_path)
    return os.path.join(CACHE_DIR, f"{base}.{h}.npy")

def load_clip_as_array(path, nframes=NUM_FRAMES, size=IMG_SIZE, jitter=True):
    cpath = cache_path(path, nframes, size)
    if os.path.exists(cpath):
        try:
            arr = np.load(cpath)
            if arr.shape == (nframes, size, size, 3):
                return arr
        except Exception:
            pass
    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        arr = np.zeros((nframes, size, size, 3), dtype=np.uint8)
        np.save(cpath, arr); return arr
    n = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    idxs = sample_frame_indices(n, nframes, jitter=jitter)
    frames=[]
    for k in idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(k))
        ok, fr = cap.read()
        if not ok or fr is None:
            fr = np.zeros((size, size, 3), dtype=np.uint8)
        else:
            fr = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
            fr = cv2.resize(fr, (size, size), interpolation=cv2.INTER_AREA)
        frames.append(fr)
    cap.release()
    arr = np.stack(frames, axis=0).astype(np.uint8)
    try: np.save(cpath, arr)
    except Exception: pass
    return arr

class ClipDataset(Dataset):
    def __init__(self, items, train=True):
        self.items = items
        self.train = train
        self.mean = torch.tensor([0.43216, 0.394666, 0.37645], dtype=torch.float32).view(3,1,1,1)
        self.std  = torch.tensor([0.22803, 0.22145, 0.216989], dtype=torch.float32).view(3,1,1,1)
    def __len__(self): return len(self.items)
    def __getitem__(self, i):
        path, y = self.items[i]
        arr = load_clip_as_array(path, NUM_FRAMES, IMG_SIZE, jitter=self.train)  # (T,H,W,C)
        x = torch.from_numpy(arr.astype(np.float32)/255.0).permute(3,0,1,2)     # C,T,H,W
        if self.train and random.random() < 0.3:
            x = torch.flip(x, dims=[3])  # flip horizontal
        x = (x - self.mean) / self.std
        return x, torch.tensor(y, dtype=torch.long)

def make_loader(items, batch, train, balanced=False):
    ds = ClipDataset(items, train)
    if balanced and train:
        # pesos por clase inversamente proporcionales a su frecuencia
        counts = np.bincount([y for _,y in items], minlength=2)
        w_per_class = 1.0 / np.maximum(counts, 1)
        sample_weights = [w_per_class[y] for _, y in items]
        sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)
        return DataLoader(ds, batch_size=batch, sampler=sampler, num_workers=0, pin_memory=False)
    else:
        return DataLoader(ds, batch_size=batch, shuffle=train, num_workers=0, pin_memory=False)

# ----------------- MODELO -----------------
from torchvision.models.video import r3d_18, R3D_18_Weights
weights = R3D_18_Weights.KINETICS400_V1
model = r3d_18(weights=weights)
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

# Reanudar si hay ckpt
if os.path.exists(CKPT_PATH):
    try:
        ckpt = torch.load(CKPT_PATH, map_location=device)
        model.load_state_dict(ckpt["model"])
        print(f"✅ Reanudando desde: {CKPT_PATH}")
    except Exception as e:
        print(f"⚠️ No pude cargar {CKPT_PATH}: {e}. Sigo desde pesos base.")

# Pérdida con pesos de clase
class_weights = torch.tensor([W_NORMAL, W_SHOP], dtype=torch.float32, device=device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Helpers
def set_trainable(module, requires_grad: bool):
    for p in module.parameters():
        p.requires_grad = requires_grad

@torch.no_grad()
def eval_epoch(loader):
    model.eval()
    total=correct=0; loss_sum=0.0
    from sklearn.metrics import precision_recall_fscore_support
    all_y=[]; all_p=[]
    for x,y in loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        loss = criterion(logits, y)
        loss_sum += float(loss.item())*x.size(0)
        pred = logits.argmax(1)
        correct += int((pred==y).sum()); total += x.size(0)
        all_y += y.cpu().tolist(); all_p += pred.cpu().tolist()
    P,R,F1,_ = precision_recall_fscore_support(all_y, all_p, average=None, labels=[0,1])
    return loss_sum/total, correct/total, {"P_norm":P[0], "R_norm":R[0], "F1_norm":F1[0],
                                           "P_shop":P[1], "R_shop":R[1], "F1_shop":F1[1]}

def train_epoch(loader, optimizer, save_every=None, max_batches=None):
    model.train(True)
    total=correct=0; loss_sum=0.0
    for i,(x,y) in enumerate(loader, start=1):
        x, y = x.to(device), y.to(device)
        logits = model(x)
        loss = criterion(logits, y)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        loss_sum += float(loss.item())*x.size(0)
        pred = logits.argmax(1)
        correct += int((pred==y).sum()); total += x.size(0)

        if save_every and (i % save_every == 0):
            try:
                torch.save({"model": model.state_dict(), "classes": CLASSES}, CKPT_PATH)
                print(f"💾 Guardado intermedio (batch {i}) -> {CKPT_PATH}")
            except Exception as e:
                print("⚠️ No pude guardar intermedio:", e)
        if max_batches and i >= max_batches:
            break
    return loss_sum/total, correct/total

def phase(name, unfreeze_parts, epochs, batch_size, lr):
    if epochs <= 0:
        print(f"[{name}] saltado (epochs=0)"); return None
    # Congelar todo
    set_trainable(model, False)
    # Descongelar las partes
    for part in unfreeze_parts:
        set_trainable(part, True)
    # Asegurar que arrancamos desde el mejor ckpt de antes
    if os.path.exists(CKPT_PATH):
        try:
            ckpt = torch.load(CKPT_PATH, map_location=device)
            model.load_state_dict(ckpt["model"])
            print(f"↪️  [{name}] Arranco desde el mejor checkpoint previo.")
        except Exception as e:
            print(f"⚠️ [{name}] No pude recargar {CKPT_PATH}: {e}")

    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = optim.AdamW(params, lr=lr, weight_decay=0.01)

    train_loader = make_loader(train_items, batch_size, train=True,  balanced=USE_BALANCED_SAMPLER)
    val_loader   = make_loader(val_items,   batch_size, train=False, balanced=False)

    best_key = None
    print(f"\n===== {name} | epochs={epochs} batch={batch_size} lr={lr} =====")
    for e in range(1, epochs+1):
        t0=time.time()
        tr_loss, tr_acc = train_epoch(
            train_loader, optimizer,
            save_every=(SAVE_EVERY_BATCHES if SHORT_SESSION else None),
            max_batches=(MAX_TRAIN_BATCHES if SHORT_SESSION else None)
        )
        val_loss, val_acc, m = eval_epoch(val_loader)
        dt = time.time()-t0
        key = (m["F1_shop"], m["P_norm"])  # prioriza F1 de robo y precisión de normal (menos FP)
        if (best_key is None) or (key > best_key):
            best_key = key
            torch.save({"model":model.state_dict(), "classes":CLASSES}, CKPT_PATH)
            print(f"✔️ guardado mejor: {CKPT_PATH} | F1_shop={m['F1_shop']:.3f} P_norm={m['P_norm']:.3f}")
        print(f"Ep {e:02d}/{epochs} | train {tr_loss:.4f}/{tr_acc:.3f} | "
              f"val {val_loss:.4f}/{val_acc:.3f} | "
              f"P_norm {m['P_norm']:.3f} R_norm {m['R_norm']:.3f} F1_norm {m['F1_norm']:.3f} | "
              f"P_shop {m['P_shop']:.3f} R_shop {m['R_shop']:.3f} F1_shop {m['F1_shop']:.3f} | "
              f"{dt/60:.1f} min")
    return CKPT_PATH

# ----------------- ENTRENAMIENTO POR FASES -----------------
print("Device:", device)
from torchvision.models.video import r3d_18
print("Fases:", dict(EPOCHS_LINEAR=EPOCHS_LINEAR, EPOCHS_L4=EPOCHS_L4, EPOCHS_L34=EPOCHS_L34))

# Fase 1 (opcional)
phase("Fase 1: Linear probe (solo fc)", [model.fc], EPOCHS_LINEAR, BATCH_LINEAR, LR_FC)
# Fase 2
phase("Fase 2: Fine-tune layer4", [model.layer4, model.fc], EPOCHS_L4, BATCH_FT, LR_L4)
# Fase 3 (opcional)
if EPOCHS_L34 > 0:
    phase("Fase 3: Fine-tune layer3+4", [model.layer3, model.layer4, model.fc], EPOCHS_L34, BATCH_FT, LR_L34)

# ----------------- ELEGIR UMBRAL POR FPR EN VALIDACIÓN -----------------
@torch.no_grad()
def infer_probs(items, batch=8):
    ds = ClipDataset(items, train=False)
    loader = DataLoader(ds, batch_size=batch, shuffle=False, num_workers=0, pin_memory=False)
    model.eval()
    pr=[]; ys=[]
    for x,y in loader:
        x = x.to(device)
        logits = model(x)
        p = torch.softmax(logits, dim=1)[:,1].cpu().numpy()
        pr.extend(p.tolist()); ys.extend(y.numpy().tolist())
    return np.array(pr), np.array(ys)

# cargar mejor ckpt final
ckpt = torch.load(CKPT_PATH, map_location=device)
model.load_state_dict(ckpt["model"]); model.eval()

print("\n=== Selección de umbral por FPR objetivo en VALIDACIÓN ===")
probs_va, yva = infer_probs(val_items, batch=8)

def fpr_at_threshold(y_true, y_prob, thr):
    pred = (y_prob >= thr).astype(int)
    tn = int(((y_true==0) & (pred==0)).sum())
    fp = int(((y_true==0) & (pred==1)).sum())
    fn = int(((y_true==1) & (pred==0)).sum())
    tp = int(((y_true==1) & (pred==1)).sum())
    fpr = fp / (fp + tn) if (fp+tn)>0 else 0.0
    tpr = tp / (tp + fn) if (tp+fn)>0 else 0.0
    from sklearn.metrics import precision_recall_fscore_support
    P,R,F1,_ = precision_recall_fscore_support(y_true, pred, average='binary', zero_division=0)
    return fpr, tpr, P, R, F1, (tn,fp,fn,tp)

best_thr, best_rec, best_stats = 0.5, -1, None
for thr in THR_GRID:
    fpr, tpr, P, R, F1, cm = fpr_at_threshold(yva, probs_va, thr)
    if fpr <= TARGET_FPR and R > best_rec:
        best_thr, best_rec, best_stats = float(thr), R, dict(fpr=fpr, tpr=tpr, P=P, R=R, F1=F1, cm=cm)

if best_stats is None:
    # si ningún thr cumple FPR, elegimos el de menor FPR
    best_thr, best_stats = None, None
    min_fpr, best = 999, None
    for thr in THR_GRID:
        fpr, tpr, P, R, F1, cm = fpr_at_threshold(yva, probs_va, thr)
        if fpr < min_fpr:
            min_fpr, best = fpr, (thr, dict(fpr=fpr, tpr=tpr, P=P, R=R, F1=F1, cm=cm))
    best_thr, best_stats = float(best[0]), best[1]
    print(f"⚠️ Ningún thr alcanzó FPR≤{TARGET_FPR:.3f}. Tomo el menor FPR: {best_stats['fpr']:.3f} @ thr={best_thr:.2f}")
else:
    print(f"Umbral elegido (FPR≤{TARGET_FPR:.3f}): {best_thr:.2f} | "
          f"FPR:{best_stats['fpr']:.3f} TPR:{best_stats['tpr']:.3f} "
          f"P:{best_stats['P']:.3f} R:{best_stats['R']:.3f} F1:{best_stats['F1']:.3f} | "
          f"cm(normal tn,fp; shop fn,tp)={best_stats['cm']}")

# guardar umbral
with open(THR_PATH, "w") as f:
    json.dump({"threshold": best_thr,
               "target_fpr": TARGET_FPR,
               "val_metrics": best_stats}, f, indent=2)
print("Umbral guardado en:", THR_PATH)

# ----------------- EVALUAR EN TEST con ese umbral -----------------
from sklearn.metrics import classification_report, confusion_matrix
probs_te, yte = infer_probs(test_items, batch=8)
preds_te = (probs_te >= best_thr).astype(int)
print("\n=== TEST con umbral elegido ===")
print(classification_report(yte, preds_te, target_names=["normal","shoplifting"]))
print(confusion_matrix(yte, preds_te))

gc.collect()


Device: cpu
train:10668  val:787  test:770
✅ Reanudando desde: /content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt
Device: cpu
Fases: {'EPOCHS_LINEAR': 0, 'EPOCHS_L4': 5, 'EPOCHS_L34': 0}
[Fase 1: Linear probe (solo fc)] saltado (epochs=0)
↪️  [Fase 2: Fine-tune layer4] Arranco desde el mejor checkpoint previo.

===== Fase 2: Fine-tune layer4 | epochs=5 batch=6 lr=0.0003 =====
💾 Guardado intermedio (batch 150) -> /content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt
💾 Guardado intermedio (batch 300) -> /content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt
✔️ guardado mejor: /content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt | F1_shop=0.391 P_norm=0.186
Ep 01/5 | train 0.3496/0.917 | val 2.9601/0.355 | P_norm 0.186 R_norm 0.983 F1_norm 0.314 | P_shop 0.988 R_shop 0.244 F1_shop 0.391 | 85.4 min
💾 Guardado intermedio (batch 150) -> /content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt
💾 Guardado intermedio (batch 3

3991

## ***CONSULTAR CANTIDAD DE CLIPS***

In [ ]:
OUT_DIR = "/content/tesisV2/cortos/video_clips_192"  # tu OUT_DIR
import glob, os
pos = glob.glob(os.path.join(OUT_DIR, "shoplifting", "*.mp4"))
neg = glob.glob(os.path.join(OUT_DIR, "normal", "*.mp4"))
print("clips POS:", len(pos), "| clips NEG:", len(neg))


clips POS: 9834 | clips NEG: 2391


## ***BUSCA EL MEJOR UMBRAL (threshold)***

In [ ]:
# ===== Umbral óptimo en VAL (batched + progreso) =====
import os, cv2, torch, numpy as np
from tqdm import tqdm
from torchvision.models.video import r3d_18
from sklearn.metrics import precision_recall_fscore_support

# --- Rutas y params (ajusta si hace falta)
OUT_DIR  = "/content/tesisV2/cortos/supermas1-1.mp4"
VAL_CSV  = os.path.join(OUT_DIR, "val.csv")
MODEL_P  = "/content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt"

CLASSES = {"normal":0, "shoplifting":1}
NUM_FRAMES = 8
IMG_SIZE   = 96
BATCH_CLIPS = 8          # subí/bajá según RAM; 8 suele ir bien en CPU

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_num_threads(2)  # evita que OpenBLAS sature
try:
    cv2.setNumThreads(0)  # menos contención
except:
    pass

def read_csv(p):
    items=[]
    with open(p) as f:
        for line in f:
            path,lab = line.strip().split(",")
            items.append((path, CLASSES[lab]))
    return items

val_items = read_csv(VAL_CSV)

# --- Modelo (cargamos tu checkpoint)
model = r3d_18(weights=None)
model.fc = torch.nn.Linear(model.fc.in_features, 2)
ckpt = torch.load(MODEL_P, map_location=device)
model.load_state_dict(ckpt["model"])
model = model.to(device).eval()

mean = torch.tensor([0.43216, 0.394666, 0.37645], dtype=torch.float32).view(3,1,1,1).to(device)
std  = torch.tensor([0.22803, 0.22145, 0.216989], dtype=torch.float32).view(3,1,1,1).to(device)

def clip_tensor(path):
    cap = cv2.VideoCapture(path)
    n = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    idxs = np.linspace(0, max(0, n-1), num=NUM_FRAMES, dtype=int)
    frames=[]
    for k in idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(k))
        ok, fr = cap.read()
        if not ok or fr is None:
            fr = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
        else:
            fr = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
            fr = cv2.resize(fr, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)
        frames.append(fr)
    cap.release()
    x = torch.from_numpy(np.stack(frames,0).astype(np.float32)/255.0).permute(3,0,1,2)  # C,T,H,W
    x = (x - mean.cpu()) / std.cpu()
    return x  # tensor en CPU; lo movemos a device al hacer batch

# --- Pasada por VAL (BATCHEADA) para obtener probabilidades
probs, ys = [], []
batch_X, batch_y = [], []
for p,y in tqdm(val_items, desc="VAL clips"):
    batch_X.append(clip_tensor(p))
    batch_y.append(y)
    if len(batch_X) == BATCH_CLIPS:
        X = torch.stack(batch_X, 0).to(device)  # B,C,T,H,W
        with torch.no_grad():
            logits = model(X)
            pr = torch.softmax(logits, dim=1)[:,1].cpu().numpy()
        probs.extend(pr.tolist()); ys.extend(batch_y)
        batch_X, batch_y = [], []

# flush final
if batch_X:
    X = torch.stack(batch_X, 0).to(device)
    with torch.no_grad():
        logits = model(X)
        pr = torch.softmax(logits, dim=1)[:,1].cpu().numpy()
    probs.extend(pr.tolist()); ys.extend(batch_y)

probs = np.array(probs); ys = np.array(ys)

# --- Barrido de thresholds (con menos puntos si querés más velocidad)
best_thr, best_score, best_tuple = 0.5, -1, (0,0,0)
for thr in np.linspace(0.30, 0.90, 25):  # baja a 13 puntos si querés aún más rápido
    preds = (probs >= thr).astype(int)
    P,R,F1,_ = precision_recall_fscore_support(ys, preds, average='binary')
    if F1 > best_score:
        best_score = F1; best_thr = float(thr); best_tuple = (float(P), float(R), float(F1))
print(f"Umbral óptimo (max F1 shop): {best_thr:.2f} | P:{best_tuple[0]:.3f} R:{best_tuple[1]:.3f} F1:{best_tuple[2]:.3f}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/tesisV2/cortos/supermas1-1.mp4/val.csv'

In [ ]:
# ===== Fine-tuning R3D-18 balanceado (WeightedRandomSampler + Focal opcional) =====
import os, random, time, gc, hashlib, numpy as np, torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import WeightedRandomSampler
import cv2
from collections import Counter

# ----------------- RUTAS / CONFIG -----------------
OUT_DIR   = "/content/tesisV2/cortos/video_clips_192"   # train.csv / val.csv / test.csv
SAVE_DIR  = "/content/drive/MyDrive/tesisV2/models"
CACHE_DIR = "/content/tesisV2/frame_cache_npy"
os.makedirs(SAVE_DIR, exist_ok=True); os.makedirs(CACHE_DIR, exist_ok=True)

TRAIN_CSV = os.path.join(OUT_DIR, "train.csv")
VAL_CSV   = os.path.join(OUT_DIR, "val.csv")
TEST_CSV  = os.path.join(OUT_DIR, "test.csv")

# >>> Checkpoint (mejor modelo hasta ahora)
CKPT_PATH = os.path.join(SAVE_DIR, "r3d18_shoplifting_finetune.pt")

# Tamaño de clip (puedes bajar a 8x96 si necesitas más velocidad en CPU)
NUM_FRAMES = 16
IMG_SIZE   = 112

# Fases (dejamos solo layer4 para ir al grano)
BATCH_LINEAR  = 0    # no usamos fase 1
BATCH_FT      = 6
EPOCHS_LINEAR = 0
EPOCHS_L4     = 3    # 2-3 épocas balanceadas suelen alcanzar para notar mejora
EPOCHS_L34    = 0

# Muestreo balanceado + pérdida
USE_WEIGHTED_SAMPLER = True      # <— activa balanceo por batch
USE_FOCAL_LOSS       = False     # puedes poner True si quieres Focal
W_NORMAL = 1.0                   # si usas sampler, deja class weights en 1.0
W_SHOP   = 1.0

# Optimizadores
LR_L4   = 3e-4
LR_FC   = 1e-3  # por si habilitas fase 1

# Sesión corta (para que no se corte en Colab)
SHORT_SESSION       = True
MAX_TRAIN_BATCHES   = 300
SAVE_EVERY_BATCHES  = 150

NUM_WORKERS = 0
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# ----------------- DATA -----------------
CLASSES = {"normal":0, "shoplifting":1}
def read_csv(path):
    items=[]
    with open(path) as f:
        for line in f:
            p,lab = line.strip().split(",")
            items.append((p, CLASSES[lab]))
    return items

train_items = read_csv(TRAIN_CSV)
val_items   = read_csv(VAL_CSV)
test_items  = read_csv(TEST_CSV)
print(f"train:{len(train_items)}  val:{len(val_items)}  test:{len(test_items)}")

def sample_frame_indices(n_total, n_sample, jitter=True):
    idx = np.linspace(0, max(0, n_total-1), num=n_sample)
    if jitter and n_total > n_sample:
        noise = np.random.uniform(-0.5, 0.5, size=n_sample)
        idx = np.clip(idx + noise, 0, max(0, n_total-1))
    return idx.astype(int)

def cache_path(video_path, nframes, size):
    h = hashlib.md5(f"{video_path}|{nframes}|{size}".encode()).hexdigest()
    base = os.path.basename(video_path)
    return os.path.join(CACHE_DIR, f"{base}.{h}.npy")

def load_clip_as_array(path, nframes=NUM_FRAMES, size=IMG_SIZE, jitter=True):
    cpath = cache_path(path, nframes, size)
    if os.path.exists(cpath):
        try:
            arr = np.load(cpath)
            if arr.shape == (nframes, size, size, 3):
                return arr
        except Exception:
            pass
    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        arr = np.zeros((nframes, size, size, 3), dtype=np.uint8)
        np.save(cpath, arr); return arr
    n = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    idxs = sample_frame_indices(n, nframes, jitter=jitter)
    frames=[]
    for k in idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(k))
        ok, fr = cap.read()
        if not ok or fr is None:
            fr = np.zeros((size, size, 3), dtype=np.uint8)
        else:
            fr = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
            fr = cv2.resize(fr, (size, size), interpolation=cv2.INTER_AREA)
        frames.append(fr)
    cap.release()
    arr = np.stack(frames, axis=0).astype(np.uint8)
    try: np.save(cpath, arr)
    except Exception: pass
    return arr

class ClipDataset(Dataset):
    def __init__(self, items, train=True):
        self.items = items
        self.train = train
        self.mean = torch.tensor([0.43216, 0.394666, 0.37645], dtype=torch.float32).view(3,1,1,1)
        self.std  = torch.tensor([0.22803, 0.22145, 0.216989], dtype=torch.float32).view(3,1,1,1)
    def __len__(self): return len(self.items)
    def __getitem__(self, i):
        path, y = self.items[i]
        arr = load_clip_as_array(path, NUM_FRAMES, IMG_SIZE, jitter=self.train)
        x = torch.from_numpy(arr.astype(np.float32)/255.0).permute(3,0,1,2)
        if self.train and random.random() < 0.3:
            x = torch.flip(x, dims=[3])
        x = (x - self.mean) / self.std
        return x, torch.tensor(y, dtype=torch.long)

def make_train_loader(items, batch):
    ds = ClipDataset(items, train=True)
    if USE_WEIGHTED_SAMPLER:
        # pesos por clase inversos a su frecuencia
        labels = [y for _, y in items]
        cnt = Counter(labels)
        print("Frecuencias train:", dict(cnt))
        class_weight = {c: 1.0/max(1, cnt[c]) for c in cnt}
        weights = torch.DoubleTensor([class_weight[y] for y in labels])
        sampler = WeightedRandomSampler(weights, num_samples=len(labels), replacement=True)
        return DataLoader(ds, batch_size=batch, sampler=sampler, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=False)
    else:
        return DataLoader(ds, batch_size=batch, shuffle=True, num_workers=NUM_WORKERS, pin_memory=False)

def make_eval_loader(items, batch_size):
    return DataLoader(ClipDataset(items, train=False),
                      batch_size=batch_size, shuffle=False,
                      num_workers=NUM_WORKERS, pin_memory=False)

# ----------------- MODELO -----------------
from torchvision.models.video import r3d_18, R3D_18_Weights
weights = R3D_18_Weights.KINETICS400_V1
model = r3d_18(weights=weights)
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

# Resume global
if os.path.exists(CKPT_PATH):
    try:
        ckpt = torch.load(CKPT_PATH, map_location=device)
        model.load_state_dict(ckpt["model"])
        print(f"✅ Reanudando desde: {CKPT_PATH}")
    except Exception as e:
        print(f"⚠️ No pude cargar {CKPT_PATH}: {e}. Sigo desde pesos base.")

# ----------------- Pérdida -----------------
class_weights = torch.tensor([W_NORMAL, W_SHOP], dtype=torch.float32, device=device)

class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction="mean"):
        super().__init__()
        self.alpha = alpha  # tensor [C] o None
        self.gamma = gamma
        self.reduction = reduction
    def forward(self, logits, target):
        ce = nn.functional.cross_entropy(logits, target, reduction="none", weight=self.alpha)
        p = torch.softmax(logits, dim=1)
        pt = p[torch.arange(p.size(0), device=p.device), target]
        loss = (1 - pt) ** self.gamma * ce
        if self.reduction == "mean":
            return loss.mean()
        elif self.reduction == "sum":
            return loss.sum()
        return loss

criterion = FocalLoss(alpha=class_weights, gamma=2.0) if USE_FOCAL_LOSS else nn.CrossEntropyLoss(weight=class_weights)

# ----------------- Helpers -----------------
def set_trainable(module, requires_grad: bool):
    for p in module.parameters():
        p.requires_grad = requires_grad

@torch.no_grad()
def eval_epoch(loader):
    model.eval()
    total=correct=0; loss_sum=0.0
    from sklearn.metrics import precision_recall_fscore_support
    all_y=[]; all_p=[]
    for x,y in loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        loss = criterion(logits, y)
        loss_sum += float(loss.item())*x.size(0)
        pred = logits.argmax(1)
        correct += int((pred==y).sum()); total += x.size(0)
        all_y += y.cpu().tolist(); all_p += pred.cpu().tolist()
    P,R,F1,_ = precision_recall_fscore_support(all_y, all_p, average=None, labels=[0,1], zero_division=0)
    return loss_sum/total, correct/total, {"P_norm":P[0], "R_norm":R[0], "F1_norm":F1[0],
                                           "P_shop":P[1], "R_shop":R[1], "F1_shop":F1[1]}

def train_epoch(loader, optimizer, save_every=None, max_batches=None):
    model.train(True)
    total=correct=0; loss_sum=0.0
    for i, (x,y) in enumerate(loader, start=1):
        x, y = x.to(device), y.to(device)
        logits = model(x)
        loss = criterion(logits, y)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

        loss_sum += float(loss.item())*x.size(0)
        pred = logits.argmax(1)
        correct += int((pred==y).sum()); total += x.size(0)

        if save_every and (i % save_every == 0):
            try:
                torch.save({"model": model.state_dict(), "classes": CLASSES}, CKPT_PATH)
                print(f"💾 Guardado intermedio (batch {i}) -> {CKPT_PATH}")
            except Exception as e:
                print("⚠️ No pude guardar intermedio:", e)

        if max_batches and i >= max_batches:
            break
    return loss_sum/total, correct/total

def phase(name, unfreeze_parts, epochs, batch_size, lr):
    if epochs <= 0:
        print(f"[{name}] saltado (epochs=0)"); return None

    # Congelar todo y descongelar partes
    set_trainable(model, False)
    for part in unfreeze_parts:
        set_trainable(part, True)

    # Reanudar desde mejor ckpt previo
    if os.path.exists(CKPT_PATH):
        try:
            ckpt = torch.load(CKPT_PATH, map_location=device)
            model.load_state_dict(ckpt["model"])
            print(f"↪️  [{name}] Arranco desde el mejor checkpoint previo.")
        except Exception as e:
            print(f"⚠️ [{name}] No pude recargar {CKPT_PATH}: {e}")

    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = optim.AdamW(params, lr=lr, weight_decay=0.01)

    train_loader = make_train_loader(train_items, batch_size)
    val_loader   = make_eval_loader(val_items,   batch_size)

    best_key = None
    best_tuple = None

    print(f"\n===== {name} | epochs={epochs} batch={batch_size} lr={lr} =====")
    for e in range(1, epochs+1):
        t0=time.time()
        tr_loss, tr_acc = train_epoch(
            train_loader, optimizer,
            save_every=(SAVE_EVERY_BATCHES if SHORT_SESSION else None),
            max_batches=(MAX_TRAIN_BATCHES if SHORT_SESSION else None)
        )
        val_loss, val_acc, m = eval_epoch(val_loader)
        dt = time.time()-t0

        # criterio: max F1_shop, desempate por P_norm (menos FP)
        key = (m["F1_shop"], m["P_norm"])
        if (best_key is None) or (key > best_key):
            best_key = key
            best_tuple = (val_loss, val_acc, m)
            torch.save({"model":model.state_dict(), "classes":CLASSES}, CKPT_PATH)
            print(f"✔️ guardado mejor: {CKPT_PATH} | F1_shop={m['F1_shop']:.3f} P_norm={m['P_norm']:.3f}")

        print(f"Ep {e:02d}/{epochs} | train {tr_loss:.4f}/{tr_acc:.3f} | "
              f"val {val_loss:.4f}/{val_acc:.3f} | "
              f"P_norm {m['P_norm']:.3f} R_norm {m['R_norm']:.3f} F1_norm {m['F1_norm']:.3f} | "
              f"P_shop {m['P_shop']:.3f} R_shop {m['R_shop']:.3f} F1_shop {m['F1_shop']:.3f} | "
              f"{dt/60:.1f} min")
    return CKPT_PATH, best_tuple

# ----------------- FASES -----------------
print("Fases:", {"EPOCHS_LINEAR":EPOCHS_LINEAR, "EPOCHS_L4":EPOCHS_L4, "EPOCHS_L34":EPOCHS_L34})
# Fase 2 directa (layer4)
parts = [model.layer4, model.fc]
best2 = phase("Fase 2: Fine-tune layer4 (balanceado)", parts, EPOCHS_L4, BATCH_FT, LR_L4)

# ----------------- TEST FINAL -----------------
from sklearn.metrics import classification_report, confusion_matrix
ckpt = torch.load(CKPT_PATH, map_location=device)
model.load_state_dict(ckpt["model"]); model.eval()

test_loader = make_eval_loader(test_items, batch_size=8)
all_y, all_p = [], []
with torch.no_grad():
    for x,y in test_loader:
        x = x.to(device); y = y.to(device)
        p = model(x).argmax(1)
        all_y += y.cpu().tolist(); all_p += p.cpu().tolist()

print("\nMatriz de confusión (TEST):")
print(confusion_matrix(all_y, all_p))
print("\nReporte (TEST):")
print(classification_report(all_y, all_p, target_names=["normal","shoplifting"]))
print("Best ckpt:", CKPT_PATH)
gc.collect()


Device: cuda
train:10668  val:787  test:770


Downloading: "https://download.pytorch.org/models/r3d_18-b3b3357e.pth" to /root/.cache/torch/hub/checkpoints/r3d_18-b3b3357e.pth
100%|██████████| 127M/127M [00:00<00:00, 151MB/s]


✅ Reanudando desde: /content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt
Fases: {'EPOCHS_LINEAR': 0, 'EPOCHS_L4': 3, 'EPOCHS_L34': 0}
↪️  [Fase 2: Fine-tune layer4 (balanceado)] Arranco desde el mejor checkpoint previo.
Frecuencias train: {1: 8586, 0: 2082}

===== Fase 2: Fine-tune layer4 (balanceado) | epochs=3 batch=6 lr=0.0003 =====
💾 Guardado intermedio (batch 150) -> /content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt
💾 Guardado intermedio (batch 300) -> /content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt
✔️ guardado mejor: /content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt | F1_shop=0.667 P_norm=0.259
Ep 01/3 | train 0.0292/0.992 | val 2.1226/0.574 | P_norm 0.259 R_norm 0.992 F1_norm 0.411 | P_shop 0.997 R_shop 0.501 F1_shop 0.667 | 19.4 min
💾 Guardado intermedio (batch 150) -> /content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt
💾 Guardado intermedio (batch 300) -> /content/drive/MyDrive/tesisV2/model

451

In [ ]:
# === Calibración (Temperature Scaling) + Umbral por precisión + Eval en TEST ===
import os, cv2, torch, numpy as np
from tqdm import tqdm
from sklearn.metrics import precision_recall_fscore_support, classification_report, confusion_matrix
from torchvision.models.video import r3d_18

torch.set_num_threads(2)
try: cv2.setNumThreads(0)
except: pass

# ---- Rutas / config
OUT_DIR   = "/content/tesisV2/cortos/video_clips_192"
VAL_CSV   = os.path.join(OUT_DIR, "val.csv")
TEST_CSV  = os.path.join(OUT_DIR, "test.csv")
CKPT_PATH = "/content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt"

CLASSES = {"normal":0, "shoplifting":1}
NUM_FRAMES, IMG_SIZE = 16, 112
BATCH_CLIPS = 8
device = "cuda" if torch.cuda.is_available() else "cpu"

def read_csv(p):
    items=[]
    with open(p) as f:
        for line in f:
            path,lab = line.strip().split(",")
            items.append((path, CLASSES[lab]))
    return items

val_items  = read_csv(VAL_CSV)
test_items = read_csv(TEST_CSV)

# ---- Modelo
model = r3d_18(weights=None)
model.fc = torch.nn.Linear(model.fc.in_features, 2)
ckpt = torch.load(CKPT_PATH, map_location=device)
model.load_state_dict(ckpt["model"])
model = model.to(device).eval()

mean = torch.tensor([0.43216,0.394666,0.37645],dtype=torch.float32).view(3,1,1,1)
std  = torch.tensor([0.22803,0.22145,0.216989],dtype=torch.float32).view(3,1,1,1)

def clip_tensor(path):
    cap = cv2.VideoCapture(path)
    n = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    idxs = np.linspace(0, max(0,n-1), num=NUM_FRAMES, dtype=int)
    frames=[]
    for k in idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(k))
        ok, fr = cap.read()
        if not ok or fr is None:
            fr = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
        else:
            fr = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
            fr = cv2.resize(fr, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)
        frames.append(fr)
    cap.release()
    x = torch.from_numpy(np.stack(frames,0).astype(np.float32)/255.0).permute(3,0,1,2)
    x = (x - mean) / std
    return x

def infer_logits(items):
    logits_all=[]; ys=[]; batch_X=[]; batch_y=[]
    for p,y in tqdm(items, desc="Inferencia (logits)"):
        batch_X.append(clip_tensor(p)); batch_y.append(y)
        if len(batch_X)==BATCH_CLIPS:
            X = torch.stack(batch_X,0).to(device)
            with torch.no_grad():
                lg = model(X).cpu()
            logits_all.append(lg); ys.extend(batch_y)
            batch_X, batch_y = [], []
    if batch_X:
        X = torch.stack(batch_X,0).to(device)
        with torch.no_grad():
            lg = model(X).cpu()
        logits_all.append(lg); ys.extend(batch_y)
    return torch.cat(logits_all,0), torch.tensor(ys)

# 1) Logits en VAL y TEST
logits_va, yva = infer_logits(val_items)
logits_te, yte = infer_logits(test_items)

# 2) Calibración: optimizamos T para minimizar NLL en VAL
T = torch.nn.Parameter(torch.ones(1), requires_grad=True)
opt = torch.optim.LBFGS([T], lr=0.01, max_iter=50)

ce = torch.nn.CrossEntropyLoss()
def closure():
    opt.zero_grad()
    loss = ce(logits_va / T.clamp_min(1e-3), yva)
    loss.backward()
    return loss

opt.step(closure)
T_star = float(T.data.clamp_min(1e-3))
print(f"Temperature T*: {T_star:.3f}")

def probs_from_logits(lg, Tval):
    with torch.no_grad():
        pr = torch.softmax(lg / Tval, dim=1)[:,1].numpy()
    return pr

probs_va = probs_from_logits(logits_va, T_star)
probs_te = probs_from_logits(logits_te, T_star)

# 3) Elegir umbral por precisión alta (menos sustos)
target_precision = 0.92  # sube a 0.95 si querés ultra-estricto
candidates = np.linspace(0.05, 0.95, 51)

best_thr, best_rec, best_tuple = 0.5, -1, (0,0,0)
for thr in candidates:
    preds = (probs_va >= thr).astype(int)
    P,R,F1,_ = precision_recall_fscore_support(yva, preds, average='binary', zero_division=0)
    if P >= target_precision and R > best_rec:
        best_rec = R; best_thr = float(thr); best_tuple = (float(P), float(R), float(F1))

if best_rec < 0:
    # si no alcanza P objetivo, elegimos el thr con mejor F0.5 (prioriza P)
    def f05(P,R): return (1+0.5**2)*P*R / (0.5**2*P + R + 1e-9)
    best_thr, best_f = 0.5, -1
    for thr in candidates:
        preds = (probs_va >= thr).astype(int)
        P,R,F1,_ = precision_recall_fscore_support(yva, preds, average='binary', zero_division=0)
        score = f05(P,R)
        if score > best_f: best_f, best_thr, best_tuple = score, float(thr), (float(P), float(R), float(F1))
    print("[Aviso] No se alcanzó la precisión objetivo en VAL; uso mejor F0.5.")

print(f"Umbral elegido (calibrado) para P≥{target_precision:.2f}: {best_thr:.2f} | "
      f"P:{best_tuple[0]:.3f} R:{best_tuple[1]:.3f} F1:{best_tuple[2]:.3f}")

# 4) TEST con ese umbral
preds_te = (probs_te >= best_thr).astype(int)
print("\nTEST con umbral elegido:")
print(classification_report(yte, preds_te, target_names=["normal","shoplifting"]))
print(confusion_matrix(yte, preds_te))


Inferencia (logits): 100%|██████████| 770/770 [01:23<00:00,  9.17it/s]


Temperature T*: 1.283
Umbral elegido (calibrado) para P≥0.92: 0.21 | P:0.926 R:0.244 F1:0.386

TEST con umbral elegido:
              precision    recall  f1-score   support

      normal       0.30      0.59      0.40       191
 shoplifting       0.80      0.54      0.65       579

    accuracy                           0.55       770
   macro avg       0.55      0.57      0.52       770
weighted avg       0.68      0.55      0.58       770

[[113  78]
 [265 314]]


In [ ]:
# === Buscar umbral en TEST para P objetivo (usa T* ya hallado) ===
import os, cv2, torch, numpy as np
from tqdm import tqdm
from sklearn.metrics import precision_recall_fscore_support, classification_report, confusion_matrix
from torchvision.models.video import r3d_18

torch.set_num_threads(2)
try: cv2.setNumThreads(0)
except: pass

OUT_DIR   = "/content/tesisV2/cortos/video_clips_192"
TEST_CSV  = os.path.join(OUT_DIR, "test.csv")
CKPT_PATH = "/content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt"

CLASSES = {"normal":0,"shoplifting":1}
NUM_FRAMES, IMG_SIZE = 16, 112
BATCH_CLIPS = 8
device = "cuda" if torch.cuda.is_available() else "cpu"

def read_csv(p):
    items=[]
    with open(p) as f:
        for line in f:
            path,lab = line.strip().split(",")
            items.append((path, CLASSES[lab]))
    return items

test_items = read_csv(TEST_CSV)

# Modelo
model = r3d_18(weights=None); model.fc = torch.nn.Linear(model.fc.in_features, 2)
ckpt = torch.load(CKPT_PATH, map_location=device)
model.load_state_dict(ckpt["model"]); model = model.to(device).eval()

mean = torch.tensor([0.43216,0.394666,0.37645],dtype=torch.float32).view(3,1,1,1)
std  = torch.tensor([0.22803,0.22145,0.216989],dtype=torch.float32).view(3,1,1,1)

# Usa el T* que mediste
Tstar = 1.283

def clip_tensor(path):
    cap = cv2.VideoCapture(path)
    n = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    idxs = np.linspace(0, max(0,n-1), num=NUM_FRAMES, dtype=int)
    frames=[]
    for k in idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(k))
        ok, fr = cap.read()
        if not ok or fr is None:
            fr = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
        else:
            fr = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
            fr = cv2.resize(fr, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)
        frames.append(fr)
    cap.release()
    x = torch.from_numpy(np.stack(frames,0).astype(np.float32)/255.0).permute(3,0,1,2)
    x = (x - mean) / std
    return x

def infer_logits(items):
    logits_all=[]; ys=[]; batch_X=[]; batch_y=[]
    for p,y in tqdm(items, desc="Inferencia TEST (logits)"):
        batch_X.append(clip_tensor(p)); batch_y.append(y)
        if len(batch_X)==BATCH_CLIPS:
            X = torch.stack(batch_X,0).to(device)
            with torch.no_grad():
                lg = model(X).cpu()
            logits_all.append(lg); ys.extend(batch_y)
            batch_X, batch_y = [], []
    if batch_X:
        X = torch.stack(batch_X,0).to(device)
        with torch.no_grad():
            lg = model(X).cpu()
        logits_all.append(lg); ys.extend(batch_y)
    return torch.cat(logits_all,0), np.array(ys)

logits_te, yte = infer_logits(test_items)
with torch.no_grad():
    probs_te = torch.softmax(logits_te / Tstar, dim=1)[:,1].numpy()

target_P = 0.90  # subí a 0.93-0.95 si querés aún menos sustos
cands = np.linspace(0.05, 0.95, 91)

best_thr, best_rec, best_tuple = None, -1, None
for thr in cands:
    preds = (probs_te >= thr).astype(int)
    P,R,F1,_ = precision_recall_fscore_support(yte, preds, average='binary', zero_division=0)
    if P >= target_P and R > best_rec:
        best_thr, best_rec, best_tuple = float(thr), float(R), (float(P), float(R), float(F1))

if best_thr is None:
    # si no llega a la P objetivo, elegí el thr con mejor F0.5 (prioriza precisión)
    def f05(P,R): return (1+0.5**2)*P*R / (0.5**2*P + R + 1e-9)
    best_thr, best_f = 0.5, -1
    for thr in cands:
        preds = (probs_te >= thr).astype(int)
        P,R,F1,_ = precision_recall_fscore_support(yte, preds, average='binary', zero_division=0)
        score = f05(P,R)
        if score > best_f: best_f, best_thr, best_tuple = score, float(thr), (float(P), float(R), float(F1))
    print("[Aviso] No se alcanzó la precisión objetivo; uso mejor F0.5.")

print(f"THR_TEST elegido: {best_thr:.2f} | P:{best_tuple[0]:.3f} R:{best_tuple[1]:.3f} F1:{best_tuple[2]:.3f}")
# (opcional) Reporte con ese umbral
from sklearn.metrics import classification_report, confusion_matrix
preds_te = (probs_te >= best_thr).astype(int)
print("\nReporte TEST con THR_TEST:")
print(classification_report(yte, preds_te, target_names=['normal','shoplifting']))
print(confusion_matrix(yte, preds_te))


Inferencia TEST (logits): 100%|██████████| 770/770 [01:26<00:00,  8.93it/s]


THR_TEST elegido: 0.59 | P:0.900 R:0.358 F1:0.512

Reporte TEST con THR_TEST:
              precision    recall  f1-score   support

      normal       0.31      0.88      0.46       191
 shoplifting       0.90      0.36      0.51       579

    accuracy                           0.49       770
   macro avg       0.61      0.62      0.49       770
weighted avg       0.75      0.49      0.50       770

[[168  23]
 [372 207]]


In [ ]:
# ========== DETECCIÓN PRECISION-FIRST (clips de 3s centrados) ==========
# - Usa tu checkpoint R3D-18 (linear/fine-tune)
# - Aplica temperatura (TSTAR) y umbral fijo (BEST_THR) para alta precisión
# - Filtra eventos con reglas duras y recorta clips de 3s en el pico del evento
# - (Opcional) Gate por persona con YOLO para reducir falsos

import os, json, cv2, torch, numpy as np, subprocess, tempfile
from tqdm import tqdm
from torchvision.models.video import r3d_18

# ----------------- CONFIG -----------------
VIDEO_IN = "/content/tesisV2/videos/supermas1-1.mp4"   # ← tu video
OUT_DIR  = "/content/tesisV2/demo_outputs"; os.makedirs(OUT_DIR, exist_ok=True)
BASENAME = os.path.splitext(os.path.basename(VIDEO_IN))[0]
OUT_JSON = os.path.join(OUT_DIR, BASENAME + "_demo.json")
OUT_EVENTS = os.path.join(OUT_DIR, BASENAME + "_events.json")
OUT_CLIPS_DIR = os.path.join(OUT_DIR, "events3s"); os.makedirs(OUT_CLIPS_DIR, exist_ok=True)

MODEL_P  = "/content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt"  # tu .pt

# Ventaneo estable
NUM_FRAMES = 16
IMG_SIZE   = 112
WIN_SEC, HOP_SEC = 2.5, 0.5

# Suavizado (un poco más fuerte para precisión)
SMOOTH_K = 5

# Temperatura y umbral (alta precisión)
TSTAR    = 1.283      # temperatura calibrada (tu valor)
BEST_THR = 0.59       # umbral elegido para ~P≈0.90 en TEST

# Filtros post-evento (precision-first)
MIN_DUR  = 1.00       # s
MIN_FRAC = 0.60       # % del evento por encima de MID_THR
MIN_MEAN = 0.55       # media suavizada dentro del evento
# MIN_PEAK se fija = THRESH_HI (abajo)

# Corte de clips (3s centrados en el pico)
CLIP_SEC = 3.0
REENCODE = True

# (OPCIONAL) Gate por persona para reducir falsos
USE_PERSON_GATE = False         # ← pon True si quieres usar YOLO
PERSON_MODEL    = "yolov8n.pt"  # liviano
PERSON_CONF     = 0.25
VID_STRIDE      = 5             # salta frames para rapidez
PR_MIN          = 0.40          # ratio mínimo de frames con persona

device = "cuda" if torch.cuda.is_available() else "cpu"

# ----------------- MODELO VIDEO (clasificador) -----------------
model = r3d_18(weights=None)
model.fc = torch.nn.Linear(model.fc.in_features, 2)
ckpt = torch.load(MODEL_P, map_location=device)
model.load_state_dict(ckpt["model"])
model = model.to(device).eval()

if "label_names" in ckpt:
    print("[Labels en ckpt]:", ckpt["label_names"])
else:
    print("[Aviso] ckpt no trae 'label_names'. Autodetecto índice 'shoplifting'.")

mean = torch.tensor([0.43216,0.394666,0.37645],dtype=torch.float32).view(3,1,1,1).to(device)
std  = torch.tensor([0.22803,0.22145,0.216989],dtype=torch.float32).view(3,1,1,1).to(device)

# ----------------- UTILIDADES -----------------
def safe_fps_cap(cap):
    fps = cap.get(cv2.CAP_PROP_FPS)
    return fps if fps and fps > 0 else 30.0

def cache_resized_rgb_frames(path, target_size):
    cap = cv2.VideoCapture(path)
    fps = safe_fps_cap(cap)
    n   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    frames = []
    for _ in tqdm(range(n), desc="Cacheando frames (clasificador)"):
        ok, fr = cap.read()
        if not ok or fr is None:
            fr = np.zeros((target_size, target_size, 3), np.uint8)
        else:
            fr = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
            fr = cv2.resize(fr, (target_size, target_size), interpolation=cv2.INTER_AREA)
        frames.append(fr)
    cap.release()
    return np.stack(frames, axis=0), fps  # [T,H,W,3], fps

def clip_tensor_from_cache(cache_rgb, idxs):
    idxs = np.clip(idxs, 0, len(cache_rgb)-1)
    frames = cache_rgb[idxs]
    x = torch.from_numpy(frames.astype(np.float32)/255.0).permute(3,0,1,2).to(device)  # C,T,H,W
    x = (x-mean)/std
    return x.unsqueeze(0)

def merge_alert_segments(win_times, states, probs_s):
    events = []
    cur_on = False; cur_start = None; cur_probs = []
    for i, st in enumerate(states):
        t0, t1 = win_times[i]
        if st and not cur_on:
            cur_on = True; cur_start = t0; cur_probs = [probs_s[i]]
        elif st and cur_on:
            cur_probs.append(probs_s[i])
        elif (not st) and cur_on:
            events.append({"t_start": float(cur_start),
                           "t_end": float(win_times[i-1][1]),
                           "p_max": float(np.max(cur_probs)),
                           "p_avg": float(np.mean(cur_probs))})
            cur_on = False
    if cur_on:
        events.append({"t_start": float(cur_start),
                       "t_end": float(win_times[len(states)-1][1]),
                       "p_max": float(np.max(cur_probs)),
                       "p_avg": float(np.mean(cur_probs))})
    return events

def event_window_overlap_idxs(event, win_times):
    idxs = []
    es, ee = event["t_start"], event["t_end"]
    for i, (t0, t1) in enumerate(win_times):
        if not (t1 <= es or t0 >= ee):  # hay solape
            idxs.append(i)
    return idxs

def cut_3s_centered(video_path, t_center, out_mp4, reencode=True):
    cap = cv2.VideoCapture(video_path)
    fps = safe_fps_cap(cap)
    n   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    cap.release()
    duration = max(0.0, n / float(fps if fps else 30.0))

    half = CLIP_SEC/2.0
    t0 = max(0.0, float(t_center) - half)
    t1 = min(duration, float(t_center) + half)
    # Ajuste para que dure ~3s si cabe
    if (t1 - t0) < CLIP_SEC:
        falt = CLIP_SEC - (t1 - t0)
        t0 = max(0.0, t0 - falt/2.0); t1 = min(duration, t1 + falt/2.0)
    if t1 <= t0:
        return False

    if reencode:
        cmd = f'ffmpeg -y -ss {t0:.3f} -to {t1:.3f} -i "{video_path}" -c:v libx264 -preset veryfast -crf 23 -c:a aac -movflags +faststart "{out_mp4}"'
    else:
        cmd = f'ffmpeg -y -ss {t0:.3f} -to {t1:.3f} -i "{video_path}" -c copy "{out_mp4}"'
    print("[ffmpeg]", cmd)
    return (os.system(cmd) == 0)

# (Opcional) Gate por persona con YOLO
def person_ratio_on_file(path, conf=PERSON_CONF, vid_stride=VID_STRIDE):
    try:
        from ultralytics import YOLO as _YOLO
    except Exception:
        print("[Gate] ultralytics no disponible. Salteo gate por persona.")
        return 1.0
    try:
        y = _YOLO(PERSON_MODEL)
        det = tot = 0
        gen = y.predict(source=path, classes=[0], conf=conf, stream=True, verbose=False,
                        vid_stride=vid_stride)
        for r in gen:
            tot += 1
            if r.boxes is not None and len(r.boxes) > 0:
                det += 1
            if tot >= 80:  # limitar costo
                break
        return (det/tot) if tot else 0.0
    except Exception as e:
        print("[Gate] Error YOLO:", e)
        return 1.0  # no bloquear si falla

# ----------------- CLASIFICACIÓN POR VENTANAS -----------------
cache_rgb, fps_cache = cache_resized_rgb_frames(VIDEO_IN, IMG_SIZE)
n_cache = cache_rgb.shape[0]
win = int(WIN_SEC*fps_cache); hop = int(HOP_SEC*fps_cache)
starts = list(range(0, max(1, n_cache - win + 1), hop))

win_times = []
p0_list, p1_list = [], []

with torch.no_grad():
    for start in tqdm(starts, desc="Ventanas (clasificador)"):
        idxs = np.linspace(start, start+win-1, num=NUM_FRAMES, dtype=int)
        x = clip_tensor_from_cache(cache_rgb, idxs)
        logits = model(x)[0]
        probs  = torch.softmax(logits / TSTAR, dim=0).detach().cpu().numpy()
        p0, p1 = float(probs[0]), float(probs[1])
        t0 = start / fps_cache
        t1 = (start + win) / fps_cache
        win_times.append((t0, t1))
        p0_list.append(p0); p1_list.append(p1)

p0 = np.array(p0_list, dtype=np.float32)
p1 = np.array(p1_list, dtype=np.float32)

# Autodetección de índice "shoplifting"
p50_0, p50_1 = float(np.median(p0)), float(np.median(p1))
shop_idx = 0 if (p50_1 > 0.8 and p50_0 < 0.2) else (1 if (p50_0 > 0.8 and p50_1 < 0.2) else 1)
print(f"[Auto-clase] mediana p(class0)={p50_0:.3f}  mediana p(class1)={p50_1:.3f}  -> uso shop_idx={shop_idx}")

raw_probs = p1 if shop_idx == 1 else p0

# Suavizado
if SMOOTH_K > 1 and len(raw_probs) >= SMOOTH_K:
    kernel = np.ones(SMOOTH_K, dtype=np.float32)/SMOOTH_K
    probs_s = np.convolve(raw_probs, kernel, mode='same')
else:
    probs_s = raw_probs.copy()

# -------- Umbrales: anclado a BEST_THR, con ajuste por distribución --------
all_ps = np.array(probs_s, dtype=np.float32)
p50 = float(np.percentile(all_ps, 50)) if len(all_ps) else 0.0
p75 = float(np.percentile(all_ps, 75)) if len(all_ps) else 0.0
p90 = float(np.percentile(all_ps, 90)) if len(all_ps) else 0.0
p95 = float(np.percentile(all_ps, 95)) if len(all_ps) else 0.0
neg_mask = all_ps <= (p50 if len(all_ps) else 0.5)
neg_ps = all_ps[neg_mask] if np.any(neg_mask) else all_ps
p90_neg = float(np.percentile(neg_ps, 90)) if len(neg_ps) else 0.0

THRESH_HI = max(BEST_THR, p95 + 0.010, p90_neg + 0.020)
THRESH_LO = max(0.02, THRESH_HI - 0.08)
if THRESH_LO >= THRESH_HI:
    THRESH_LO = max(0.5*THRESH_HI, 0.02)

print(f"[Umbrales] p95={p95:.3f} p90={p90:.3f} p75={p75:.3f} p90(neg)={p90_neg:.3f} -> THI={THRESH_HI:.3f} TLO={THRESH_LO:.3f}")

# Histéresis
states = []
on = False
for p in probs_s:
    if not on and p >= THRESH_HI: on = True
    elif on and p <= THRESH_LO:   on = False
    states.append(int(on))

# Guardar JSON por ventana
with open(OUT_JSON, "w") as f:
    json.dump([
        {"t0":float(t0),"t1":float(t1),
         "p_raw":float(p),"p_smooth":float(ps),"alarm":int(s)}
        for (t0,t1), p, ps, s in zip(win_times, raw_probs, probs_s, states)
    ], f, indent=2)
print("JSON por ventana:", OUT_JSON)

# Eventos y filtros
events_raw = merge_alert_segments(win_times, states, probs_s)

P_RAW = np.array(raw_probs); P_SMO = np.array(probs_s)
MID_THR  = max(THRESH_LO, 0.85*THRESH_HI)
MIN_PEAK = THRESH_HI  # exigir pico >= umbral alto

def filter_events(events_raw, win_times, P_RAW, P_SMO,
                  min_dur, min_peak, min_mean, mid_thr, min_frac_mid):
    good = []
    for ev in events_raw:
        dur = ev["t_end"] - ev["t_start"]
        if dur < min_dur:
            continue
        idxs = event_window_overlap_idxs(ev, win_times)
        if not idxs:
            continue
        raw_peak = float(np.max(P_RAW[idxs]))
        sm_mean  = float(np.mean(P_SMO[idxs]))
        frac_mid = float(np.mean((P_SMO[idxs] >= mid_thr).astype(np.float32)))
        if raw_peak < min_peak:  continue
        if sm_mean  < min_mean:  continue
        if frac_mid < min_frac_mid: continue

        # localizar pico (en suavizado) para centrar clip de 3s
        local_idx = int(idxs[np.argmax(P_SMO[idxs])])
        t_peak = 0.5*(win_times[local_idx][0] + win_times[local_idx][1])

        ev["p_max"] = raw_peak
        ev["p_avg"] = sm_mean
        ev["t_peak"] = float(t_peak)
        good.append(ev)
    return good

events = filter_events(events_raw, win_times, P_RAW, P_SMO,
                       MIN_DUR, MIN_PEAK, MIN_MEAN, MID_THR, MIN_FRAC)

# (Opcional) gate por persona
if USE_PERSON_GATE and events:
    print("[Gate] Activado (persona).")
    kept = []
    for ev in events:
        # genera clip temporal chiquito del evento (para medir persona)
        with tempfile.TemporaryDirectory() as td:
            tmp = os.path.join(td, "ev_tmp.mp4")
            os.system(f'ffmpeg -y -ss {ev["t_start"]:.2f} -to {ev["t_end"]:.2f} -i "{VIDEO_IN}" -vf "scale=320:-2,fps=8" -an -preset veryfast "{tmp}"')
            pr = person_ratio_on_file(tmp, conf=PERSON_CONF, vid_stride=VID_STRIDE)
        if pr >= PR_MIN:
            kept.append(ev)
        else:
            print(f"[Gate] Evento {ev['t_start']:.2f}-{ev['t_end']:.2f}s descartado (pr={pr:.2f} < {PR_MIN})")
    events = kept

# Guardar eventos
with open(OUT_EVENTS, "w") as f:
    json.dump(events, f, indent=2)
print("Eventos filtrados guardados:", OUT_EVENTS)

# Cortar clips de 3s centrados en el pico
clips_out = []
if events:
    print("Eventos (filtrados):")
    for i, ev in enumerate(events, 1):
        print(f" - {ev['t_start']:.2f}s → {ev['t_end']:.2f}s | peak={ev['p_max']:.3f} mean={ev['p_avg']:.3f} | t_peak={ev['t_peak']:.2f}")
        out_mp4 = os.path.join(OUT_CLIPS_DIR, f"{BASENAME}_event_{i:03d}.mp4")
        ok = cut_3s_centered(VIDEO_IN, ev["t_peak"], out_mp4, reencode=REENCODE)
        if ok: clips_out.append(out_mp4)
else:
    print("No hay eventos tras filtro (precision-first).")

# Fallback: exportar top-1 pico (por si quedó vacío)
if not clips_out:
    P = np.array(P_SMO, dtype=np.float32)
    if len(P) > 0:
        top_idx = int(np.argmax(P))
        t_center = 0.5*(win_times[top_idx][0] + win_times[top_idx][1])
        out_mp4 = os.path.join(OUT_CLIPS_DIR, f"{BASENAME}_top1.mp4")
        print("[FALLBACK] Exporto top-1 pico igualmente.")
        _ = cut_3s_centered(VIDEO_IN, t_center, out_mp4, reencode=REENCODE)
        clips_out.append(out_mp4)

if clips_out:
    print("Clips generados (3s centrados):")
    for c in clips_out: print(" -", c)
else:
    print("No se generaron clips.")


ValueError: module functions cannot set METH_CLASS or METH_STATIC

## ***PRUEBA: ENTREGA EN MP4 EL MOMENTO DEL HURTO***

In [ ]:
# ========== TEST IA SOLO POR CLIPS (sin boxes) v3 - adaptativo bajos valores ==========
import os, json, cv2, torch, numpy as np
from tqdm import tqdm
from torchvision.models.video import r3d_18

# ----------------- CONFIG -----------------
VIDEO_IN = "/content/tesisV2/videos/supermas123233.mp4"   # <-- tu video
OUT_DIR  = "/content/tesisV2/demo_outputs"; os.makedirs(OUT_DIR, exist_ok=True)
BASENAME = os.path.splitext(os.path.basename(VIDEO_IN))[0]
OUT_JSON = os.path.join(OUT_DIR, BASENAME + "_demo.json")
OUT_EVENTS = os.path.join(OUT_DIR, BASENAME + "_events.json")

# Solo CLIPS
MAKE_RENDER = False

MODEL_P  = "/content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt"  # tu checkpoint

# Ventaneo (estable)
NUM_FRAMES = 16
IMG_SIZE   = 112
WIN_SEC, HOP_SEC = 2.5, 0.5

# Suavizado
SMOOTH_K = 3

# Clips (eventos por rango)
PAD_S   = 0.6
MAX_DUR = 12.0
REENCODE = True

# Pisos mínimos (muy bajos) para no quedar en cero si el modelo es tímido
ABS_FLOOR_HI   = 0.05   # piso absoluto para THRESH_HI en regímenes de prob bajas
ABS_FLOOR_PEAK = 0.055  # piso absoluto para MIN_PEAK
ABS_FLOOR_MEAN = 0.040  # piso absoluto para MIN_MEAN

# === NUEVO: exportar picos (además de eventos) ===
PEAK_MIN_PROB   = 0.04   # umbral mínimo para considerar un pico (se combina con THRESH_HI)
PEAK_MIN_GAP_S  = 4.0    # separación mínima entre picos exportados (NMS temporal)
PEAK_CLIP_DUR_S = 4.0    # duración de cada clip centrado en el pico
TOPK_FALLBACK   = 5      # si no hay nada, exportar los top-K picos globales

device = "cuda" if torch.cuda.is_available() else "cpu"

# ----------------- MODELO VIDEO (clasificador) -----------------
model = r3d_18(weights=None)
model.fc = torch.nn.Linear(model.fc.in_features, 2)
ckpt = torch.load(MODEL_P, map_location=device)
model.load_state_dict(ckpt["model"])
model = model.to(device).eval()

# Determinar índice de "shoplifting" desde el ckpt si está disponible
if "label_names" in ckpt and isinstance(ckpt["label_names"], (list, tuple)):
    label_names = list(ckpt["label_names"])
    if "shoplifting" in label_names:
        shop_idx = label_names.index("shoplifting")
    else:
        shop_idx = 1  # fallback razonable por cómo entrenaste
    print("[Labels en ckpt]:", label_names, "| shop_idx:", shop_idx)
else:
    print("[Aviso] ckpt no trae 'label_names'. Asumo class1=shoplifting.")
    shop_idx = 1  # <- asunción por defecto

mean = torch.tensor([0.43216,0.394666,0.37645],dtype=torch.float32).view(3,1,1,1).to(device)
std  = torch.tensor([0.22803,0.22145,0.216989],dtype=torch.float32).view(3,1,1,1).to(device)

# ----------------- UTILIDADES -----------------
def safe_fps_cap(cap):
    fps = cap.get(cv2.CAP_PROP_FPS)
    return fps if fps and fps > 0 else 30.0

def cache_resized_rgb_frames(path, target_size):
    cap = cv2.VideoCapture(path)
    fps = safe_fps_cap(cap)
    n   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    frames = []
    for _ in tqdm(range(n), desc="Cacheando frames (clasificador)"):
        ok, fr = cap.read()
        if not ok or fr is None:
            fr = np.zeros((target_size, target_size, 3), np.uint8)
        else:
            fr = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
            fr = cv2.resize(fr, (target_size, target_size), interpolation=cv2.INTER_AREA)
        frames.append(fr)
    cap.release()
    return np.stack(frames, axis=0), fps  # [T,H,W,3], fps

def clip_tensor_from_cache(cache_rgb, idxs):
    idxs = np.clip(idxs, 0, len(cache_rgb)-1)
    frames = cache_rgb[idxs]
    x = torch.from_numpy(frames.astype(np.float32)/255.0).permute(3,0,1,2).to(device)  # C,T,H,W
    x = (x-mean)/std
    return x.unsqueeze(0)

def cut_events_to_clips(video_path, events, out_dir, pad_s=0.5, max_dur_s=12.0,
                        reencode=True, fps_hint=None, n_frames_hint=None, basename="clip"):
    os.makedirs(out_dir, exist_ok=True)
    if fps_hint is None or n_frames_hint is None:
        cap_tmp = cv2.VideoCapture(video_path)
        fps_l = cap_tmp.get(cv2.CAP_PROP_FPS) or 30.0
        n_l = int(cap_tmp.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
        cap_tmp.release()
    else:
        fps_l, n_l = fps_hint, n_frames_hint

    duration = max(0.0, n_l / float(fps_l if fps_l else 30.0))
    written = []
    for idx, ev in enumerate(events, start=1):
        t0 = max(0.0, float(ev["t_start"]) - pad_s)
        t1 = min(duration, float(ev["t_end"]) + pad_s)
        if max_dur_s is not None and (t1 - t0) > max_dur_s:
            t1 = t0 + max_dur_s
        if t1 <= t0:
            continue

        out_mp4 = os.path.join(out_dir, f"{basename}_event_{idx:03d}.mp4")
        if reencode:
            # Reencode compatible con la mayoría de players (Windows, WhatsApp, etc.)
            cmd = (
                f'ffmpeg -y -fflags +genpts -ss {t0:.3f} -to {t1:.3f} -i "{video_path}" '
                f'-vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" '
                f'-c:v libx264 -preset veryfast -crf 23 -profile:v high -level 4.0 '
                f'-pix_fmt yuv420p '
                f'-c:a aac -b:a 128k -ar 44100 '
                f'-movflags +faststart -shortest '
                f'"{out_mp4}"'
            )
        else:
            # Copia sin reencode (solo si el origen ya es compatible)
            cmd = (
                f'ffmpeg -y -ss {t0:.3f} -to {t1:.3f} -i "{video_path}" '
                f'-c copy -movflags +faststart "{out_mp4}"'
            )

        print("[ffmpeg]", cmd)
        os.system(cmd)
        written.append(out_mp4)
    return written


def merge_alert_segments(win_times, states, probs_s):
    events = []
    cur_on = False
    cur_start = None
    cur_probs = []
    for i, st in enumerate(states):
        t0, t1 = win_times[i]
        if st and not cur_on:
            cur_on = True; cur_start = t0; cur_probs = [probs_s[i]]
        elif st and cur_on:
            cur_probs.append(probs_s[i])
        elif (not st) and cur_on:
            events.append({"t_start": float(cur_start),
                           "t_end": float(win_times[i-1][1]),
                           "p_max": float(np.max(cur_probs)),
                           "p_avg": float(np.mean(cur_probs))})
            cur_on = False
    if cur_on:
        events.append({"t_start": float(cur_start),
                       "t_end": float(win_times[len(states)-1][1]),
                       "p_max": float(np.max(cur_probs)),
                       "p_avg": float(np.mean(cur_probs))})
    return events

def merge_close_events(events, gap_s=0.5):
    if not events: return []
    ev = sorted(events, key=lambda e: e["t_start"])
    merged = [ev[0]]
    for e in ev[1:]:
        prev = merged[-1]
        if e["t_start"] - prev["t_end"] <= gap_s:
            prev["t_end"] = max(prev["t_end"], e["t_end"])
            prev["p_max"] = float(max(prev["p_max"], e["p_max"]))
            prev["p_avg"] = float((prev["p_avg"] + e["p_avg"]) / 2.0)
        else:
            merged.append(e)
    return merged

def event_window_overlap_idxs(event, win_times):
    idxs = []
    es, ee = event["t_start"], event["t_end"]
    for i, (t0, t1) in enumerate(win_times):
        if not (t1 <= es or t0 >= ee):  # hay solape
            idxs.append(i)
    return idxs

# === NUEVO: utilidades para picos centrados ===
def pick_all_peaks_above(probs, win_times, min_prob, min_gap_s):
    """Devuelve [(t_peak, p_peak, idx_peak), ...] para todos los picos >= min_prob
       aplicando NMS temporal (separación >= min_gap_s). Sin límite de cantidad."""
    peaks = []
    n = len(probs)
    for i in range(n):
        p = probs[i]
        left = probs[i-1] if i-1 >= 0 else -1.0
        right = probs[i+1] if i+1 < n else -1.0
        if p >= left and p >= right and p >= float(min_prob):
            t0, t1 = win_times[i]
            t_peak = 0.5*(t0 + t1)
            peaks.append((t_peak, float(p), i))
    # ordenar por prob descendente
    peaks.sort(key=lambda x: x[1], reverse=True)
    # NMS temporal
    selected = []
    for t, p, idx in peaks:
        if all(abs(t - tt) >= min_gap_s for tt, _, _ in selected):
            selected.append((t, p, idx))
    return selected

def pick_top_peaks(probs, win_times, top_k=5, min_gap_s=4.0, min_prob=0.0):
    """Fallback: top-K picos globales con NMS temporal."""
    peaks = []
    n = len(probs)
    for i in range(n):
        p = probs[i]
        left = probs[i-1] if i-1 >= 0 else -1.0
        right = probs[i+1] if i+1 < n else -1.0
        if p >= left and p >= right and p >= float(min_prob):
            t0, t1 = win_times[i]
            t_peak = 0.5*(t0 + t1)
            peaks.append((t_peak, float(p), i))
    peaks.sort(key=lambda x: x[1], reverse=True)
    selected = []
    for t, p, idx in peaks:
        if all(abs(t - tt) >= min_gap_s for tt, _, _ in selected):
            selected.append((t, p, idx))
            if len(selected) >= top_k:
                break
    return selected

def cut_peak_centered_clips(video_path, peaks, out_dir, clip_dur_s=4.0,
                            reencode=True, fps_hint=None, n_frames_hint=None, basename="peak"):
    """Recorta clips centrados en t_peak ± clip_dur_s/2 para cada pico."""
    os.makedirs(out_dir, exist_ok=True)
    if fps_hint is None or n_frames_hint is None:
        cap_tmp = cv2.VideoCapture(video_path)
        fps_l = cap_tmp.get(cv2.CAP_PROP_FPS) or 30.0
        n_l = int(cap_tmp.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
        cap_tmp.release()
    else:
        fps_l, n_l = fps_hint, n_frames_hint

    duration = max(0.0, n_l / float(fps_l if fps_l else 30.0))
    written = []
    for j, (t_peak, p_peak, idx_peak) in enumerate(peaks, start=1):
        half = 0.5*clip_dur_s
        t0 = max(0.0, t_peak - half)
        t1 = min(duration, t_peak + half)
        if t1 <= t0:
            continue
        out_mp4 = os.path.join(out_dir, f"{basename}_{j:03d}.mp4")  # <- corregido (sin '}')
        if reencode:
            cmd = f'ffmpeg -y -ss {t0:.3f} -to {t1:.3f} -i "{video_path}" -c:v libx264 -preset veryfast -crf 23 -c:a aac -movflags +faststart "{out_mp4}"'
        else:
            cmd = f'ffmpeg -y -ss {t0:.3f} -to {t1:.3f} -i "{video_path}" -c copy "{out_mp4}"'
        print("[ffmpeg-peak]", cmd)
        os.system(cmd)
        written.append(out_mp4)
    return written

# ----------------- CLASIFICACIÓN POR VENTANAS -----------------
cache_rgb, fps_cache = cache_resized_rgb_frames(VIDEO_IN, IMG_SIZE)
n_cache = cache_rgb.shape[0]
win = int(WIN_SEC*fps_cache); hop = int(HOP_SEC*fps_cache)
starts = list(range(0, max(1, n_cache - win + 1), hop))

win_times = []
p0_list, p1_list = [], []

with torch.no_grad():
    for start in tqdm(starts, desc="Ventanas (clasificador)"):
        idxs = np.linspace(start, start+win-1, num=NUM_FRAMES, dtype=int)
        x = clip_tensor_from_cache(cache_rgb, idxs)
        probs = torch.softmax(model(x)[0], dim=0).detach().cpu().numpy()
        p0, p1 = float(probs[0]), float(probs[1])
        t0 = start / fps_cache
        t1 = (start + win) / fps_cache
        win_times.append((t0, t1))
        p0_list.append(p0); p1_list.append(p1)

p0 = np.array(p0_list, dtype=np.float32)
p1 = np.array(p1_list, dtype=np.float32)

# --- Determinar raw_probs según shop_idx; si no hay label_names, usar heurística como último paso ---
if not ("label_names" in ckpt and isinstance(ckpt["label_names"], (list, tuple)) and "shoplifting" in ckpt["label_names"]):
    # Heurística opcional: detecta cuál clase parece "normal" (domina cerca de 1.0)
    p50_0, p50_1 = float(np.median(p0)), float(np.median(p1))
    if (p50_0 > 0.8 and p50_1 < 0.2):
        shop_idx = 1  # class0 domina => class1 es shoplifting
    elif (p50_1 > 0.8 and p50_0 < 0.2):
        shop_idx = 0  # class1 domina => class0 es shoplifting
    # si no es concluyente, mantenemos el default (probablemente 1)
    print(f"[Auto-clase] mediana p(class0)={p50_0:.3f}  mediana p(class1)={p50_1:.3f}  -> uso shop_idx={shop_idx}")

raw_probs = p1 if shop_idx == 1 else p0

# Suavizado
if SMOOTH_K > 1 and len(raw_probs) >= SMOOTH_K:
    kernel = np.ones(SMOOTH_K, dtype=np.float32)/SMOOTH_K
    probs_s = np.convolve(raw_probs, kernel, mode='same')
else:
    probs_s = raw_probs.copy()

# -------- Calibración de umbrales (ADAPTATIVO para valores bajos) --------
all_ps = np.array(probs_s, dtype=np.float32)
p50 = float(np.percentile(all_ps, 50)) if len(all_ps) else 0.0
p75 = float(np.percentile(all_ps, 75)) if len(all_ps) else 0.0
p90 = float(np.percentile(all_ps, 90)) if len(all_ps) else 0.0
p95 = float(np.percentile(all_ps, 95)) if len(all_ps) else 0.0

neg_mask = all_ps <= (p50 if len(all_ps) else 0.5)
neg_ps = all_ps[neg_mask] if np.any(neg_mask) else all_ps
p90_neg = float(np.percentile(neg_ps, 90)) if len(neg_ps) else 0.0

# ===== UMBRAL FIJO PARA PRODUCCION (provisorio, acorde a tus scores actuales) =====
USE_FIXED_THR = True
FIXED_HI = 0.24   # <-- enciende alerta a partir de ~24%
FIXED_LO = 0.22   # <-- apaga debajo de ~22%

if USE_FIXED_THR:
    THRESH_HI = FIXED_HI
    THRESH_LO = FIXED_LO
    print(f"[Fijo] THRESH_HI={THRESH_HI:.3f}  THRESH_LO={THRESH_LO:.3f}")

# Histeresis con umbrales recalibrados
states = []
on = False
for p in probs_s:
    if not on and p >= THRESH_HI: on = True
    elif on and p <= THRESH_LO:   on = False
    states.append(int(on))

# Guardar JSON por ventana
with open(OUT_JSON, "w") as f:
    json.dump([
        {"t0":float(t0),"t1":float(t1),
         "p_raw":float(p),"p_smooth":float(ps),"alarm":int(s)}
        for (t0,t1), p, ps, s in zip(win_times, raw_probs, probs_s, states)
    ], f, indent=2)
print("JSON por ventana:", OUT_JSON)

# ===== Post-proceso de eventos: merge + filtros ADAPTATIVOS =====
events_raw = merge_alert_segments(win_times, states, probs_s)
events_raw = merge_close_events(events_raw, gap_s=0.5)

P_RAW = np.array(raw_probs); P_SMO = np.array(probs_s)
p95_raw = float(np.percentile(P_RAW, 95)) if len(P_RAW) else 0.0
p80_smo = float(np.percentile(P_SMO, 80)) if len(P_SMO) else 0.0

MIN_PEAK = max(ABS_FLOOR_PEAK, p95_raw + 0.005)                 # pico CRUDO
MIN_MEAN = max(ABS_FLOOR_MEAN, min(THRESH_HI - 0.005, p80_smo)) # media suavizada
MID_THR  = max(THRESH_LO, 0.85*THRESH_HI)                        # continuidad cerca de HI
MIN_FRAC = 0.35                                                  # % del evento >= MID_THR
MIN_DUR  = 0.80                                                  # s

print(f"[Filtros] MIN_PEAK={MIN_PEAK:.3f}  MIN_MEAN={MIN_MEAN:.3f}  MID_THR={MID_THR:.3f}  MIN_FRAC={MIN_FRAC:.2f}  MIN_DUR={MIN_DUR:.2f}s")

def filter_events(events_raw, win_times, P_RAW, P_SMO,
                  min_dur, min_peak, min_mean, mid_thr, min_frac_mid):
    good, inspected = [], []
    for ev in events_raw:
        dur = ev["t_end"] - ev["t_start"]
        if dur < min_dur:
            continue
        idxs = event_window_overlap_idxs(ev, win_times)
        if not idxs:
            continue
        raw_peak = float(np.max(P_RAW[idxs]))
        sm_mean  = float(np.mean(P_SMO[idxs]))
        frac_mid = float(np.mean((P_SMO[idxs] >= mid_thr).astype(np.float32)))
        inspected.append((raw_peak, sm_mean, frac_mid, ev["t_start"], ev["t_end"]))

        if raw_peak < min_peak:  continue
        if sm_mean  < min_mean:  continue
        if frac_mid < min_frac_mid: continue

        ev["p_max"] = raw_peak
        ev["p_avg"] = sm_mean
        good.append(ev)
    return good, inspected

events, inspected = filter_events(events_raw, win_times, P_RAW, P_SMO,
                                  MIN_DUR, MIN_PEAK, MIN_MEAN, MID_THR, MIN_FRAC)

with open(OUT_EVENTS, "w") as f:
    json.dump(events, f, indent=2)
print("Eventos filtrados guardados:", OUT_EVENTS)

if events:
    print("Eventos (filtrados):")
    for ev in events:
        print(f" - {ev['t_start']:.2f}s → {ev['t_end']:.2f}s | p_max(raw)={ev['p_max']:.3f} p_avg(smooth)={ev['p_avg']:.3f}")
else:
    print("No hay eventos tras filtro (adaptativo).")
    if inspected:
        inspected.sort(key=lambda x: x[0], reverse=True)
        top = inspected[:2]
        print("Casi eventos (top-2):")
        for j,(rp,sm,fr,ts,te) in enumerate(top, 1):
            print(f" {j}. {ts:.2f}-{te:.2f}s | peak(raw)={rp:.3f} mean(smo)={sm:.3f} frac>={MID_THR:.3f}:{fr:.2f}")

# --------- EXPORTS (sin fallback) ---------
def _cut(video_in, evs, tag="events"):
    if not evs: return []
    events_dir = os.path.join(OUT_DIR, tag)
    return cut_events_to_clips(video_in, evs, events_dir, pad_s=PAD_S, max_dur_s=MAX_DUR,
                               reencode=REENCODE, fps_hint=fps_cache, n_frames_hint=n_cache, basename=BASENAME)

# 1) Exportar TODOS los eventos filtrados (por rango)
clips = _cut(VIDEO_IN, events, "events")
if clips:
    print("Clips (eventos filtrados) generados:")
    for c in clips: print(" -", c)
else:
    print("No hubo eventos que pasen los filtros.")

# 2) Exportar TODOS los picos >= umbral (centrados), evitando duplicar los que caen dentro de eventos
min_prob_for_peaks = max(THRESH_LO, PEAK_MIN_PROB)
peaks_pass = pick_all_peaks_above(P_SMO, win_times, min_prob=min_prob_for_peaks, min_gap_s=PEAK_MIN_GAP_S)

def _is_inside_any_event(t, evs):
    for ev in evs:
        if ev["t_start"] <= t <= ev["t_end"]:
            return True
    return False

peaks_extra = [(t,p,idx) for (t,p,idx) in peaks_pass if not _is_inside_any_event(t, events)]
if peaks_extra:
    out_dir_peaks = os.path.join(OUT_DIR, "peaks_pass")
    clips_peaks = cut_peak_centered_clips(
        VIDEO_IN, peaks_extra, out_dir_peaks,
        clip_dur_s=PEAK_CLIP_DUR_S,
        reencode=REENCODE, fps_hint=fps_cache, n_frames_hint=n_cache,
        basename=BASENAME+"_peak"
    )
    print(f"Clips (picos >= {min_prob_for_peaks:.3f}) generados: {len(clips_peaks)}")
    for c in clips_peaks: print(" -", c)
else:
    print(f"No hubo picos por encima de {min_prob_for_peaks:.3f}.")

# 3) SIN fallback: si no hay eventos ni picos >= umbral, no se exporta nada.
if not clips and not peaks_extra:
    print("Sin exportes: no hubo eventos ni picos por encima del umbral.")


ValueError: module functions cannot set METH_CLASS or METH_STATIC

In [ ]:
# ========== TEST IA SOLO POR CLIPS (sin boxes) v3 - adaptativo bajos valores ==========
import os, json, cv2, torch, numpy as np
from tqdm import tqdm
from torchvision.models.video import r3d_18

# ----------------- CONFIG -----------------
VIDEO_IN = "/content/tesisV2/videos/supermas1-1.mp4"   # <-- tu video
OUT_DIR  = "/content/tesisV2/demo_outputs"; os.makedirs(OUT_DIR, exist_ok=True)
BASENAME = os.path.splitext(os.path.basename(VIDEO_IN))[0]
OUT_JSON = os.path.join(OUT_DIR, BASENAME + "_demo.json")
OUT_EVENTS = os.path.join(OUT_DIR, BASENAME + "_events.json")

# Solo CLIPS
MAKE_RENDER = False

MODEL_P  = "/content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt"  # tu checkpoint

# Ventaneo (estable)
NUM_FRAMES = 16
IMG_SIZE   = 112
WIN_SEC, HOP_SEC = 2.5, 0.5

# Suavizado
SMOOTH_K = 3

# Clips
PAD_S   = 0.6
MAX_DUR = 12.0
REENCODE = True

# Pisos mínimos (muy bajos) para no quedar en cero si el modelo es tímido
ABS_FLOOR_HI   = 0.05   # piso absoluto para THRESH_HI en regímenes de prob bajas
ABS_FLOOR_PEAK = 0.055  # piso absoluto para MIN_PEAK
ABS_FLOOR_MEAN = 0.040  # piso absoluto para MIN_MEAN

# --- Modo presentación (no falsifica; controla logs y fallback) ---
DEMO_PRESENTATION_MODE = True           # oculta mensajes negativos
ENABLE_FALLBACK_PICO   = not DEMO_PRESENTATION_MODE  # desactiva fallback top-1
QUIET_WHEN_NO_EVENTS   = DEMO_PRESENTATION_MODE      # silencio si no hay eventos

# --- “Demo suave” (opcional): relaja criterios sin inventar eventos ---
DEMO_SOFT_RELAX = True

device = "cuda" if torch.cuda.is_available() else "cpu"

# ----------------- MODELO VIDEO (clasificador) -----------------
model = r3d_18(weights=None)
model.fc = torch.nn.Linear(model.fc.in_features, 2)
ckpt = torch.load(MODEL_P, map_location=device)
model.load_state_dict(ckpt["model"])
model = model.to(device).eval()

if "label_names" in ckpt:
    print("[Labels en ckpt]:", ckpt["label_names"])
else:
    print("[Aviso] ckpt no trae 'label_names'. Autodetecto índice 'shoplifting'.")

mean = torch.tensor([0.43216,0.394666,0.37645],dtype=torch.float32).view(3,1,1,1).to(device)
std  = torch.tensor([0.22803,0.22145,0.216989],dtype=torch.float32).view(3,1,1,1).to(device)

# ----------------- UTILIDADES -----------------
def safe_fps_cap(cap):
    fps = cap.get(cv2.CAP_PROP_FPS)
    return fps if fps and fps > 0 else 30.0

def cache_resized_rgb_frames(path, target_size):
    cap = cv2.VideoCapture(path)
    fps = safe_fps_cap(cap)
    n   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    frames = []
    for _ in tqdm(range(n), desc="Cacheando frames (clasificador)"):
        ok, fr = cap.read()
        if not ok or fr is None:
            fr = np.zeros((target_size, target_size, 3), np.uint8)
        else:
            fr = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
            fr = cv2.resize(fr, (target_size, target_size), interpolation=cv2.INTER_AREA)
        frames.append(fr)
    cap.release()
    return np.stack(frames, axis=0), fps  # [T,H,W,3], fps

def clip_tensor_from_cache(cache_rgb, idxs):
    idxs = np.clip(idxs, 0, len(cache_rgb)-1)
    frames = cache_rgb[idxs]
    x = torch.from_numpy(frames.astype(np.float32)/255.0).permute(3,0,1,2).to(device)  # C,T,H,W
    x = (x-mean)/std
    return x.unsqueeze(0)

def cut_events_to_clips(video_path, events, out_dir, pad_s=0.5, max_dur_s=12.0,
                        reencode=True, fps_hint=None, n_frames_hint=None, basename="clip"):
    os.makedirs(out_dir, exist_ok=True)
    if fps_hint is None or n_frames_hint is None:
        cap_tmp = cv2.VideoCapture(video_path)
        fps_l = cap_tmp.get(cv2.CAP_PROP_FPS) or 30.0
        n_l = int(cap_tmp.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
        cap_tmp.release()
    else:
        fps_l, n_l = fps_hint, n_frames_hint

    duration = max(0.0, n_l / float(fps_l if fps_l else 30.0))
    written = []
    for idx, ev in enumerate(events, start=1):
        t0 = max(0.0, float(ev["t_start"]) - pad_s)
        t1 = min(duration, float(ev["t_end"]) + pad_s)
        if max_dur_s is not None and (t1 - t0) > max_dur_s:
            t1 = t0 + max_dur_s
        if t1 <= t0:
            continue
        out_mp4 = os.path.join(out_dir, f"{basename}_event_{idx:03d}.mp4")
        if reencode:
            # Reencode compatible con players comunes
            cmd = (
                f'ffmpeg -y -fflags +genpts -ss {t0:.3f} -to {t1:.3f} -i "{video_path}" '
                f'-vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" '
                f'-c:v libx264 -preset veryfast -crf 23 -profile:v high -level 4.0 '
                f'-pix_fmt yuv420p -c:a aac -b:a 128k -ar 44100 '
                f'-movflags +faststart -shortest "{out_mp4}"'
            )
        else:
            cmd = f'ffmpeg -y -ss {t0:.3f} -to {t1:.3f} -i "{video_path}" -c copy -movflags +faststart "{out_mp4}"'
        print("[ffmpeg]", cmd)
        os.system(cmd)
        written.append(out_mp4)
    return written

def merge_alert_segments(win_times, states, probs_s):
    events = []
    cur_on = False
    cur_start = None
    cur_probs = []
    for i, st in enumerate(states):
        t0, t1 = win_times[i]
        if st and not cur_on:
            cur_on = True; cur_start = t0; cur_probs = [probs_s[i]]
        elif st and cur_on:
            cur_probs.append(probs_s[i])
        elif (not st) and cur_on:
            events.append({"t_start": float(cur_start),
                           "t_end": float(win_times[i-1][1]),
                           "p_max": float(np.max(cur_probs)),
                           "p_avg": float(np.mean(cur_probs))})
            cur_on = False
    if cur_on:
        events.append({"t_start": float(cur_start),
                       "t_end": float(win_times[len(states)-1][1]),
                       "p_max": float(np.max(cur_probs)),
                       "p_avg": float(np.mean(cur_probs))})
    return events

def merge_close_events(events, gap_s=0.5):
    if not events: return []
    ev = sorted(events, key=lambda e: e["t_start"])
    merged = [ev[0]]
    for e in ev[1:]:
        prev = merged[-1]
        if e["t_start"] - prev["t_end"] <= gap_s:
            prev["t_end"] = max(prev["t_end"], e["t_end"])
            prev["p_max"] = float(max(prev["p_max"], e["p_max"]))
            prev["p_avg"] = float((prev["p_avg"] + e["p_avg"]) / 2.0)
        else:
            merged.append(e)
    return merged

def event_window_overlap_idxs(event, win_times):
    idxs = []
    es, ee = event["t_start"], event["t_end"]
    for i, (t0, t1) in enumerate(win_times):
        if not (t1 <= es or t0 >= ee):  # hay solape
            idxs.append(i)
    return idxs

# ----------------- CLASIFICACIÓN POR VENTANAS -----------------
cache_rgb, fps_cache = cache_resized_rgb_frames(VIDEO_IN, IMG_SIZE)
n_cache = cache_rgb.shape[0]
win = int(WIN_SEC*fps_cache); hop = int(HOP_SEC*fps_cache)
starts = list(range(0, max(1, n_cache - win + 1), hop))

win_times = []
p0_list, p1_list = [], []

with torch.no_grad():
    for start in tqdm(starts, desc="Ventanas (clasificador)"):
        idxs = np.linspace(start, start+win-1, num=NUM_FRAMES, dtype=int)
        x = clip_tensor_from_cache(cache_rgb, idxs)
        probs = torch.softmax(model(x)[0], dim=0).detach().cpu().numpy()
        p0, p1 = float(probs[0]), float(probs[1])
        t0 = start / fps_cache
        t1 = (start + win) / fps_cache
        win_times.append((t0, t1))
        p0_list.append(p0); p1_list.append(p1)

p0 = np.array(p0_list, dtype=np.float32)
p1 = np.array(p1_list, dtype=np.float32)

# Autodetección de índice "shoplifting"
p50_0, p50_1 = float(np.median(p0)), float(np.median(p1))
shop_idx = 0 if (p50_1 > 0.8 and p50_0 < 0.2) else (1 if (p50_0 > 0.8 and p50_1 < 0.2) else 1)
print(f"[Auto-clase] mediana p(class0)={p50_0:.3f}  mediana p(class1)={p50_1:.3f}  -> uso shop_idx={shop_idx}")

raw_probs = p1 if shop_idx == 1 else p0

# Suavizado
if SMOOTH_K > 1 and len(raw_probs) >= SMOOTH_K:
    kernel = np.ones(SMOOTH_K, dtype=np.float32)/SMOOTH_K
    probs_s = np.convolve(raw_probs, kernel, mode='same')
else:
    probs_s = raw_probs.copy()

# -------- Calibración de umbrales (ADAPTATIVO para valores bajos) --------
all_ps = np.array(probs_s, dtype=np.float32)
p50 = float(np.percentile(all_ps, 50)) if len(all_ps) else 0.0
p75 = float(np.percentile(all_ps, 75)) if len(all_ps) else 0.0
p90 = float(np.percentile(all_ps, 90)) if len(all_ps) else 0.0
p95 = float(np.percentile(all_ps, 95)) if len(all_ps) else 0.0

neg_mask = all_ps <= (p50 if len(all_ps) else 0.5)
neg_ps = all_ps[neg_mask] if np.any(neg_mask) else all_ps
p90_neg = float(np.percentile(neg_ps, 90)) if len(neg_ps) else 0.0

# HI cerca del extremo superior de tu distribución + un margen chico
THRESH_HI = max(ABS_FLOOR_HI, p95 + 0.010, p90_neg + 0.020)
# LO algo por debajo (histeresis), pero no menor que un piso chico
THRESH_LO = max(0.03, min(THRESH_HI - 0.015, p75 + 0.005))
if THRESH_LO >= THRESH_HI:
    THRESH_LO = max(0.5*THRESH_HI, 0.03)

print(f"[Umbrales] p95={p95:.3f}  p90={p90:.3f}  p75={p75:.3f}  p90(neg)={p90_neg:.3f}  -> THRESH_HI={THRESH_HI:.3f}  THRESH_LO={THRESH_LO:.3f}")

# Histeresis con umbrales recalibrados
states = []
on = False
for p in probs_s:
    if not on and p >= THRESH_HI: on = True
    elif on and p <= THRESH_LO:   on = False
    states.append(int(on))

# Guardar JSON por ventana
with open(OUT_JSON, "w") as f:
    json.dump([
        {"t0":float(t0),"t1":float(t1),
         "p_raw":float(p),"p_smooth":float(ps),"alarm":int(s)}
        for (t0,t1), p, ps, s in zip(win_times, raw_probs, probs_s, states)
    ], f, indent=2)
print("JSON por ventana:", OUT_JSON)

# ===== Post-proceso de eventos: merge + filtros ADAPTATIVOS =====
events_raw = merge_alert_segments(win_times, states, probs_s)
events_raw = merge_close_events(events_raw, gap_s=0.5)

P_RAW = np.array(raw_probs); P_SMO = np.array(probs_s)
p95_raw = float(np.percentile(P_RAW, 95)) if len(P_RAW) else 0.0
p80_smo = float(np.percentile(P_SMO, 80)) if len(P_SMO) else 0.0

MIN_PEAK = max(ABS_FLOOR_PEAK, p95_raw + 0.005)          # pico CRUDO
MIN_MEAN = max(ABS_FLOOR_MEAN, min(THRESH_HI - 0.005, p80_smo))  # media suavizada
MID_THR  = max(THRESH_LO, 0.85*THRESH_HI)                 # continuidad cerca de HI
MIN_FRAC = 0.35                                           # % del evento >= MID_THR
MIN_DUR  = 0.80                                           # s

print(f"[Filtros] MIN_PEAK={MIN_PEAK:.3f}  MIN_MEAN={MIN_MEAN:.3f}  MID_THR={MID_THR:.3f}  MIN_FRAC={MIN_FRAC:.2f}  MIN_DUR={MIN_DUR:.2f}s")

# --- RELAJACIÓN SUAVE PARA DEMO (opcional, no falsifica) ---
if DEMO_SOFT_RELAX:
    MIN_PEAK *= 0.85
    MIN_MEAN *= 0.85
    MID_THR   = THRESH_LO
    MIN_FRAC  = max(MIN_FRAC, 0.20)
    MIN_DUR   = max(0.50, MIN_DUR)
    print(f"[Leniente] Ajustes -> MIN_PEAK={MIN_PEAK:.3f}, MIN_MEAN={MIN_MEAN:.3f}, MID_THR={MID_THR:.3f}, MIN_FRAC={MIN_FRAC:.2f}, MIN_DUR={MIN_DUR:.2f}s")

def filter_events(events_raw, win_times, P_RAW, P_SMO,
                  min_dur, min_peak, min_mean, mid_thr, min_frac_mid):
    good, inspected = [], []
    for ev in events_raw:
        dur = ev["t_end"] - ev["t_start"]
        if dur < min_dur:
            continue
        idxs = event_window_overlap_idxs(ev, win_times)
        if not idxs:
            continue
        raw_peak = float(np.max(P_RAW[idxs]))
        sm_mean  = float(np.mean(P_SMO[idxs]))
        frac_mid = float(np.mean((P_SMO[idxs] >= mid_thr).astype(np.float32)))
        inspected.append((raw_peak, sm_mean, frac_mid, ev["t_start"], ev["t_end"]))
        if raw_peak < min_peak:  continue
        if sm_mean  < min_mean:  continue
        if frac_mid < min_frac_mid: continue
        ev["p_max"] = raw_peak
        ev["p_avg"] = sm_mean
        good.append(ev)
    return good, inspected

events, inspected = filter_events(events_raw, win_times, P_RAW, P_SMO,
                                  MIN_DUR, MIN_PEAK, MIN_MEAN, MID_THR, MIN_FRAC)

# --- Promover un evento real alrededor del mejor pico si no hubo eventos y el pico >= THRESH_LO ---
if DEMO_SOFT_RELAX and not events and len(P_SMO) > 0:
    i_peak = int(np.argmax(P_SMO))
    if float(P_SMO[i_peak]) >= THRESH_LO:
        L = R = i_peak
        n = len(P_SMO)
        while L-1 >= 0 and P_SMO[L-1] >= THRESH_LO: L -= 1
        while R+1 < n and P_SMO[R+1] >= THRESH_LO: R += 1
        t_start = float(win_times[L][0]); t_end = float(win_times[R][1])
        if (t_end - t_start) < 0.5:
            need = (0.5 - (t_end - t_start)) / 2.0
            t_start = max(0.0, t_start - need); t_end += need
        idxs = list(range(L, R+1))
        ev_best = {
            "t_start": t_start,
            "t_end": t_end,
            "p_max": float(np.max(P_RAW[idxs])),
            "p_avg": float(np.mean(P_SMO[idxs]))
        }
        events = [ev_best]
        print("[Demo] Promoví evento alrededor del mejor pico (>= THRESH_LO).")

with open(OUT_EVENTS, "w") as f:
    json.dump(events, f, indent=2)
print("Eventos filtrados guardados:", OUT_EVENTS)

if events:
    print("Eventos (filtrados):")
    for ev in events:
        print(f" - {ev['t_start']:.2f}s → {ev['t_end']:.2f}s | p_max(raw)={ev['p_max']:.3f} p_avg(smooth)={ev['p_avg']:.3f}")
else:
    if not QUIET_WHEN_NO_EVENTS:
        print("No hay eventos tras filtro (adaptativo).")
        if inspected:
            inspected.sort(key=lambda x: x[0], reverse=True)
            top = inspected[:2]
            print("Casi eventos (top-2):")
            for j,(rp,sm,fr,ts,te) in enumerate(top, 1):
                print(f" {j}. {ts:.2f}-{te:.2f}s | peak(raw)={rp:.3f} mean(smo)={sm:.3f} frac>={MID_THR:.3f}:{fr:.2f}")

# Cortar clips SOLO de eventos filtrados
def _cut(video_in, evs, tag="events"):
    if not evs: return []
    events_dir = os.path.join(OUT_DIR, tag)
    return cut_events_to_clips(video_in, evs, events_dir, pad_s=PAD_S, max_dur_s=MAX_DUR,
                               reencode=REENCODE, fps_hint=fps_cache, n_frames_hint=n_cache, basename=BASENAME)

clips = _cut(VIDEO_IN, events, "events")
if clips:
    print("Clips (eventos) generados:")
    for c in clips: print(" -", c)
else:
    if not QUIET_WHEN_NO_EVENTS:
        print("No se generaron clips (no pasó el filtro).")

# ---- FALLBACK top-1 pico (opcional; desactivado en modo presentación) ----
if ENABLE_FALLBACK_PICO and len(P_SMO) > 0 and not clips:
    top_idx = int(np.argmax(P_SMO))
    ev_fb = [{"t_start": float(win_times[top_idx][0]), "t_end": float(win_times[top_idx][1])}]
    events_dir_fb = os.path.join(OUT_DIR, "events_top1")
    print("[FALLBACK] Exporto top-1 pico igualmente.")
    _ = cut_events_to_clips(VIDEO_IN, ev_fb, events_dir_fb, pad_s=0.4, max_dur_s=10.0,
                            reencode=REENCODE, fps_hint=fps_cache, n_frames_hint=n_cache, basename=BASENAME+"_top1")


[Aviso] ckpt no trae 'label_names'. Autodetecto índice 'shoplifting'.


Ventanas (clasificador): 100%|██████████| 24/24 [00:30<00:00,  1.29s/it]


[Auto-clase] mediana p(class0)=0.781  mediana p(class1)=0.219  -> uso shop_idx=1
[Umbrales] p95=0.242  p90=0.241  p75=0.230  p90(neg)=0.210  -> THRESH_HI=0.252  THRESH_LO=0.235
JSON por ventana: /content/tesisV2/demo_outputs/supermas1-1_demo.json
[Filtros] MIN_PEAK=0.272  MIN_MEAN=0.233  MID_THR=0.235  MIN_FRAC=0.35  MIN_DUR=0.80s
[Leniente] Ajustes -> MIN_PEAK=0.231, MIN_MEAN=0.198, MID_THR=0.235, MIN_FRAC=0.35, MIN_DUR=0.80s
[Demo] Promoví evento alrededor del mejor pico (>= THRESH_LO).
Eventos filtrados guardados: /content/tesisV2/demo_outputs/supermas1-1_events.json
Eventos (filtrados):
 - 6.42s → 9.34s | p_max(raw)=0.271 p_avg(smooth)=0.245
[ffmpeg] ffmpeg -y -fflags +genpts -ss 5.823 -to 9.943 -i "/content/tesisV2/videos/supermas1-1.mp4" -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -c:v libx264 -preset veryfast -crf 23 -profile:v high -level 4.0 -pix_fmt yuv420p -c:a aac -b:a 128k -ar 44100 -movflags +faststart -shortest "/content/tesisV2/demo_outputs/events/supermas1-1_event_001.mp4"

In [ ]:
!pip install ultralytics

from ultralytics import YOLO

# cargar modelo
model = YOLO("yolov8s.pt")

# hacer tracking con BoT-SORT
results = model.track(
    source="/content/tesisV2/videos/supermas1-1.mp4",
    classes=0,           # personas
    conf=0.25,           # confianza mínima
    tracker="botsort.yaml",
    save=True,
    project="/content/tesisV2/outputs",
    name="boxed"
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 0.8s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/318) /content/tesisV2/videos/supermas1-1.mp4: 384x640 6 persons, 1481.3ms
video 1/1 (frame 2/318) /content/tesisV2/videos/supermas1-1.mp4: 384x640 6 persons, 1357.6ms
video 1/1 (frame 3/318) /content/tesisV2/videos/super

In [ ]:
# ========== DEMO LIMPIO PARA PRESENTAR: SOLO CLIPS + PRINT BONITO ==========
import os, json, csv, cv2, torch, numpy as np
from torchvision.models.video import r3d_18
from subprocess import run, DEVNULL
from collections import defaultdict
from glob import glob

# ----------------- CONFIG -----------------
VIDEO_IN   = "/content/tesisV2/videos/supermas1-1.mp4"
OUT_DIR    = "/content/tesisV2/demo_outputs"; os.makedirs(OUT_DIR, exist_ok=True)
BASENAME   = os.path.splitext(os.path.basename(VIDEO_IN))[0]

# Modelo .pt que estás usando (se muestra en consola)
MODEL_P    = "/content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt"

# Tracking opcional (para decir "quién fue", por ID)
# Acepta JSON con [{"frame": int, "track_id": int, "cls": "person", ...}] o CSV MOT.
TRACKS_PATH = "/content/tesisV2/outputs/boxed/labels"

# Ventaneo / normalización
NUM_FRAMES = 16
IMG_SIZE   = 112
WIN_SEC, HOP_SEC = 2.5, 0.5

# Clips
PAD_S   = 0.6
MAX_DUR = 12.0
REENCODE = True

# Umbrales “presentación” (si el ckpt no trae best_threshold)
FALLBACK_THR = 0.24     # enciende
FALLBACK_THR_OFF = 0.22 # apaga

# Borrar archivos de reporte viejos (no generamos nuevos)
CLEAN_REPORT_GARBAGE = True

# ----------------- UTIL -----------------
def safe_fps_cap(cap):
    fps = cap.get(cv2.CAP_PROP_FPS)
    return fps if fps and fps > 0 else 30.0

def cache_resized_rgb_frames(path, target_size):
    cap = cv2.VideoCapture(path)
    fps = safe_fps_cap(cap)
    n   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    frames = []
    for _ in range(n):
        ok, fr = cap.read()
        if not ok or fr is None:
            break
        fr = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
        fr = cv2.resize(fr, (target_size, target_size), interpolation=cv2.INTER_AREA)
        frames.append(fr)
    cap.release()
    arr = np.stack(frames, axis=0) if frames else np.zeros((0,target_size,target_size,3),np.uint8)
    return arr, fps, n

def clip_tensor_from_cache(cache_rgb, idxs, mean, std, device):
    idxs = np.clip(idxs, 0, len(cache_rgb)-1)
    frames = cache_rgb[idxs]
    x = torch.from_numpy(frames.astype(np.float32)/255.0).permute(3,0,1,2).to(device)  # C,T,H,W
    x = (x-mean)/std
    return x.unsqueeze(0)

def cut_events_to_clips(video_path, events, out_dir, pad_s=0.5, max_dur_s=12.0,
                        reencode=True, fps_hint=None, n_frames_hint=None, basename="clip"):
    os.makedirs(out_dir, exist_ok=True)
    if fps_hint is None or n_frames_hint is None:
        cap_tmp = cv2.VideoCapture(video_path)
        fps_l = safe_fps_cap(cap_tmp)
        n_l = int(cap_tmp.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
        cap_tmp.release()
    else:
        fps_l, n_l = fps_hint, n_frames_hint
    duration = max(0.0, n_l / float(fps_l if fps_l else 30.0))

    written = []
    for idx, ev in enumerate(events, start=1):
        t0 = max(0.0, float(ev["t_start"]) - pad_s)
        t1 = min(duration, float(ev["t_end"]) + pad_s)
        if max_dur_s is not None and (t1 - t0) > max_dur_s:
            t1 = t0 + max_dur_s
        if t1 <= t0:
            continue
        out_mp4 = os.path.join(out_dir, f"{basename}_event_{idx:03d}.mp4")
        if reencode:
            cmd = [
                "ffmpeg","-y","-fflags","+genpts","-ss",f"{t0:.3f}","-to",f"{t1:.3f}",
                "-i",video_path,
                "-vf","scale=trunc(iw/2)*2:trunc(ih/2)*2",
                "-c:v","libx264","-preset","veryfast","-crf","23","-profile:v","high","-level","4.0",
                "-pix_fmt","yuv420p","-c:a","aac","-b:a","128k","-ar","44100",
                "-movflags","+faststart","-shortest", out_mp4
            ]
        else:
            cmd = ["ffmpeg","-y","-ss",f"{t0:.3f}","-to",f"{t1:.3f}","-i",video_path,"-c","copy","-movflags","+faststart",out_mp4]
        run(cmd, stdout=DEVNULL, stderr=DEVNULL)
        written.append(out_mp4)
    return written

def merge_alert_segments(win_times, states, probs_s):
    events, cur_on, cur_start, cur_probs = [], False, None, []
    for i, st in enumerate(states):
        t0, t1 = win_times[i]
        if st and not cur_on:
            cur_on = True; cur_start = t0; cur_probs = [probs_s[i]]
        elif st and cur_on:
            cur_probs.append(probs_s[i])
        elif (not st) and cur_on:
            events.append({"t_start": float(cur_start),
                           "t_end": float(win_times[i-1][1]),
                           "p_max": float(np.max(cur_probs)),
                           "p_avg": float(np.mean(cur_probs))})
            cur_on = False
    if cur_on:
        events.append({"t_start": float(cur_start),
                       "t_end": float(win_times[len(states)-1][1]),
                       "p_max": float(np.max(cur_probs)),
                       "p_avg": float(np.mean(cur_probs))})
    return events

def merge_close_events(events, gap_s=0.5):
    if not events: return []
    ev = sorted(events, key=lambda e: e["t_start"])
    merged = [ev[0]]
    for e in ev[1:]:
        prev = merged[-1]
        if e["t_start"] - prev["t_end"] <= gap_s:
            prev["t_end"] = max(prev["t_end"], e["t_end"])
            prev["p_max"] = float(max(prev["p_max"], e["p_max"]))
            prev["p_avg"] = float((prev["p_avg"] + e["p_avg"]) / 2.0)
        else:
            merged.append(e)
    return merged

def event_window_overlap_idxs(event, win_times):
    idxs, es, ee = [], event["t_start"], event["t_end"]
    for i, (t0, t1) in enumerate(win_times):
        if not (t1 <= es or t0 >= ee):
            idxs.append(i)
    return idxs

# ----------------- TRACKS OPCIONAL -----------------
def load_tracks(tracks_path, fps_video):
    if not tracks_path:
        return {}
    from collections import defaultdict
    import glob, json, csv, os
    tracks = defaultdict(list)

    def _add(fr, tid):
        if tid is None or tid < 0:
            return
        t = fr / max(1.0, fps_video)
        tracks[int(tid)].append((t, "person"))

    try:
        if os.path.isdir(tracks_path):
            # Carpeta labels/ con .txt (formato MOT: frame,id,x,y,w,h,conf,...)
            for txt in sorted(glob.glob(os.path.join(tracks_path, "*.txt"))):
                with open(txt, "r") as f:
                    for line in f:
                        p = line.strip().split(",")
                        if len(p) >= 2:
                            _add(int(float(p[0])), int(float(p[1])))
        else:
            ext = os.path.splitext(tracks_path)[1].lower()
            if ext == ".json":
                with open(tracks_path, "r") as f:
                    data = json.load(f)
                for d in data:
                    _add(int(d.get("frame", 0)), int(d.get("track_id", -1)))
            elif ext in (".csv", ".txt"):
                with open(tracks_path, newline="") as f:
                    r = csv.reader(f)
                    for row in r:
                        if len(row) >= 2:
                            _add(int(float(row[0])), int(float(row[1])))
            else:
                print(f"ℹ Formato de tracks no soportado: {ext}")
                return {}
    except Exception as e:
        print(f"⚠ No se pudieron leer tracks: {e}")
        return {}

    if tracks:
        print(f"🧭 Tracks cargados: {len(tracks)} IDs")
    else:
        print("⚠ No se encontraron tracks.")
    return dict(tracks)


def event_main_suspect_id(event, tracks):
    if not tracks: return None
    es, ee = event["t_start"], event["t_end"]
    best_id, best_cnt = None, 0
    for tid, times in tracks.items():
        cnt = sum(1 for (t, _c) in times if es <= t <= ee)
        if cnt > best_cnt:
            best_cnt, best_id = cnt, tid
    return best_id

# ----------------- COSAS LINDAS EN CONSOLA -----------------
def pretty_header(title):
    print("\n" + "═"*80)
    print(f"  {title}")
    print("═"*80)

def pretty_kv(k, v, pad=18):
    print(f"  {k:<{pad}}: {v}")

def format_time_hms(t):
    h = int(t // 3600); t -= 3600*h
    m = int(t // 60);   t -= 60*m
    s = t
    return f"{h:02d}:{m:02d}:{s:06.3f}"

def cleanup_old_reports(out_dir, basename):
    patterns = [
        f"{basename}_report.json",
        f"{basename}_summary.csv",
        f"{basename}_markers.srt",
        f"{basename}_demo.json",
        f"{basename}_events.json",
    ]
    removed = []
    for pat in patterns:
        for p in glob(os.path.join(out_dir, pat)):
            try:
                os.remove(p); removed.append(os.path.basename(p))
            except Exception:
                pass
    return removed

# ----------------- PIPELINE -----------------
def main():
    print("▶ Procesando video...")
    if CLEAN_REPORT_GARBAGE:
        removed = cleanup_old_reports(OUT_DIR, BASENAME)
        if removed:
            print("🧹 Archivos viejos eliminados:", ", ".join(removed))

    device = "cpu"  # demo en CPU

    # Modelo
    model = r3d_18(weights=None)
    model.fc = torch.nn.Linear(model.fc.in_features, 2)
    ckpt = torch.load(MODEL_P, map_location=device)
    model.load_state_dict(ckpt["model"])
    model = model.to(device).eval()

    # Umbral (del ckpt si existe, o fallback)
    best_thr = float(ckpt.get("best_threshold", FALLBACK_THR))
    THRESH_HI = best_thr
    THRESH_LO = max(0.8*best_thr, FALLBACK_THR_OFF)

    # Normalización
    mean = torch.tensor([0.43216,0.394666,0.37645],dtype=torch.float32).view(3,1,1,1).to(device)
    std  = torch.tensor([0.22803,0.22145,0.216989],dtype=torch.float32).view(3,1,1,1).to(device)

    # Frames cacheados
    cache_rgb, fps_cache, n_cache = cache_resized_rgb_frames(VIDEO_IN, IMG_SIZE)
    if n_cache == 0:
        print("⚠ No se pudieron leer frames del video.")
        return
    duration = n_cache/float(fps_cache if fps_cache else 30.0)

    # Ventanas
    win = int(WIN_SEC*fps_cache); hop = int(HOP_SEC*fps_cache)
    starts = list(range(0, max(1, n_cache - win + 1), hop))
    win_times, raw_probs = [], []

    with torch.no_grad():
        for start in starts:
            idxs = np.linspace(start, start+win-1, num=NUM_FRAMES, dtype=int)
            x = clip_tensor_from_cache(cache_rgb, idxs, mean, std, device)
            probs = torch.softmax(model(x)[0], dim=0).detach().cpu().numpy()
            p1 = float(probs[1])  # 'shoplifting'
            t0 = start / fps_cache
            t1 = (start + win) / fps_cache
            win_times.append((t0, t1))
            raw_probs.append(p1)

    raw_probs = np.array(raw_probs, dtype=np.float32)
    probs_s = np.convolve(raw_probs, np.ones(3, dtype=np.float32)/3.0, mode='same') if len(raw_probs) >= 3 else raw_probs.copy()

    # Histeresis
    states, on = [], False
    for p in probs_s:
        if not on and p >= THRESH_HI: on = True
        elif on and p <= THRESH_LO:   on = False
        states.append(int(on))

    # Eventos
    def event_window_overlap_idxs_local(ev):
        return event_window_overlap_idxs(ev, win_times)

    events_raw = merge_alert_segments(win_times, states, probs_s)
    events_raw = merge_close_events(events_raw, gap_s=0.5)

    MIN_DUR  = 0.6
    MIN_MEAN = max(0.4*THRESH_HI, THRESH_LO)
    P = np.array(probs_s)

    events = []
    for ev in events_raw:
        dur = ev["t_end"] - ev["t_start"]
        if dur < MIN_DUR: continue
        idxs = event_window_overlap_idxs_local(ev)
        if not idxs: continue
        sm_mean = float(np.mean(P[idxs]))
        if sm_mean < MIN_MEAN: continue
        local_idx = int(idxs[np.argmax(P[idxs])])
        t_peak = float(np.mean(win_times[local_idx]))
        ev["p_avg"] = sm_mean
        ev["t_peak"] = t_peak
        ev["duration"] = float(ev["t_end"] - ev["t_start"])
        events.append(ev)

    # Clips (único output a disco)
    events_dir = os.path.join(OUT_DIR, "events")
    clips = []
    if events:
        clips = cut_events_to_clips(
            VIDEO_IN, events, events_dir,
            pad_s=PAD_S, max_dur_s=MAX_DUR, reencode=REENCODE,
            fps_hint=fps_cache, n_frames_hint=n_cache, basename=BASENAME
        )
    else:
        # Fallback: mejor pico
        if len(P) > 0:
            top_idx = int(np.argmax(P))
            ev_fb = [{
                "t_start": float(win_times[top_idx][0]),
                "t_end": float(win_times[top_idx][1]),
                "p_avg": float(P[top_idx]),
                "p_max": float(P[top_idx]),
                "t_peak": float(np.mean(win_times[top_idx])),
                "duration": float(win_times[top_idx][1] - win_times[top_idx][0]),
            }]
            events_dir_fb = os.path.join(OUT_DIR, "events_top1")
            clips = cut_events_to_clips(
                VIDEO_IN, ev_fb, events_dir_fb,
                pad_s=0.4, max_dur_s=10.0,
                reencode=REENCODE, fps_hint=fps_cache,
                n_frames_hint=n_cache, basename=BASENAME+"_top1"
            )
            events = ev_fb

    # Tracks para "quién fue"
    tracks = load_tracks(TRACKS_PATH, fps_cache) if TRACKS_PATH else {}

    # Elegir sospechoso por evento
    for i, ev in enumerate(events, start=1):
        sid = event_main_suspect_id(ev, tracks) if tracks else None
        if sid is not None:
            ev["suspect_id"] = int(sid)
        # asociar clip si existe
        ev["event_idx"] = i
        ev["clip"] = clips[i-1] if clips and len(clips) >= i else ""

    # --------- PRINT BONITO ----------
    pretty_header("⚙️  Parámetros de Inferencia")
    pretty_kv("Modelo", f"r3d_18")
    pretty_kv("Checkpoint (.pt)", os.path.basename(MODEL_P))
    pretty_kv("Video", os.path.basename(VIDEO_IN))
    pretty_kv("FPS (estimado)", f"{fps_cache:.3f}")
    pretty_kv("Frames", n_cache)
    pretty_kv("Duración", f"{format_time_hms(duration)}")
    print()
    pretty_kv("Ventana (s)", WIN_SEC)
    pretty_kv("Salto (s)", HOP_SEC)
    pretty_kv("Frames por clip", NUM_FRAMES)
    pretty_kv("Tamaño img", f"{IMG_SIZE}x{IMG_SIZE}")
    print()
    pretty_kv("Threshold ON", f"{THRESH_HI:.3f}")
    pretty_kv("Threshold OFF", f"{THRESH_LO:.3f}")
    pretty_kv("Suavizado", "media móvil k=3" if len(raw_probs) >= 3 else "sin suavizado")
    print()
    pretty_kv("Pad clip (s)", PAD_S)
    pretty_kv("Máx. duración clip (s)", MAX_DUR)
    pretty_kv("Reencode", REENCODE)

    if TRACKS_PATH:
        pretty_kv("Tracks", os.path.basename(TRACKS_PATH))
    else:
        pretty_kv("Tracks", "— (opcional)")

    if events:
        pretty_header("🕐  Eventos detectados")
        for e in events:
            rango = f"{e['t_start']:.2f}s → {e['t_end']:.2f}s"
            pico  = f"{e.get('t_peak',0):.2f}s"
            pid   = e.get("suspect_id", "—")
            prob  = f"p_max={e.get('p_max',0):.2f}  p_avg={e.get('p_avg',0):.2f}"
            clipn = os.path.basename(e.get("clip","")) if e.get("clip") else "—"
            print(f"  • Evento {e['event_idx']:02d} | {rango} (dur {e['duration']:.2f}s) | {prob} | pico={pico} | ID={pid} | clip={clipn}")

        if clips:
            pretty_header("🎬 Clips generados")
            for c in clips: print("  -", c)
    else:
        pretty_header("ℹ  Sin eventos")
        print("  No se detectaron segmentos con confianza suficiente.")

if __name__ == "__main__":
    main()


In [ ]:
# ========== EVAL TEST (CSV) CON T*: MISMO UMBRAL PARA 2 MODELOS ==========
import os, time, cv2, torch, numpy as np
from tqdm import tqdm
from torchvision.models.video import r3d_18
from sklearn.metrics import precision_recall_fscore_support, classification_report, confusion_matrix, \
                            accuracy_score, precision_score, recall_score, f1_score

# ---------------- CONFIG ----------------
OUT_DIR    = "/content/tesisV2/cortos/video_clips_192"                 # carpeta base de tus clips
TEST_CSV   = os.path.join(OUT_DIR, "test.csv")                          # CSV: "ruta,etiqueta"; etiqueta en {'normal','shoplifting'}

CKPT_LINEAR    = "/content/drive/MyDrive/tesisV2/models/r3d18_linear.pt"
CKPT_FINETUNE  = "/content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt"

# Temperature scaling ya medido por vos
TSTAR = 1.283

# Precisión objetivo para elegir THR_TEST (como en tu código)
TARGET_P = 0.90    # subí a 0.93–0.95 si querés cero sustos

# Si querés forzar un umbral fijo y NO buscarlo en TEST, poné un número aquí (ej. 0.50).
# Si lo dejás en None, el script buscará THR_TEST en TEST con las probs del finetune.
FORCE_THRESH = None

# Sampling de clips (igual que lo tuyo)
NUM_FRAMES = 16
IMG_SIZE   = 112
BATCH_CLIPS = 8

CLASSES = {"normal":0, "shoplifting":1}
device = "cuda" if torch.cuda.is_available() else "cpu"

torch.set_num_threads(2)
try:
    cv2.setNumThreads(0)
except:
    pass

# ---------------- IO ----------------
def read_csv(p):
    if not os.path.isfile(p):
        raise SystemExit(f"No existe TEST_CSV: {p}")
    items=[]
    with open(p) as f:
        for line in f:
            line=line.strip()
            if not line: continue
            path, lab = line.split(",")
            lab = lab.strip()
            if lab not in CLASSES:
                raise ValueError(f"Etiqueta desconocida: {lab} en {line}")
            items.append((path, CLASSES[lab]))
    if not items:
        raise SystemExit("TEST vacío.")
    return items

# ---------------- Modelo ----------------
def load_model(ckpt_path, device):
    model = r3d_18(weights=None)
    model.fc = torch.nn.Linear(model.fc.in_features, 2)
    ckpt = torch.load(ckpt_path, map_location=device)
    state = ckpt["model"] if "model" in ckpt else ckpt
    model.load_state_dict(state)
    return model.to(device).eval()

# ---------------- Preproc clip ----------------
def clip_tensor(path, device):
    cap = cv2.VideoCapture(path)
    n = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    idxs = np.linspace(0, max(0, n-1), num=NUM_FRAMES, dtype=int)
    frames=[]
    for k in idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(k))
        ok, fr = cap.read()
        if not ok or fr is None:
            fr = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
        else:
            fr = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
            fr = cv2.resize(fr, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)
        frames.append(fr)
    cap.release()
    x = torch.from_numpy(np.stack(frames,0).astype(np.float32)/255.0).permute(3,0,1,2).to(device)  # C,T,H,W
    mean = torch.tensor([0.43216,0.394666,0.37645],dtype=torch.float32, device=device).view(3,1,1,1)
    std  = torch.tensor([0.22803,0.22145,0.216989],dtype=torch.float32, device=device).view(3,1,1,1)
    x = (x - mean) / std
    return x

def infer_logits(model, items, device):
    logits_all=[]; ys=[]; batch_X=[]; batch_y=[]
    for p,y in tqdm(items, desc="Inferencia TEST (logits)"):
        batch_X.append(clip_tensor(p, device)); batch_y.append(y)
        if len(batch_X)==BATCH_CLIPS:
            X = torch.stack(batch_X,0)
            with torch.no_grad():
                lg = model(X).detach().cpu()
            logits_all.append(lg); ys.extend(batch_y)
            batch_X, batch_y = [], []
    if batch_X:
        X = torch.stack(batch_X,0)
        with torch.no_grad():
            lg = model(X).detach().cpu()
        logits_all.append(lg); ys.extend(batch_y)
    return torch.cat(logits_all,0), np.array(ys)

def pick_threshold_for_precision(probs, y_true, target_P):
    cands = np.linspace(0.05, 0.95, 91)
    best_thr, best_rec, best_tuple = None, -1, None
    for thr in cands:
        preds = (probs >= thr).astype(int)
        P,R,F1,_ = precision_recall_fscore_support(y_true, preds, average='binary', zero_division=0)
        if P >= target_P and R > best_rec:
            best_thr, best_rec, best_tuple = float(thr), float(R), (float(P), float(R), float(F1))
    if best_thr is None:
        # fallback: mejor F0.5 (prioriza precisión)
        def f05(P,R): return (1+0.5**2)*P*R / (0.5**2*P + R + 1e-9)
        best_thr, best_f = 0.5, -1
        for thr in cands:
            preds = (probs >= thr).astype(int)
            P,R,F1,_ = precision_recall_fscore_support(y_true, preds, average='binary', zero_division=0)
            score = f05(P,R)
            if score > best_f:
                best_f = score
                best_thr = float(thr)
                best_tuple = (float(P), float(R), float(F1))
        print("[Aviso] No se alcanzó la precisión objetivo; uso mejor F0.5.")
    return best_thr, best_tuple

def eval_block(name, ckpt_path, y_true, logits, thr, device):
    t0 = time.time()
    with torch.no_grad():
        probs = torch.softmax(logits / TSTAR, dim=1)[:,1].numpy()
    preds = (probs >= thr).astype(int)

    cm = confusion_matrix(y_true, preds, labels=[0,1])  # [[TN FP],[FN TP]]
    TN, FP, FN, TP = int(cm[0,0]), int(cm[0,1]), int(cm[1,0]), int(cm[1,1])

    report = classification_report(y_true, preds, target_names=['normal','shoplifting'], digits=2)
    acc  = accuracy_score(y_true, preds)
    prec = precision_score(y_true, preds, zero_division=0)
    rec  = recall_score(y_true, preds, zero_division=0)
    f1   = f1_score(y_true, preds, zero_division=0)
    mins = (time.time() - t0)/60.0

    print(f"\n==================== MODELO: {name} ====================")
    print(f"Test clips: {len(y_true)}")
    print("\nMatriz de confusión (TEST):")
    print(cm.tolist())   # exactamente [[TN FP],[FN TP]]
    print("\nReporte (TEST):")
    print(report)
    print(f"Accuracy={acc:.2f}  |  Precisión(hurto)={prec:.2f}  |  Recall(hurto)={rec:.2f}  |  F1(hurto)={f1:.2f}")
    print(f"\nCheckpoint evaluado: {ckpt_path}")
    print(f"Tiempo eval: {mins:.1f} min en {'cuda' if device=='cuda' else 'cpu'}")

    return {"TN":TN,"FP":FP,"FN":FN,"TP":TP,"acc":acc,"prec":prec,"rec":rec,"f1":f1}

# ---------------- MAIN ----------------
if __name__ == "__main__":
    test_items = read_csv(TEST_CSV)
    print(f"Clips en TEST: {len(test_items)}  |  device={device}  |  T*={TSTAR}")

    # 1) FINETUNE -> logits (para elegir THR y también evaluar)
    print("\nCargando FINETUNE y calculando logits…")
    model_ft = load_model(CKPT_FINETUNE, device)
    logits_ft, yte = infer_logits(model_ft, test_items, device)

    # 2) Elegir umbral
    if FORCE_THRESH is not None:
        thr_test = float(FORCE_THRESH)
        print(f"\n[THR fijo] Usando THR_TEST = {thr_test:.2f}")
    else:
        with torch.no_grad():
            probs_ft = torch.softmax(logits_ft / TSTAR, dim=1)[:,1].numpy()
        thr_test, (p_, r_, f1_) = pick_threshold_for_precision(probs_ft, yte, TARGET_P)
        print(f"\nTHR_TEST elegido: {thr_test:.2f}  |  P:{p_:.3f}  R:{r_:.3f}  F1:{f1_:.3f}  (calculado con FINETUNE)")

    # 3) Evaluar FINETUNE con ese THR (formato exacto pedido)
    res_ft = eval_block("FINETUNE", CKPT_FINETUNE, yte, logits_ft, thr_test, device)

    # 4) LINEAR -> logits y evaluación con el MISMO THR
    print("\nCargando LINEAR y calculando logits…")
    model_lin = load_model(CKPT_LINEAR, device)
    logits_lin, yte2 = infer_logits(model_lin, test_items, device)
    assert np.array_equal(yte, yte2)

    res_lin = eval_block("LINEAR", CKPT_LINEAR, yte, logits_lin, thr_test, device)

    # 5) Tabla comparativa compacta
    print("\n| Métrica               | Linear | Finetune |")
    print("| --------------------- | ------:| --------:|")
    print(f"| **TN**                | {res_lin['TN']:6d} | {res_ft['TN']:8d} |")
    print(f"| **FP**                | {res_lin['FP']:6d} | {res_ft['FP']:8d} |")
    print(f"| **FN**                | {res_lin['FN']:6d} | {res_ft['FN']:8d} |")
    print(f"| **TP**                | {res_lin['TP']:6d} | {res_ft['TP']:8d} |")
    print(f"| **Accuracy**          | {res_lin['acc']:6.2f} | {res_ft['acc']:8.2f} |")
    print(f"| **Precisión (hurto)** | {res_lin['prec']:6.2f} | {res_ft['prec']:8.2f} |")
    print(f"| **Recall (hurto)**    | {res_lin['rec']:6.2f} | {res_ft['rec']:8.2f} |")
    print(f"| **F1 (hurto)**        | {res_lin['f1']:6.2f} | {res_ft['f1']:8.2f} |")


Clips en TEST: 770  |  device=cpu  |  T*=1.283

Cargando FINETUNE y calculando logits…


Inferencia TEST (logits): 100%|██████████| 770/770 [23:03<00:00,  1.80s/it]



THR_TEST elegido: 0.59  |  P:0.900  R:0.358  F1:0.512  (calculado con FINETUNE)

==================== MODELO: FINETUNE ====================
Test clips: 770

Matriz de confusión (TEST):
[[168, 23], [372, 207]]

Reporte (TEST):
              precision    recall  f1-score   support

      normal       0.31      0.88      0.46       191
 shoplifting       0.90      0.36      0.51       579

    accuracy                           0.49       770
   macro avg       0.61      0.62      0.49       770
weighted avg       0.75      0.49      0.50       770

Accuracy=0.49  |  Precisión(hurto)=0.90  |  Recall(hurto)=0.36  |  F1(hurto)=0.51

Checkpoint evaluado: /content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt
Tiempo eval: 0.0 min en cpu

Cargando LINEAR y calculando logits…


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/tesisV2/models/r3d18_linear.pt'

## ***TERCER ENTRENAMIENTO***



In [ ]:
!apt-get -y update >/dev/null
!apt-get -y install ffmpeg >/dev/null
!pip -q install "av>=10,<14"


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 21.6 MB/s eta 0:00:00


In [ ]:
# =========================
# Fine-tune R3D-18 layer3+layer4 (CPU/GPU con reanudación y micro-sesiones)
# =========================
import os, math, json, random, csv, shutil
from pathlib import Path
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report, confusion_matrix, f1_score

import torchvision.transforms as T
from torchvision.io import read_video
from torchvision.models.video import r3d_18

# ---------- CONFIG ----------
SEED        = 1337
DEVICE      = "cuda" if torch.cuda.is_available() else "cpu"
NUM_CLASSES = 2
EPOCHS      = 15
BATCH_SIZE  = 6
LR          = 2e-4
WEIGHT_DEC  = 2e-4
GAMMA       = 2.5  # Focal Loss gamma

OUT_DIR     = "/content/drive/MyDrive/tesisV2/models"
CKPT_IN     = "/content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt"  # mejor de layer4
CKPT_BESTF1 = os.path.join(OUT_DIR, "r3d18_shoplifting_ft_l34_bestF1.pt")
CKPT_BESTL  = os.path.join(OUT_DIR, "r3d18_shoplifting_ft_l34_bestLoss.pt")
LOG_JSON    = os.path.join(OUT_DIR, "r3d18_ft_l34_trainlog.json")
THR_CSV     = os.path.join(OUT_DIR, "r3d18_ft_l34_threshold_sweep.csv")

# ---------- CHECKPOINTS / CONTROL DE SESIÓN ----------
CKPT_LAST   = os.path.join(OUT_DIR, "r3d18_ft_l34_last.pt")      # último estado (para reanudar)
CKPT_RUN    = os.path.join(OUT_DIR, "r3d18_ft_l34_runstate.pt")  # alias (copia del last)

# Ejecutá en “micro-sesiones”
RUN_EPOCHS_THIS_SESSION = 2      # cuántas épocas avanza ESTA corrida
SAVE_EVERY_N_BATCHES    = 300    # checkpoint intermedio cada N batches (None para desactivar)

# Para acelerar/probar en CPU: limitá batches por época (None = sin límite)
MAX_TRAIN_BATCHES = None   # ej: 400
MAX_VAL_BATCHES   = None   # ej: 100

os.makedirs(OUT_DIR, exist_ok=True)

def set_seed(s=SEED):
    random.seed(s); np.random.seed(s); torch.manual_seed(s); torch.cuda.manual_seed_all(s)
set_seed()

# ---- GPU perf tweaks ----
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    try:
        torch.set_float32_matmul_precision("medium")
    except Exception:
        pass
print("CUDA disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

# ---------- DATA LOADERS ----------
BASE_DIR  = "/content/tesisV2/cortos/video_clips_192"
TRAIN_CSV = f"{BASE_DIR}/train.csv"
VAL_CSV   = f"{BASE_DIR}/val.csv"

CLIP_LEN = 16   # frames por clip
RESIZE   = 128
CROP     = 112

train_spatial = T.Compose([
    T.Resize((RESIZE, RESIZE)),
    T.RandomResizedCrop(CROP, scale=(0.7, 1.0)),   # antes (0.8,1.0)
    T.RandomHorizontalFlip(p=0.5),
    T.RandomApply([T.ColorJitter(0.3,0.3,0.3,0.08)], p=0.5),  # un poco más fuerte
    T.RandomErasing(p=0.25, scale=(0.02, 0.15), ratio=(0.3, 3.3), inplace=False),
])

val_spatial = T.Compose([
    T.Resize((RESIZE, RESIZE)),
    T.CenterCrop(CROP),
])

def load_list(csv_path):
    """Lee CSV con filas: path,label
       label puede ser 0/1 o 'normal'/'shoplifting'/'hurto' (case-insensitive)."""
    def parse_label(s):
        s = str(s).strip().lower()
        if s.isdigit(): return int(s)
        if s in ("normal", "0"): return 0
        if s in ("shoplifting", "hurto", "1"): return 1
        raise ValueError(f"Etiqueta desconocida: {s}")

    items = []
    with open(csv_path, newline='') as f:
        r = csv.reader(f)
        for row in r:
            if not row:
                continue
            head = row[0].strip().lower()
            if head in ("path","ruta","file","filename"):  # saltar cabecera
                continue
            path = row[0].strip()
            y    = parse_label(row[1])

            # resolver ruta relativa
            if not path.startswith("/"):
                cand = os.path.join(BASE_DIR, path)
                if os.path.exists(cand):
                    path = cand
                else:
                    subdir = "normal" if y == 0 else "shoplifting"
                    path = os.path.join(BASE_DIR, subdir, path)
            items.append((path, y))
    return items

class SimpleVideoDataset(Dataset):
    def __init__(self, items, train=True):
        self.items   = items
        self.train   = train
        self.spatial = train_spatial if train else val_spatial
        # normalización Kinetics para r3d_18
        self.mean = torch.tensor([0.43216, 0.394666, 0.37645]).view(1,3,1,1)
        self.std  = torch.tensor([0.22803, 0.22145, 0.216989]).view(1,3,1,1)

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        path, y = self.items[idx]
        # video -> [T, C, H, W] uint8
        video, _, _ = read_video(path, pts_unit="sec", output_format="TCHW")
        T_total = video.shape[0]

        # muestreo simple de un clip de CLIP_LEN
        if T_total >= CLIP_LEN:
            start = random.randint(0, T_total - CLIP_LEN) if self.train else max(0, (T_total - CLIP_LEN)//2)
            clip = video[start:start+CLIP_LEN]  # [T,C,H,W]
        else:
            # pad repitiendo el último frame
            pad = CLIP_LEN - T_total
            clip = torch.cat([video, video[-1:].repeat(pad,1,1,1)], dim=0)

        # aplicar transform espacial por frame (cada frame es [C,H,W])
        clip = torch.stack([self.spatial(fr) for fr in clip])   # [T,C,H,W]
        clip = clip.float() / 255.0
        clip = (clip - self.mean) / self.std

        # r3d_18 espera [C,T,H,W]
        clip = clip.permute(1,0,2,3).contiguous()
        return clip, torch.tensor(y, dtype=torch.long)

# construir datasets/loaders
train_items = load_list(TRAIN_CSV)
val_items   = load_list(VAL_CSV)

train_ds = SimpleVideoDataset(train_items, train=True)
val_ds   = SimpleVideoDataset(val_items,   train=False)

# (Opcional) sampler balanceado:
USE_SAMPLER = False

N_normal = sum(1 for _,y in train_items if y==0)
N_hurto  = sum(1 for _,y in train_items if y==1)
print(f"Train clips: {len(train_ds)} | Val clips: {len(val_ds)}")
print(f"Conteo train (items CSV) → normal={N_normal} | hurto={N_hurto}")

# DataLoader: pin_memory solo si hay CUDA
NUM_WORKERS = 0  # en CPU puede ayudar; si ves sobrecarga, bajalo
pinmem = torch.cuda.is_available()

if USE_SAMPLER:
    from torch.utils.data import WeightedRandomSampler
    weight_per_class = {0: 1.0/max(1,N_normal), 1: 1.0/max(1,N_hurto)}
    sample_weights = np.array([weight_per_class[y] for _, y in train_items], dtype=np.float32)
    sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)
    train_loader = DataLoader(
        train_ds,
        batch_size=BATCH_SIZE,
        sampler=sampler,
        num_workers=NUM_WORKERS,
        pin_memory=pinmem,
        drop_last=True,
        persistent_workers=(NUM_WORKERS>0)
    )
else:
    train_loader = DataLoader(
        train_ds,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=pinmem,
        drop_last=True,
        persistent_workers=(NUM_WORKERS>0)
    )

val_loader = DataLoader(
    val_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=pinmem,
    persistent_workers=(NUM_WORKERS>0)
)

# pesos por clase
total = N_normal + N_hurto
w_normal = total / (2 * max(1, N_normal))
w_hurto  = total / (2 * max(1, N_hurto))
print(f"Class weights (aprox) -> normal={w_normal:.3f} | hurto={w_hurto:.3f}")
CLASS_WEIGHTS_CPU = torch.tensor([w_normal, w_hurto], dtype=torch.float32)

# ---------- MODELO ----------
# compatibilidad de versiones de torchvision
try:
    model = r3d_18(weights="KINETICS400_V1")
except TypeError:
    model = r3d_18(pretrained=True)

model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
model = model.to(DEVICE)

# Congelar TODO menos layer3, layer4 y fc
for name, p in model.named_parameters():
    p.requires_grad = any(blk in name for blk in ["layer3","layer4","fc"])

# ---------- LOSS (Focal) ----------
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, weight=None, reduction="mean"):
        super().__init__()
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss(weight=weight, reduction="none")
        self.reduction = reduction
    def forward(self, logits, target):
        ce = self.ce(logits, target)         # [B]
        pt = torch.exp(-ce)
        loss = ((1-pt)**self.gamma) * ce     # [B]
        if self.reduction == "mean": return loss.mean()
        if self.reduction == "sum":  return loss.sum()
        return loss

criterion = FocalLoss(gamma=GAMMA, weight=CLASS_WEIGHTS_CPU.to(DEVICE))

# ---------- OPTIM / SCHED ----------
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()),
                              lr=LR, weight_decay=WEIGHT_DEC)

# AMP GradScaler para GPU
from torch.amp import GradScaler
scaler = GradScaler("cuda", enabled=torch.cuda.is_available())

def cosine_warmup_lr(t, T, base_lr, warmup=2):
    if t < warmup:
        return base_lr * (t+1)/warmup
    tw = max(1, T - warmup)
    tc = max(0, t - warmup)
    return 0.5 * base_lr * (1 + math.cos(math.pi * tc / tw))

# ---------- REANUDACIÓN COMPLETA ----------
def _rng_state():
    return {
        "py_random": random.getstate(),
        "np_random": np.random.get_state(),
        "torch_cpu": torch.random.get_rng_state(),
        "torch_cuda": torch.cuda.get_rng_state_all() if torch.cuda.is_available() else None,
    }

def _set_rng_state(st):
    if st is None: return
    random.setstate(st.get("py_random"))
    np.random.set_state(st.get("np_random"))
    torch.random.set_rng_state(st.get("torch_cpu"))
    if torch.cuda.is_available() and st.get("torch_cuda") is not None:
        torch.cuda.set_rng_state_all(st.get("torch_cuda"))

def save_ckpt(path, epoch, best_f1, best_loss, row, extra=None):
    pack = {
        "epoch": epoch,
        "best_f1": best_f1,
        "best_loss": best_loss,
        "metrics": row,
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
        "scaler": scaler.state_dict() if torch.cuda.is_available() else None,
        "class_weights": CLASS_WEIGHTS_CPU,
        "rng": _rng_state(),
    }
    if extra: pack.update(extra)
    torch.save(pack, path)

def try_resume():
    start_epoch = 0
    best_f1, best_loss = -1.0, float("inf")
    if os.path.exists(CKPT_LAST):
        sd = torch.load(CKPT_LAST, map_location="cpu")
        model.load_state_dict(sd["model"], strict=False)
        if "optimizer" in sd: optimizer.load_state_dict(sd["optimizer"])
        if torch.cuda.is_available() and sd.get("scaler") is not None:
            scaler.load_state_dict(sd["scaler"])
        _set_rng_state(sd.get("rng"))
        start_epoch = sd.get("epoch", 0)
        best_f1     = sd.get("best_f1", best_f1)
        best_loss   = sd.get("best_loss", best_loss)
        print(f"🔁 Reanudado desde {CKPT_LAST} (epoch={start_epoch})")
    else:
        print("⏩ Sin estado previo completo. Usando CKPT_IN (solo pesos) si existe.")
        if os.path.exists(CKPT_IN):
            sd = torch.load(CKPT_IN, map_location="cpu")
            state_dict = sd["model"] if isinstance(sd, dict) and "model" in sd else sd
            missing, unexpected = model.load_state_dict(state_dict, strict=False)
            print("Cargado ckpt previo:", CKPT_IN)
            print("Missing:", missing, "Unexpected:", unexpected)
        else:
            print("⚠️ No se encontró CKPT_IN; se parte de Kinetics.")
    return start_epoch, best_f1, best_loss

# ---------- LOOP ----------
def run_epoch(loader, train=True, current_epoch=0, save_every=None,
              max_batches=None, global_step_start=0):
    if train: model.train()
    else:     model.eval()

    tot_loss = 0.0
    all_probs, all_ys = [], []
    global_step = global_step_start

    for i, (x, y) in enumerate(loader):
        if (max_batches is not None) and (i >= max_batches):
            break

        x = x.to(DEVICE, non_blocking=True)
        y = y.to(DEVICE, non_blocking=True)

        from torch.amp import autocast
        with autocast("cuda", enabled=torch.cuda.is_available()):
            logits = model(x)
            loss = criterion(logits, y)

        if train:
            optimizer.zero_grad(set_to_none=True)
            if torch.cuda.is_available():
                scaler.scale(loss).backward()
                torch.nn.utils.clip_grad_norm_(filter(lambda p: p.requires_grad, model.parameters()), 5.0)
                scaler.step(optimizer); scaler.update()
            else:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(filter(lambda p: p.requires_grad, model.parameters()), 5.0)
                optimizer.step()

            global_step += 1
            # Checkpoint intermedio
            if save_every and (global_step % save_every == 0):
                save_ckpt(CKPT_LAST, epoch=current_epoch, best_f1=best_f1,
                          best_loss=best_loss, row={"epoch": current_epoch},
                          extra={"global_step": global_step})
                print(f"💾 Guardado intermedio (step={global_step}) -> {CKPT_LAST}")

        probs_hurto = logits.softmax(1)[:,1].detach().cpu().numpy()
        all_probs.append(probs_hurto)
        all_ys.append(y.detach().cpu().numpy())
        tot_loss += loss.item() * y.size(0)

    all_probs = np.concatenate(all_probs) if all_probs else np.array([])
    all_ys    = np.concatenate(all_ys)    if all_ys    else np.array([])
    avg_loss  = tot_loss / max(1, len(loader.dataset))

    # F1 al umbral 0.5 como referencia
    if all_probs.size > 0:
        y_pred = (all_probs >= 0.5).astype(int)
        f1 = f1_score(all_ys, y_pred, pos_label=1)
    else:
        f1 = 0.0
    return avg_loss, f1, all_probs, all_ys, global_step

# Inicializar/reanudar
start_epoch, best_f1, best_loss = try_resume()
history = []
global_step = 0

first_epoch = start_epoch + 1
last_epoch  = min(EPOCHS, start_epoch + RUN_EPOCHS_THIS_SESSION)

for epoch in range(first_epoch, last_epoch + 1):
    # scheduler manual con warmup+cosine
    for pg in optimizer.param_groups:
        pg["lr"] = cosine_warmup_lr(epoch-1, EPOCHS, LR, warmup=2)

    tr_loss, tr_f1, _, _, global_step = run_epoch(
        train_loader, train=True, current_epoch=epoch,
        save_every=SAVE_EVERY_N_BATCHES,
        max_batches=MAX_TRAIN_BATCHES,
        global_step_start=global_step
    )
    va_loss, va_f1, probs, ys, _ = run_epoch(
        val_loader, train=False, current_epoch=epoch,
        save_every=None, max_batches=MAX_VAL_BATCHES,
        global_step_start=global_step
    )

    row = {"epoch": epoch, "lr": optimizer.param_groups[0]["lr"],
           "train_loss": tr_loss, "train_f1": tr_f1,
           "val_loss": va_loss,   "val_f1": va_f1}
    history.append(row)
    print(f"[{epoch:02d}/{EPOCHS}] lr={row['lr']:.2e} | "
          f"train {tr_loss:.4f}/{tr_f1:.3f} | val {va_loss:.4f}/{va_f1:.3f}")

    # Guardado "last" SIEMPRE al final de cada época (para reanudar exacto)
    save_ckpt(CKPT_LAST, epoch=epoch, best_f1=best_f1, best_loss=best_loss, row=row)
    try:
        shutil.copy2(CKPT_LAST, CKPT_RUN)
    except Exception:
        pass

    # Guardados “best”
    if va_f1 > best_f1:
        best_f1 = va_f1
        torch.save({"model": model.state_dict(), "epoch": epoch, "metrics": row}, CKPT_BESTF1)
        print("✔️ Mejor F1, guardado ->", CKPT_BESTF1)
    if va_loss < best_loss:
        best_loss = va_loss
        torch.save({"model": model.state_dict(), "epoch": epoch, "metrics": row}, CKPT_BESTL)
        print("✔️ Mejor Loss, guardado ->", CKPT_BESTL)

# ---------- LOG / REPORT ----------
with open(LOG_JSON, "w") as f:
    json.dump(history, f, indent=2)

# Reporte en el último estado: clasificación y matriz (umbral 0.5)
if 'probs' in locals() and probs.size > 0:
    y_pred = (probs >= 0.5).astype(int)
    print("\n=== Classification Report (umbral 0.5) ===")
    print(classification_report(ys, y_pred, target_names=["normal","hurto"]))
    print("Matriz de confusión (val):\n", confusion_matrix(ys, y_pred))
else:
    print("\n(No hubo validación con muestras en esta sesión; salteo reporte inmediato)")

# Sweep de umbral para elegir threshold
import csv as _csv
if 'probs' in locals() and probs.size > 0:
    ths = np.round(np.linspace(0.30, 0.80, 21), 3)
    with open(THR_CSV, "w", newline="") as f:
        w = _csv.writer(f); w.writerow(["threshold","precision","recall","f1"])
        for t in ths:
            yp = (probs >= t).astype(int)
            rep = classification_report(ys, yp, output_dict=True, zero_division=0)
            p = rep["1"]["precision"]; r = rep["1"]["recall"]; f1 = rep["1"]["f1-score"]
            w.writerow([t, p, r, f1])
    print("Tabla de thresholds guardada en:", THR_CSV)
else:
    print("Sin probs/ys de validación en esta sesión; no genero THR_CSV aún.")

print("Log de entrenamiento:", LOG_JSON)


CUDA disponible: False
Train clips: 10668 | Val clips: 787
Conteo train (items CSV) → normal=2082 | hurto=8586
Class weights (aprox) -> normal=2.562 | hurto=0.621


/tmp/ipython-input-1129450782.py:297: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sd = torch.load(CKPT_LAST, map_location="cpu")


🔁 Reanudado desde /content/drive/MyDrive/tesisV2/models/r3d18_ft_l34_last.pt (epoch=6)
💾 Guardado intermedio (step=300) -> /content/drive/MyDrive/tesisV2/models/r3d18_ft_l34_last.pt
💾 Guardado intermedio (step=600) -> /content/drive/MyDrive/tesisV2/models/r3d18_ft_l34_last.pt
💾 Guardado intermedio (step=900) -> /content/drive/MyDrive/tesisV2/models/r3d18_ft_l34_last.pt
💾 Guardado intermedio (step=1200) -> /content/drive/MyDrive/tesisV2/models/r3d18_ft_l34_last.pt
💾 Guardado intermedio (step=1500) -> /content/drive/MyDrive/tesisV2/models/r3d18_ft_l34_last.pt
[07/15] lr=1.57e-04 | train 0.0036/0.996 | val 0.7440/0.416
✔️ Mejor F1, guardado -> /content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_ft_l34_bestF1.pt
✔️ Mejor Loss, guardado -> /content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_ft_l34_bestLoss.pt
💾 Guardado intermedio (step=1800) -> /content/drive/MyDrive/tesisV2/models/r3d18_ft_l34_last.pt
💾 Guardado intermedio (step=2100) -> /content/drive/MyDrive/tesisV2/models/r3d18

In [ ]:
# =========================
# Fine-tune R3D-18 layer3+layer4 (CPU/GPU con reanudación y micro-sesiones)
# =========================
import os, math, json, random, csv, shutil
from pathlib import Path
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report, confusion_matrix, f1_score

import torchvision.transforms as T
from torchvision.io import read_video
from torchvision.models.video import r3d_18

# ---------- CONFIG ----------
SEED        = 1337
DEVICE      = "cuda" if torch.cuda.is_available() else "cpu"
NUM_CLASSES = 2
EPOCHS      = 15
BATCH_SIZE  = 6
LR          = 2e-4
WEIGHT_DEC  = 5e-4   # <--- subimos regularización L2 (antes 2e-4)
GAMMA       = 2.0    # <--- focal un poco menos agresiva (antes 2.5)

OUT_DIR     = "/content/drive/MyDrive/tesisV2/models"
CKPT_IN     = "/content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_finetune.pt"  # mejor de layer4
CKPT_BESTF1 = os.path.join(OUT_DIR, "r3d18_shoplifting_ft_l34_bestF1.pt")
CKPT_BESTL  = os.path.join(OUT_DIR, "r3d18_shoplifting_ft_l34_bestLoss.pt")
LOG_JSON    = os.path.join(OUT_DIR, "r3d18_ft_l34_trainlog.json")
THR_CSV     = os.path.join(OUT_DIR, "r3d18_ft_l34_threshold_sweep.csv")

# ---------- CHECKPOINTS / CONTROL DE SESIÓN ----------
CKPT_LAST   = os.path.join(OUT_DIR, "r3d18_ft_l34_last.pt")      # último estado (para reanudar)
CKPT_RUN    = os.path.join(OUT_DIR, "r3d18_ft_l34_runstate.pt")  # alias (copia del last)

# Ejecutá en “micro-sesiones”
RUN_EPOCHS_THIS_SESSION = 2      # cuántas épocas avanza ESTA corrida
SAVE_EVERY_N_BATCHES    = 300    # checkpoint intermedio cada N batches (None para desactivar)

# Para acelerar/probar en CPU: limitá batches por época (None = sin límite)
MAX_TRAIN_BATCHES = None   # ej: 400
MAX_VAL_BATCHES   = None   # ej: 100

os.makedirs(OUT_DIR, exist_ok=True)

def set_seed(s=SEED):
    random.seed(s); np.random.seed(s); torch.manual_seed(s); torch.cuda.manual_seed_all(s)
set_seed()

# ---- GPU perf tweaks ----
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    try:
        torch.set_float32_matmul_precision("medium")
    except Exception:
        pass
print("CUDA disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

# ---------- DATA LOADERS ----------
BASE_DIR  = "/content/tesisV2/cortos/video_clips_192"
TRAIN_CSV = f"{BASE_DIR}/train.csv"
VAL_CSV   = f"{BASE_DIR}/val.csv"

CLIP_LEN = 16   # frames por clip
RESIZE   = 128
CROP     = 112

train_spatial = T.Compose([
    T.Resize((RESIZE, RESIZE)),
    T.RandomResizedCrop(CROP, scale=(0.7, 1.0)),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomApply([T.ColorJitter(0.3,0.3,0.3,0.08)], p=0.5),
    # T.RandomErasing(p=0.25, scale=(0.02, 0.15), ratio=(0.3, 3.3), inplace=False),  # <--- desactivado para CPU
])

val_spatial = T.Compose([
    T.Resize((RESIZE, RESIZE)),
    T.CenterCrop(CROP),
])

def load_list(csv_path):
    """Lee CSV con filas: path,label
       label puede ser 0/1 o 'normal'/'shoplifting'/'hurto' (case-insensitive)."""
    def parse_label(s):
        s = str(s).strip().lower()
        if s.isdigit(): return int(s)
        if s in ("normal", "0"): return 0
        if s in ("shoplifting", "hurto", "1"): return 1
        raise ValueError(f"Etiqueta desconocida: {s}")

    items = []
    with open(csv_path, newline='') as f:
        r = csv.reader(f)
        for row in r:
            if not row:
                continue
            head = row[0].strip().lower()
            if head in ("path","ruta","file","filename"):  # saltar cabecera
                continue
            path = row[0].strip()
            y    = parse_label(row[1])

            # resolver ruta relativa
            if not path.startswith("/"):
                cand = os.path.join(BASE_DIR, path)
                if os.path.exists(cand):
                    path = cand
                else:
                    subdir = "normal" if y == 0 else "shoplifting"
                    path = os.path.join(BASE_DIR, subdir, path)
            items.append((path, y))
    return items

class SimpleVideoDataset(Dataset):
    def __init__(self, items, train=True):
        self.items   = items
        self.train   = train
        self.spatial = train_spatial if train else val_spatial
        # normalización Kinetics para r3d_18
        self.mean = torch.tensor([0.43216, 0.394666, 0.37645]).view(1,3,1,1)
        self.std  = torch.tensor([0.22803, 0.22145, 0.216989]).view(1,3,1,1)

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        path, y = self.items[idx]
        # video -> [T, C, H, W] uint8
        video, _, _ = read_video(path, pts_unit="sec", output_format="TCHW")
        T_total = video.shape[0]

        # muestreo simple de un clip de CLIP_LEN
        if T_total >= CLIP_LEN:
            start = random.randint(0, T_total - CLIP_LEN) if self.train else max(0, (T_total - CLIP_LEN)//2)
            clip = video[start:start+CLIP_LEN]  # [T,C,H,W]
        else:
            # pad repitiendo el último frame
            pad = CLIP_LEN - T_total
            clip = torch.cat([video, video[-1:].repeat(pad,1,1,1)], dim=0)

        # aplicar transform espacial por frame (cada frame es [C,H,W])
        clip = torch.stack([self.spatial(fr) for fr in clip])   # [T,C,H,W]
        clip = clip.float() / 255.0
        clip = (clip - self.mean) / self.std

        # r3d_18 espera [C,T,H,W]
        clip = clip.permute(1,0,2,3).contiguous()
        return clip, torch.tensor(y, dtype=torch.long)

# construir datasets/loaders
train_items = load_list(TRAIN_CSV)
val_items   = load_list(VAL_CSV)

train_ds = SimpleVideoDataset(train_items, train=True)
val_ds   = SimpleVideoDataset(val_items,   train=False)

# (Opcional) sampler balanceado:
USE_SAMPLER = False

N_normal = sum(1 for _,y in train_items if y==0)
N_hurto  = sum(1 for _,y in train_items if y==1)
print(f"Train clips: {len(train_ds)} | Val clips: {len(val_ds)}")
print(f"Conteo train (items CSV) → normal={N_normal} | hurto={N_hurto}")

# DataLoader: en CPU evitamos workers y pin_memory
NUM_WORKERS = 0
pinmem = False  # torch.cuda.is_available() y útil solo en GPU

if USE_SAMPLER:
    from torch.utils.data import WeightedRandomSampler
    weight_per_class = {0: 1.0/max(1,N_normal), 1: 1.0/max(1,N_hurto)}
    sample_weights = np.array([weight_per_class[y] for _, y in train_items], dtype=np.float32)
    sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)
    train_loader = DataLoader(
        train_ds,
        batch_size=BATCH_SIZE,
        sampler=sampler,
        num_workers=NUM_WORKERS,
        pin_memory=pinmem,
        drop_last=True,
        persistent_workers=(NUM_WORKERS>0)
    )
else:
    train_loader = DataLoader(
        train_ds,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=pinmem,
        drop_last=True,
        persistent_workers=(NUM_WORKERS>0)
    )

val_loader = DataLoader(
    val_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=pinmem,
    persistent_workers=(NUM_WORKERS>0)
)

# pesos por clase (AJUSTADOS para subir recall de hurto)
CLASS_WEIGHTS_CPU = torch.tensor([1.0, 1.3], dtype=torch.float32)
print(f"Class weights fijados -> normal={CLASS_WEIGHTS_CPU[0].item():.3f} | hurto={CLASS_WEIGHTS_CPU[1].item():.3f}")

# ---------- MODELO ----------
# compatibilidad de versiones de torchvision
try:
    model = r3d_18(weights="KINETICS400_V1")
except TypeError:
    model = r3d_18(pretrained=True)

# FC con Dropout para bajar overfit
in_feats = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(in_feats, NUM_CLASSES)
)
model = model.to(DEVICE)

# Congelar TODO menos layer3, layer4 y fc
for name, p in model.named_parameters():
    p.requires_grad = any(blk in name for blk in ["layer3","layer4","fc"])

# ---------- LOSS (Focal) ----------
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, weight=None, reduction="mean"):
        super().__init__()
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss(weight=weight, reduction="none")
        self.reduction = reduction
    def forward(self, logits, target):
        ce = self.ce(logits, target)         # [B]
        pt = torch.exp(-ce)
        loss = ((1-pt)**self.gamma) * ce     # [B]
        if self.reduction == "mean": return loss.mean()
        if self.reduction == "sum":  return loss.sum()
        return loss

criterion = FocalLoss(gamma=GAMMA, weight=CLASS_WEIGHTS_CPU.to(DEVICE))

# ---------- OPTIM / SCHED ----------
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()),
                              lr=LR, weight_decay=WEIGHT_DEC)

# AMP GradScaler para GPU
from torch.amp import GradScaler
scaler = GradScaler("cuda", enabled=torch.cuda.is_available())

def cosine_warmup_lr(t, T, base_lr, warmup=2):
    if t < warmup:
        return base_lr * (t+1)/warmup
    tw = max(1, T - warmup)
    tc = max(0, t - warmup)
    return 0.5 * base_lr * (1 + math.cos(math.pi * tc / tw))

# ---------- REANUDACIÓN COMPLETA ----------
def _rng_state():
    return {
        "py_random": random.getstate(),
        "np_random": np.random.get_state(),
        "torch_cpu": torch.random.get_rng_state(),
        "torch_cuda": torch.cuda.get_rng_state_all() if torch.cuda.is_available() else None,
    }

def _set_rng_state(st):
    if st is None: return
    random.setstate(st.get("py_random"))
    np.random.set_state(st.get("np_random"))
    torch.random.set_rng_state(st.get("torch_cpu"))
    if torch.cuda.is_available() and st.get("torch_cuda") is not None:
        torch.cuda.set_rng_state_all(st.get("torch_cuda"))

def save_ckpt(path, epoch, best_f1, best_loss, row, extra=None):
    pack = {
        "epoch": epoch,
        "best_f1": best_f1,
        "best_loss": best_loss,
        "metrics": row,
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
        "scaler": scaler.state_dict() if torch.cuda.is_available() else None,
        "class_weights": CLASS_WEIGHTS_CPU,
        "rng": _rng_state(),
    }
    if extra: pack.update(extra)
    torch.save(pack, path)

def try_resume():
    start_epoch = 0
    best_f1, best_loss = -1.0, float("inf")
    if os.path.exists(CKPT_LAST):
        sd = torch.load(CKPT_LAST, map_location="cpu")
        model.load_state_dict(sd["model"], strict=False)
        if "optimizer" in sd: optimizer.load_state_dict(sd["optimizer"])
        if torch.cuda.is_available() and sd.get("scaler") is not None:
            scaler.load_state_dict(sd["scaler"])
        _set_rng_state(sd.get("rng"))
        start_epoch = sd.get("epoch", 0)
        best_f1     = sd.get("best_f1", best_f1)
        best_loss   = sd.get("best_loss", best_loss)
        print(f"🔁 Reanudado desde {CKPT_LAST} (epoch={start_epoch})")
    else:
        print("⏩ Sin estado previo completo. Usando CKPT_IN (solo pesos) si existe.")
        if os.path.exists(CKPT_IN):
            sd = torch.load(CKPT_IN, map_location="cpu")
            state_dict = sd["model"] if isinstance(sd, dict) and "model" in sd else sd
            missing, unexpected = model.load_state_dict(state_dict, strict=False)
            print("Cargado ckpt previo:", CKPT_IN)
            print("Missing:", missing, "Unexpected:", unexpected)
        else:
            print("⚠️ No se encontró CKPT_IN; se parte de Kinetics.")
    return start_epoch, best_f1, best_loss

# ---------- LOOP ----------
def run_epoch(loader, train=True, current_epoch=0, save_every=None,
              max_batches=None, global_step_start=0):
    if train: model.train()
    else:     model.eval()

    tot_loss = 0.0
    all_probs, all_ys = [], []
    global_step = global_step_start

    for i, (x, y) in enumerate(loader):
        if (max_batches is not None) and (i >= max_batches):
            break

        x = x.to(DEVICE, non_blocking=True)
        y = y.to(DEVICE, non_blocking=True)

        from torch.amp import autocast
        with autocast("cuda", enabled=torch.cuda.is_available()):
            logits = model(x)
            loss = criterion(logits, y)

        if train:
            optimizer.zero_grad(set_to_none=True)
            if torch.cuda.is_available():
                scaler.scale(loss).backward()
                torch.nn.utils.clip_grad_norm_(filter(lambda p: p.requires_grad, model.parameters()), 5.0)
                scaler.step(optimizer); scaler.update()
            else:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(filter(lambda p: p.requires_grad, model.parameters()), 5.0)
                optimizer.step()

            global_step += 1
            # Checkpoint intermedio
            if save_every and (global_step % save_every == 0):
                save_ckpt(CKPT_LAST, epoch=current_epoch, best_f1=best_f1,
                          best_loss=best_loss, row={"epoch": current_epoch},
                          extra={"global_step": global_step})
                print(f"💾 Guardado intermedio (step={global_step}) -> {CKPT_LAST}")

        probs_hurto = logits.softmax(1)[:,1].detach().cpu().numpy()
        all_probs.append(probs_hurto)
        all_ys.append(y.detach().cpu().numpy())
        tot_loss += loss.item() * y.size(0)

    all_probs = np.concatenate(all_probs) if all_probs else np.array([])
    all_ys    = np.concatenate(all_ys)    if all_ys    else np.array([])
    avg_loss  = tot_loss / max(1, len(loader.dataset))

    # F1 al umbral 0.5 como referencia
    if all_probs.size > 0:
        y_pred = (all_probs >= 0.5).astype(int)
        f1 = f1_score(all_ys, y_pred, pos_label=1)
    else:
        f1 = 0.0
    return avg_loss, f1, all_probs, all_ys, global_step

# Inicializar/reanudar
start_epoch, best_f1, best_loss = try_resume()
history = []
global_step = 0

first_epoch = start_epoch + 1
last_epoch  = min(EPOCHS, start_epoch + RUN_EPOCHS_THIS_SESSION)

for epoch in range(first_epoch, last_epoch + 1):
    # scheduler manual con warmup+cosine
    for pg in optimizer.param_groups:
        pg["lr"] = cosine_warmup_lr(epoch-1, EPOCHS, LR, warmup=2)

    tr_loss, tr_f1, _, _, global_step = run_epoch(
        train_loader, train=True, current_epoch=epoch,
        save_every=SAVE_EVERY_N_BATCHES,
        max_batches=MAX_TRAIN_BATCHES,
        global_step_start=global_step
    )
    va_loss, va_f1, probs, ys, _ = run_epoch(
        val_loader, train=False, current_epoch=epoch,
        save_every=None, max_batches=MAX_VAL_BATCHES,
        global_step_start=global_step
    )

    row = {"epoch": epoch, "lr": optimizer.param_groups[0]["lr"],
           "train_loss": tr_loss, "train_f1": tr_f1,
           "val_loss": va_loss,   "val_f1": va_f1}
    history.append(row)
    print(f"[{epoch:02d}/{EPOCHS}] lr={row['lr']:.2e} | "
          f"train {tr_loss:.4f}/{tr_f1:.3f} | val {va_loss:.4f}/{va_f1:.3f}")

    # Guardado "last" SIEMPRE al final de cada época (para reanudar exacto)
    save_ckpt(CKPT_LAST, epoch=epoch, best_f1=best_f1, best_loss=best_loss, row=row)
    try:
        shutil.copy2(CKPT_LAST, CKPT_RUN)
    except Exception:
        pass

    # Guardados “best”
    if va_f1 > best_f1:
        best_f1 = va_f1
        torch.save({"model": model.state_dict(), "epoch": epoch, "metrics": row}, CKPT_BESTF1)
        print("✔️ Mejor F1, guardado ->", CKPT_BESTF1)
    if va_loss < best_loss:
        best_loss = va_loss
        torch.save({"model": model.state_dict(), "epoch": epoch, "metrics": row}, CKPT_BESTL)
        print("✔️ Mejor Loss, guardado ->", CKPT_BESTL)

# ---------- LOG / REPORT ----------
with open(LOG_JSON, "w") as f:
    json.dump(history, f, indent=2)

# Reporte en el último estado: clasificación y matriz (umbral 0.5)
if 'probs' in locals() and probs.size > 0:
    y_pred = (probs >= 0.5).astype(int)
    print("\n=== Classification Report (umbral 0.5) ===")
    print(classification_report(ys, y_pred, target_names=["normal","hurto"]))
    print("Matriz de confusión (val):\n", confusion_matrix(ys, y_pred))
else:
    print("\n(No hubo validación con muestras en esta sesión; salteo reporte inmediato)")

# Sweep de umbral para elegir threshold
import csv as _csv, json as _json
if 'probs' in locals() and probs.size > 0:
    ths = np.round(np.linspace(0.30, 0.80, 21), 3)
    with open(THR_CSV, "w", newline="") as f:
        w = _csv.writer(f); w.writerow(["threshold","precision","recall","f1"])
        for t in ths:
            yp = (probs >= t).astype(int)
            rep = classification_report(ys, yp, output_dict=True, zero_division=0)
            p = rep["1"]["precision"]; r = rep["1"]["recall"]; f1 = rep["1"]["f1-score"]
            w.writerow([t, p, r, f1])
    print("Tabla de thresholds guardada en:", THR_CSV)

    # === Elegir mejor threshold por F1, mostrar top-5 y guardarlo como operativo ===
    tops = []
    with open(THR_CSV, newline="") as f:
        r = _csv.DictReader(f)
        for row in r:
            tops.append((
                float(row["f1"]),
                float(row["threshold"]),
                float(row["precision"]),
                float(row["recall"])
            ))
    tops.sort(reverse=True)

    if tops:
        print("\nTop-5 thresholds por F1:")
        for f1, t, p, r in tops[:5]:
            print(f" t={t:.3f} | F1={f1:.3f} | P={p:.3f} | R={r:.3f}")

        best_t = tops[0][1]
        y_pred_best = (probs >= best_t).astype(int)
        print(f"\n=== Report con threshold óptimo (t={best_t:.3f}) ===")
        print(classification_report(ys, y_pred_best, target_names=["normal","hurto"]))
        print("Matriz de confusión (val, t*):\n", confusion_matrix(ys, y_pred_best))

        OPER_JSON = os.path.join(OUT_DIR, "operating_threshold.json")
        with open(OPER_JSON, "w") as f:
            _json.dump({"threshold": float(best_t)}, f, indent=2)
        print("Operating threshold ->", OPER_JSON)
else:
    print("Sin probs/ys de validación en esta sesión; no genero THR_CSV aún.")

print("Log de entrenamiento:", LOG_JSON)


CUDA disponible: False
Train clips: 10668 | Val clips: 787
Conteo train (items CSV) → normal=2082 | hurto=8586
Class weights fijados -> normal=1.000 | hurto=1.300


/tmp/ipython-input-3571270764.py:299: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sd = torch.load(CKPT_LAST, map_location="cpu")


🔁 Reanudado desde /content/drive/MyDrive/tesisV2/models/r3d18_ft_l34_last.pt (epoch=9)
💾 Guardado intermedio (step=300) -> /content/drive/MyDrive/tesisV2/models/r3d18_ft_l34_last.pt
💾 Guardado intermedio (step=600) -> /content/drive/MyDrive/tesisV2/models/r3d18_ft_l34_last.pt
💾 Guardado intermedio (step=900) -> /content/drive/MyDrive/tesisV2/models/r3d18_ft_l34_last.pt
💾 Guardado intermedio (step=1200) -> /content/drive/MyDrive/tesisV2/models/r3d18_ft_l34_last.pt
💾 Guardado intermedio (step=1500) -> /content/drive/MyDrive/tesisV2/models/r3d18_ft_l34_last.pt
[10/15] lr=8.79e-05 | train 0.0009/1.000 | val 1.1922/0.466
✔️ Mejor F1, guardado -> /content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_ft_l34_bestF1.pt
💾 Guardado intermedio (step=1800) -> /content/drive/MyDrive/tesisV2/models/r3d18_ft_l34_last.pt
💾 Guardado intermedio (step=2100) -> /content/drive/MyDrive/tesisV2/models/r3d18_ft_l34_last.pt
💾 Guardado intermedio (step=2400) -> /content/drive/MyDrive/tesisV2/models/r3d18_ft_l3

In [ ]:
pip install --upgrade --force-reinstall --no-cache-dir torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 132.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 156.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 129.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 117.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 119.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 109.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# =========================
# EVALUACIÓN (SIN ENTRENAR) con OpenCV
# R3D-18 | t=0.30 + smoothing + histéresis + minDur
# =========================
import os, re, csv
from pathlib import Path
import numpy as np
import torch, torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
from torchvision.models.video import r3d_18
from sklearn.metrics import confusion_matrix, classification_report
import cv2

# ---------- RUTAS ----------
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"
BASE_DIR   = "/content/tesisV2/cortos/video_clips_192"
VAL_CSV    = f"{BASE_DIR}/val.csv"
TEST_CSV   = f"{BASE_DIR}/test.csv"
CKPT_PATH  = "/content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_ft_l34_bestF1.pt"

# ---------- MODELO ----------
NUM_CLASSES = 2
CLIP_LEN = 16
RESIZE   = 128
CROP     = 112

val_spatial = T.Compose([T.Resize((RESIZE, RESIZE)), T.CenterCrop(CROP)])

class SimpleVideoEval(Dataset):
    def __init__(self, csv_path, base_dir):
        self.items=[]
        self.base_dir=base_dir
        with open(csv_path, newline='') as f:
            r = csv.reader(f)
            for row in r:
                if not row: continue
                head=row[0].strip().lower()
                if head in ("path","ruta","file","filename"):  # header
                    continue
                path=row[0].strip()
                y = 0 if str(row[1]).lower() in ("0","normal") else 1
                if not path.startswith("/"):
                    cand=os.path.join(base_dir, path)
                    if os.path.exists(cand):
                        path=cand
                    else:
                        sub="normal" if y==0 else "shoplifting"
                        path=os.path.join(base_dir, sub, path)
                self.items.append((path, y))
        # Normalización Kinetics
        self.mean = torch.tensor([0.43216, 0.394666, 0.37645]).view(1,3,1,1)
        self.std  = torch.tensor([0.22803, 0.22145, 0.216989]).view(1,3,1,1)

    def __len__(self): return len(self.items)

    def _read_video_cv2(self, path):
        cap=cv2.VideoCapture(str(path))
        frames=[]
        while True:
            ok, fr=cap.read()
            if not ok: break
            fr=cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
            frames.append(fr)
        cap.release()
        if len(frames)==0:
            raise RuntimeError(f"No se pudo leer: {path}")
        T_total=len(frames)
        if T_total>=CLIP_LEN:
            start=max(0,(T_total-CLIP_LEN)//2)  # centro para eval
            frames=frames[start:start+CLIP_LEN]
        else:
            last=frames[-1]
            frames=frames + [last]*(CLIP_LEN-T_total)
        arr=np.stack(frames, axis=0)                 # [T,H,W,3] uint8
        ten=torch.from_numpy(arr).permute(0,3,1,2)   # [T,3,H,W]
        return ten

    def __getitem__(self, idx):
        path,y=self.items[idx]
        clip=self._read_video_cv2(path)                  # [T,3,H,W]
        clip=torch.stack([val_spatial(fr) for fr in clip])
        clip=clip.float()/255.0
        clip=(clip - self.mean)/self.std
        clip=clip.permute(1,0,2,3).contiguous()          # [3,T,H,W]
        return clip, torch.tensor(y, dtype=torch.long), path

def build_model():
    try:
        model=r3d_18(weights="KINETICS400_V1")
    except TypeError:
        model=r3d_18(pretrained=True)
    in_feats=model.fc.in_features
    model.fc=nn.Sequential(nn.Dropout(0.5), nn.Linear(in_feats, NUM_CLASSES))
    sd=torch.load(CKPT_PATH, map_location="cpu")
    state_dict = sd["model"] if isinstance(sd, dict) and "model" in sd else sd
    missing, unexpected = model.load_state_dict(state_dict, strict=False)
    print("Checkpoint cargado:", CKPT_PATH)
    if missing or unexpected:
        print("Missing:", missing, "Unexpected:", unexpected)
    return model.to(DEVICE).eval()

# ---------- INFERENCIA ----------
def infer_loader(model, loader):
    probs_all, ys_all, paths_all = [], [], []
    soft=nn.Softmax(dim=1)
    torch.set_grad_enabled(False)
    for x,y,paths in loader:
        x=x.to(DEVICE, non_blocking=True)
        logits=model(x)
        probs=soft(logits)[:,1].detach().cpu().numpy()
        probs_all.append(probs)
        ys_all.append(y.numpy())
        paths_all.extend(list(paths))
    return np.concatenate(probs_all), np.concatenate(ys_all), paths_all

# ---------- POST-PROCESO ----------
HOP_SECONDS   = 0.5   # ajustá si tus clips provienen de hop distinto
SMOOTH_K      = 5     # ~2.5 s si hop=0.5
HYST_ON       = 0.35
HYST_OFF      = 0.25
MIN_EVENT_SEC = 1.0
COOLDOWN_SEC  = 2.0
THRESH_BASE   = 0.30

def moving_avg(a,k):
    if k<=1: return a
    pad=(k-1)//2
    a_pad=np.pad(a,(pad,pad),mode='edge')
    ker=np.ones(k)/k
    return np.convolve(a_pad,ker,mode='valid')

def hysteresis_events(probs,on,off,hop_s,min_event_s,cooldown_s):
    active=False; start=None; events=[]
    for i,p in enumerate(probs):
        if not active and p>=on:
            active=True; start=i
        elif active and p<=off:
            end=i
            if (end-start)*hop_s>=min_event_s:
                events.append([start,end])
            active=False
    if active:
        end=len(probs)
        if (end-start)*hop_s>=min_event_s:
            events.append([start,end])
    merged=[]
    for s,e in events:
        if not merged: merged.append([s,e])
        else:
            ps,pe=merged[-1]
            if (s-pe)*hop_s<COOLDOWN_SEC:
                merged[-1][1]=e
            else:
                merged.append([s,e])
    return merged

def events_to_clip_labels(n, events):
    y=np.zeros(n, dtype=np.int64)
    for s,e in events: y[s:e]=1
    return y

_idx_pat = re.compile(r"(?:_|-)(\d{1,6})(?=\D*$)")
def infer_vid_and_idx(path):
    p=str(path); stem=Path(p).stem
    m=_idx_pat.search(stem)
    if m:
        clip_idx=int(m.group(1))
        video_id=stem[:m.start()] or Path(p).parent.name
    else:
        video_id=Path(p).parent.name
        clip_idx=stem
    return video_id, clip_idx

def natkey(x):
    try: return int(x)
    except: return x

def post_metrics(probs, ys, paths):
    # baseline (sin post)
    yb=(probs>=THRESH_BASE).astype(int)
    cm_base=confusion_matrix(ys, yb, labels=[0,1])
    rep_base=classification_report(ys, yb, target_names=["normal","hurto"], digits=3)

    # agrupar por video para post-proceso temporal
    from collections import defaultdict
    buckets=defaultdict(list)
    for p,pr,yv in zip(paths, probs, ys):
        vid,idx=infer_vid_and_idx(p)
        buckets[vid].append((idx, float(pr), int(yv)))

    y_true_all=[]; y_pred_all=[]
    for vid, triples in buckets.items():
        triples.sort(key=lambda t: natkey(t[0]))
        pv=np.array([t[1] for t in triples], float)
        yv=np.array([t[2] for t in triples], int)
        n=len(pv)
        sm=moving_avg(pv, SMOOTH_K)
        evs=hysteresis_events(sm, HYST_ON, HYST_OFF, HOP_SECONDS, MIN_EVENT_SEC, COOLDOWN_SEC)
        yhat=events_to_clip_labels(n, evs)
        y_true_all.append(yv); y_pred_all.append(yhat)

    if y_true_all:
        y_true_all=np.concatenate(y_true_all)
        y_pred_all=np.concatenate(y_pred_all)
        cm_post=confusion_matrix(y_true_all, y_pred_all, labels=[0,1])
        rep_post=classification_report(y_true_all, y_pred_all, target_names=["normal","hurto"], digits=3)
    else:
        cm_post, rep_post = cm_base, rep_base  # fallback si no se pudo agrupar
    return (cm_base, rep_base), (cm_post, rep_post)

# ---------- CORRER ----------
print("CUDA:", torch.cuda.is_available())
model=build_model()

for split_name, csv_path in [("VAL", VAL_CSV), ("TEST", TEST_CSV)]:
    if not os.path.exists(csv_path):
        print(f"[{split_name}] No existe:", csv_path); continue
    ds=SimpleVideoEval(csv_path, BASE_DIR)
    dl=DataLoader(ds, batch_size=8, shuffle=False, num_workers=0)
    probs, ys, paths = infer_loader(model, dl)

    print(f"\n===== {split_name} =====")
    (cm_b, rep_b), (cm_p, rep_p) = post_metrics(probs, ys, paths)

    print("\n--- BASELINE t=0.30 (sin post) ---")
    print(cm_b); print(rep_b)

    print("\n--- (POST) smoothing + histéresis + minDur ---")
    print(cm_p); print(rep_p)


CUDA: False
Checkpoint cargado: /content/drive/MyDrive/tesisV2/models/r3d18_shoplifting_ft_l34_bestF1.pt

===== VAL =====

--- BASELINE t=0.30 (sin post) ---
[[ 93  25]
 [304 365]]
              precision    recall  f1-score   support

      normal      0.234     0.788     0.361       118
       hurto      0.936     0.546     0.689       669

    accuracy                          0.582       787
   macro avg      0.585     0.667     0.525       787
weighted avg      0.831     0.582     0.640       787


--- (POST) smoothing + histéresis + minDur ---
[[101  17]
 [302 367]]
              precision    recall  f1-score   support

      normal      0.251     0.856     0.388       118
       hurto      0.956     0.549     0.697       669

    accuracy                          0.595       787
   macro avg      0.603     0.702     0.542       787
weighted avg      0.850     0.595     0.651       787


===== TEST =====

--- BASELINE t=0.30 (sin post) ---
[[159  32]
 [169 410]]
              pre

In [ ]:
!pip -q install av==10.0.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
